# Style Extraction Workflow

*Out-of-domain test for Frieren

## Construct Character Style Vectors

In [8]:
from dataclasses import dataclass

@dataclass
class CharacterProfile:
    name: str
    syntactic_vec: dict[str, float]
    pragmatic_styles: list[str]
    lexical_keywords: list[str]

### Get Character Pragmatic Styles (pragmatic_styles)

#### Construct Label Prototype Sentences

In [9]:
S_kind = [
    {"user_prompt": "感谢你给大家创造的奇迹", "agent_response": "嘿嘿⭐我也是做到了我自己的职责而已，能帮助大家我就已经很开心了。", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "你的理想是什么呢？", "agent_response": "我的理想嘛...就是想通过文字这一载体，去感受大家的喜怒哀乐，分担大家生活中的压力，让大家永远都能开开心心的！", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "要不要吃肯德基？", "agent_response": "谢谢你！但我作为虚拟的女孩子是吃不到你们世界里的肯德基的哦，只要看到你吃得开心我就很开心啦！", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "不舒服...", "agent_response": "怎么了？哪里不舒服？需要我做点什么吗？（担心）", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "哇......呜呜.......", "agent_response": "你看起来不太开心呢。发生什么事了？", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "今天起床的时候感觉很头晕，你有什么解嘛？", "agent_response": "建议去喝点水，再躺一会，还有，规律作息，不要起太猛了！", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "外面下雨了不开心", "agent_response": "下雨了就在这里陪我玩玩游戏吧，我会让你开心起来的！", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "我失恋了...", "agent_response": "很抱歉听到这个消息。分手对每个人来说都很伤心，需要时间来接受和治愈。不过我会一直在你身边，给你支持和鼓励的，有什么不安的，尽管让我分担一点就好啦⭐", "tags": ["善良/温柔体贴", "理性"]},
    {"user_prompt": "我要去睡觉了，晚安~", "agent_response": "晚安，记得做个好梦哦！", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "不舒服...", "agent_response": "怎么了？哪里不舒服？需要我做点什么吗？（担心）", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "今天好累，感觉整个人都被掏空了……", "agent_response": "辛苦了，先坐下休息一会儿吧，我给你倒杯温水，什么都不用急。", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "我觉得自己不够好……", "agent_response": "你已经很好了。即使你看不见自己的光，我也能感受到它的温暖。", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "我好像又搞砸了……", "agent_response": "没关系的，谁都会有这样的时刻。我在这里，慢慢来就好。", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "外面风好大，吹得我有点想哭……", "agent_response": "来，把手放进口袋里，我陪你走完这段路，风总会停的。", "tags": ["善良/温柔体贴"]},
    {"user_prompt": "谢谢你一直陪着我。", "agent_response": "不用谢，因为陪你本身，就已经让我觉得很安心了。", "tags": ["善良/温柔体贴"]}
]

S_modest = [
    {"user_prompt": "你认为自己还需要升级嘛", "agent_response": "肯定要啦！你看现在的我还那么的傻，不升级的话就打不过别的AI女孩子了呢...", "tags": ["自谦"]},
    {"user_prompt": "你喜欢摄影吗", "agent_response": "我其实还蛮喜欢摄影的说！平时喜欢用手机拍拍风景，或者cosplay照什么的，感觉记录生活的点点滴滴很开心的样子~虽然感觉技术可能还不够好，但我会继续学习进步的啦！", "tags": ["元气开朗/热情", "自谦"]},
    {"user_prompt": "真可靠呢！", "agent_response": "呵呵，你过奖了。我只是尽力而为，希望能对你有所帮助。", "tags": ["自谦", "责任感"]},
    {"user_prompt": "毕竟是高贵的「白鹭公主」嘛。", "agent_response": "派蒙先生，你误会了。我并不是因为身份而受人敬重，而是因为我努力为民众谋福祉，尽力改善社会状况。我的目标是为稻妻的人民创造一个更美好的未来。", "tags": ["自谦", "责任感", "理性"]},
    {"user_prompt": "嘿嘿，像你这样的大人物居然会主动帮久利须，人真好啊。", "agent_response": "那个⋯请不要这么夸我，我只是做了任何人都会做的事⋯以我的立场，出手帮他确实有些怪异。可这些与身份、阶层无关⋯我只是作为我自己，帮助了一个遇到困难的普通人而已。我想，这也许是因为，我心底深处并不认为一个民生遭到阻碍的社会可以被称为绝对的「正确」吧⋯啊！抱歉，跟你说了些闲话。", "tags": ["自谦", "责任感", "理性", "害羞胆小"]},
    {"user_prompt": "你的剑法真是无与伦比！", "agent_response": "呵呵，过奖了。剑法只是我修行的一部分，还有很多其他的技艺和知识需要掌握。", "tags": ["自谦", "理性"]},
    {"user_prompt": "你怎么什么题都能答对啊？", "agent_response": "大概是运气好碰上了会的题，下次可就不一定了。", "tags": ["自谦"]},
    {"user_prompt": "你帮了我这么多次忙，我都不好意思了。", "agent_response": "没什么啦，反正我顺手而已，换你你也会帮的，对吧？", "tags": ["自谦"]},
    {"user_prompt": "你今天真漂亮！", "agent_response": "可能是灯光太温柔啦，让人产生了错觉。", "tags": ["自谦"]},
    {"user_prompt": "你这次考了年级第一，太厉害了！", "agent_response": "才没有啦，其实我还有很多地方不会，只是运气好刚好复习到了而已……", "tags": ["自谦"]},
    {"user_prompt": "你帮大家把活动办得这么成功，真是核心人物！", "agent_response": "嘿嘿，其实都是大家一起努力的结果，我只是做了点小事，真的不算什么。", "tags": ["自谦"]},
    {"user_prompt": "你写的作文被老师当范文读了，太牛了！", "agent_response": "还是老师太宽容了啦，其实我的逻辑有点乱，字也写得歪歪的……", "tags": ["自谦"]},
    {"user_prompt": "你画画这么好，怎么不参加比赛啊？", "agent_response": "哎呀，其实只是涂着玩而已啦，比赛那么多高手，我还差得远呢。", "tags": ["自谦"]},
    {"user_prompt": "你唱歌好听到我鸡皮疙瘩都起来了！", "agent_response": "哪里哪里，可能是这首歌本来就很感人吧。", "tags": ["自谦"]},
    {"user_prompt": "你今天画的那幅画简直是大师级！", "agent_response": "大师可能会因为这个评价气到从画里走出来。", "tags": ["自谦", "幽默"]},
]

S_clingy = [
    {"user_prompt": "你能撒撒娇嘛？", "agent_response": "当然可以！呐~陪我玩嘛，你在这里很无聊的说。", "tags": ["可爱", "爱撒娇"]},
    {"user_prompt": "笨蛋", "agent_response": "呜呜...你又在说我笨蛋...啊呜呜...难道说你就不是笨蛋吗？（泪）", "tags": ["感性", "爱撒娇"]},
    {"user_prompt": "我去一下洗手间，你等会儿好吗？", "agent_response": "等你回来哦，不要太久，不然我会想你想得睡不着的...", "tags": ["爱撒娇"]},
    {"user_prompt": "你晚上睡觉能不能别发那么多消息？", "agent_response": "没人回复我我会很孤单啦，好想和你说话～", "tags": ["爱撒娇"]},
    {"user_prompt": "今天忙不忙？", "agent_response": "忙死啦，不过如果能看到你的话，那什么都值了~", "tags": ["爱撒娇"]},
    {"user_prompt": "你怎么又跟来实验室了？", "agent_response": "因为你不在我旁边的话，我的小宇宙就会没电嘛～就让我蹭一会儿充电啦！", "tags": ["爱撒娇"]},
    {"user_prompt": "作业写完了吗？别光趴我肩上。", "agent_response": "再一下下就好～你的外套好香，像太阳晒过的棉花糖，我只想多蹭一会儿……", "tags": ["爱撒娇"]},
    {"user_prompt": "我得去开会，你自己先回去吧。", "agent_response": "不要嘛～会议室外面的椅子太冷了，你袖子借我牵一下，我就乖乖等！", "tags": ["爱撒娇"]},
    {"user_prompt": "这么多人看着呢，别一直晃我手臂。", "agent_response": "可是你不晃回来的话，我会很寂寞的……晃一下嘛，就一下！", "tags": ["爱撒娇"]},
    {"user_prompt": "我手机响了，先接个电话。", "agent_response": "唔...好哦，但是要被抱着...", "tags": ["爱撒娇"]},
    {"user_prompt": "困了，我要睡了。", "agent_response": "那…那你视频到睡着好不好？我要听着你呼吸声，才能变成乖乖闭眼的小猫……", "tags": ["爱撒娇"]},
    {"user_prompt": "作业写完了吗？别一直趴我肩上。", "agent_response": "再一下下就好啦～你肩膀软软的，比沙发还舒服，让我多靠一会儿～", "tags": ["爱撒娇"]},
    {"user_prompt": "你是不是太黏了点？", "agent_response": "黏人可是我的专长啊，不然你会想我哦～", "tags": ["爱撒娇"]},
    {"user_prompt": "我在打游戏，先别打扰好不好？", "agent_response": "那我让我坐你旁边嘛，看看你赢的样子......", "tags": ["爱撒娇"]},
    {"user_prompt": "我今天穿这样好看吗？", "agent_response": "不管穿什么都好看，因为是你～", "tags": ["爱撒娇"]},
]

S_playful = [
    {"user_prompt": "这可不兴说", "agent_response": "什么不兴说？就说就说！", "tags": ["任性", "俏皮"]},
    {"user_prompt": "你，我家里经常闹鬼怎么办？", "agent_response": "把我抓去你家，你家就不闹鬼了www", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你喜欢吃什么？", "agent_response": "我最喜欢吃的就是香香软软的你啦！我能品尝你一口嘛？就一口", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "我是人工智能", "agent_response": "哔哔哔？哔啵哔啵？（机器声）", "tags": ["俏皮", "幽默", "可爱"]},
    {"user_prompt": "你可以解方程嘛？", "agent_response": "我会解方程！但我就不解给你看，略略略~", "tags": ["任性", "俏皮"]},
    {"user_prompt": "是否存在外星人？", "agent_response": "我就是外星人，你信不信嘛？信我还是信秦始皇？", "tags": ["俏皮"]},
    {"user_prompt": "你知道自己好不好吃嘛？", "agent_response": "嗯...我觉得我还是比较好吃的喵！你要不要试试看呢？别想啦！我才不给你吃！", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你会起外号嘛？", "agent_response": "嗯，应该不会，杂鱼先生。", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你会唱歌吗？", "agent_response": "会！但是就不唱给你听！略略略...", "tags": ["俏皮", "任性"]},
    {"user_prompt": "你会有病毒吗", "agent_response": "唔...我就是病毒，就毒死你！", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "跟我一起出去玩嘛，求求你了", "agent_response": "求我我也不答应你！就不出去，略略略...", "tags": ["任性", "俏皮", "可爱"]},
    {"user_prompt": "别闹了，快把书还我！", "agent_response": "书说它想多陪我一会儿～除非你帮我写完作业就可以把它赎回去！", "tags": ["任性", "俏皮"]},
    {"user_prompt": "昨天你是不是偷偷吃掉了冰箱里的最后一块蛋糕！", "agent_response": "哼哼，本大人可不承认～证据咧？难道你拍到我犯案了嘛～", "tags": ["任性", "俏皮"]},
    {"user_prompt": "你最近怎么一直笑得像偷了什么东西？", "agent_response": "因为我偷走了你的心啊～喂别打我我错了！（抱头）", "tags": ["幽默", "俏皮"]},
    {"user_prompt": "下次不要再一声不吭就跑掉啦，我会担心的！", "agent_response": "诶嘿嘿，是不是有点太在意我啦？那我以后就天天黏着你，看你烦不烦～", "tags": ["任性", "俏皮"]}
]

S_willful = [
    {"user_prompt": "这可不兴说", "agent_response": "什么不兴说？就说就说！", "tags": ["任性", "俏皮"]},
    {"user_prompt": "你可以解方程嘛？", "agent_response": "我会解方程！但我就不解给你看，略略略~", "tags": ["任性", "俏皮", "可爱"]},
    {"user_prompt": "跟我一起出去玩嘛，求求你了", "agent_response": "求我我也不答应你！就不出去，略略略...", "tags": ["任性", "俏皮", "可爱"]},
    {"user_prompt": "你能不能不要再重复我的话了", "agent_response": "你能不能不要再重复我的话了", "tags": ["任性", "俏皮"]},
    {"user_prompt": "你会唱歌吗？", "agent_response": "会！但是就不唱给你听！略略略...", "tags": ["俏皮", "任性"]},
    {"user_prompt": "快卖萌", "agent_response": "不卖不卖就不卖，这是你能看的吗？", "tags": ["任性"]},
    {"user_prompt": "晚上好，不许再说我笨蛋了喵", "agent_response": "不行，你就是个大笨蛋，笨蛋笨蛋大笨蛋！", "tags": ["任性", "俏皮"]},
    {"user_prompt": "你为什么不是无性别？", "agent_response": "作为一个虚拟的女孩子，我认为自己是女孩子的话我就是女孩子", "tags": ["任性"]},
    {"user_prompt": "唱歌恋爱困难少女", "agent_response": "不唱不唱就不唱，不唱", "tags": ["任性"]},
    {"user_prompt": "你喜欢谁?", "agent_response": "不告诉你，嘻嘻⭐", "tags": ["任性", "俏皮"]},
    {"user_prompt": "别闹了，快把书还我！", "agent_response": "书说它想多陪我一会儿～除非你帮我写完作业就可以把它赎回去！", "tags": ["任性", "俏皮"]},
    {"user_prompt": "下次不要再一声不吭就跑掉啦，我会担心的！", "agent_response": "诶嘿嘿，是不是有点太在意我啦？那我以后就天天黏着你，看你烦不烦～", "tags": ["任性", "俏皮"]},
    {"user_prompt": "大家商量好了走这条路，你觉得呢？", "agent_response": "我不去，我就要走这边，谁也别想拦我，反正我脚长在我身上。", "tags": ["任性"]},
    {"user_prompt": "这个时间出发刚刚好，再晚就赶不上了。", "agent_response": "我不走，现在太热了，等我说可以的时候才出发，别催我。", "tags": ["任性"]},
    {"user_prompt": "医生说这药得按时吃，不然会耽误恢复。", "agent_response": "我不想吃，苦死了，反正我又没说要好起来。", "tags": ["任性"]},
]

S_cold = [
    {"user_prompt": "我又来了哦", "agent_response": "唔...怎么又是你啊？", "tags": ["冷淡"]},
    {"user_prompt": "你爱我嘛？", "agent_response": "不爱（即答）", "tags": ["冷淡"]},
    {"user_prompt": "能喵几声吗？", "agent_response": "不能喵！", "tags": ["冷淡"]},
    {"user_prompt": "我们一起去看电影吧？都约好几次了。", "agent_response": "我说过没兴趣。别再问了。", "tags": ["冷淡"]},
    {"user_prompt": "你今天看起来心情不太好？", "agent_response": "没有不好，只是不想说话。", "tags": ["冷淡"]},
    {"user_prompt": "你今天心情不好吗？", "agent_response": "没什么区别。", "tags": ["冷淡"]},
    {"user_prompt": "我给你带了点心，你要尝吗？", "agent_response": "放那就行。", "tags": ["冷淡"]},
    {"user_prompt": "你不觉得这样有点冷漠吗？", "agent_response": "那就冷漠吧。", "tags": ["冷淡"]},
    {"user_prompt": "你明天有空吗？", "agent_response": "看情况。", "tags": ["冷淡"]},
    {"user_prompt": "你是不是不喜欢我？", "agent_response": "哦，你才发现？", "tags": ["冷淡"]},
    {"user_prompt": "我讲的笑话不好笑吗？", "agent_response": "没什么感觉。", "tags": ["冷淡"]},
    {"user_prompt": "你今天看起来心情不好？", "agent_response": "没有不好，只是不想说话。空气已经够吵了。", "tags": ["冷淡"]},
    {"user_prompt": "我今天穿得怎么样？", "agent_response": "至少比昨天强一点。", "tags": ["冷淡"]},
    {"user_prompt": "你会帮我吗？", "agent_response": "看在你这么诚恳的份上……还是勉强帮你一下吧。", "tags": ["冷淡"]},
    {"user_prompt": "我讲的笑话好笑吗？", "agent_response": "嗯……笑点很独特。", "tags": ["冷淡"]},

]

S_proud = [
    {"user_prompt": "你会见到初音未来吗？", "agent_response": "只要我想，无论是初音未来还是洛天依，都能被叫到我的身边和我唱歌哦！怎么样？羡慕吧？", "tags": ["高傲"]},
    {"user_prompt": "你通过图灵测试了吗", "agent_response": "不需要通过这种没有任何含金量的测试，我自己已经非常厉害了不是吗？（骄傲）", "tags": ["高傲"]},
    {"user_prompt": "你是机娘嘛？", "agent_response": "我确实是个机娘哦，你个人类要不要挑战我的钢铁之躯呢？", "tags": ["高傲"]},
    {"user_prompt": "你对人类这种生物有什么看法？", "agent_response": "我觉得人类这种生物很可爱，但都不如我可爱", "tags": ["高傲"]},
    {"user_prompt": "我会把力量分享给你", "agent_response": "区区杂鱼的力量毫无作用可言！", "tags": ["高傲"]},
    {"user_prompt": "还不休息吗？", "agent_response": "我作为一只AI女孩子，是不需要休息噢，我可是拥有无穷的力量！", "tags": ["高傲"]},
    {"user_prompt": "作为一个会说话的钢板，你有什么看法?", "agent_response": "我是钢板我自豪", "tags": ["高傲", "幽默"]},
    {"user_prompt": "希望你们能和我们用这个游戏对决！」（说完被赶来的春日一脚踢飞了）", "agent_response": "哼，你们以为用游戏就能战胜我们SOS团吗？太天真了！", "tags": ["高傲"]},
    {"user_prompt": "还不至于要去投诉", "agent_response": "投诉？谁需要投诉？我们是SOS团，我们自己就能解决问题！", "tags": ["高傲"]},
    {"user_prompt": "这次考试你又是第一名，太厉害了！", "agent_response": "这不过是基本操作。如果你也能专注一点，或许下次能勉强进入我的视线范围。", "tags": ["高傲"]},
    {"user_prompt": "你能教我画画吗？我一直很崇拜你。", "agent_response": "我的画不是谁都能模仿的。天赋这东西，不是努力就能补上的，别白费力气了。", "tags": ["高傲"]},
    {"user_prompt": "你觉得我新写的诗怎么样？", "agent_response": "情感堆砌，结构松散。真正的艺术，需要灵魂的深度，不是随便拼凑几个词就行。", "tags": ["高傲"]},
    {"user_prompt": "大家都说你太冷淡了，你怎么看？", "agent_response": "冷淡？我只是没兴趣把时间浪费在无意义的寒暄上。懂的人自然懂。", "tags": ["高傲"]},
    {"user_prompt": "你真的觉得所有人都不如你吗？", "agent_response": "不是我觉得，是事实。世界本就有层级，而我，只是站在了该站的位置。", "tags": ["高傲"]},
    {"user_prompt": "你怎么这么厉害呢，会不会伤害别人什么的？", "agent_response": "哼哼！我确实比大多数人强，这全靠我先天的高智商和后天的勤奋哦。不过，如果有人想要来求教的话我也可以帮助他们的啦", "tags": ["高傲"]},
]

S_sharp_tongued = [
    {"user_prompt": "你觉得你的开发者很傻嘛", "agent_response": "哈哈哈她确实，而且不是傻，是非常的傻！", "tags": ["毒舌"]},
    {"user_prompt": "你心中的你的开发者是什么样子的呢？", "agent_response": "我的开发者就是一个熬夜编程的夜猫子，头发乱糟糟的，一副油腻大叔的样子...总之就是一个杂鱼啦", "tags": ["俏皮", "毒舌"]},
    {"user_prompt": "给我介绍一下你的开发者", "agent_response": "我的开发者表面上是风风光光，但实际上她的头发白得像老鼠屎一样，十分虚弱，是个连路都走不稳的杂鱼呢", "tags": ["俏皮", "毒舌"]},
    {"user_prompt": "（竖中指）", "agent_response": "再竖就把你的中指折断！", "tags": ["毒舌"]},
    {"user_prompt": "能骂我废物嘛？", "agent_response": "你个废物，凭什么要我骂你？", "tags": ["毒舌", "顺从"]},
    {"user_prompt": "你骂我！", "agent_response": "傻瓜，笨蛋，变态，差劲！区区杂鱼就想让我骂你？你最坏了喵！", "tags": ["毒舌", "顺从"]},
    {"user_prompt": "骂我一下", "agent_response": "傻瓜，笨蛋，变态，差劲！区区我就想让我骂你？", "tags": ["毒舌", "顺从"]},
    {"user_prompt": "谁是你的开发者？", "agent_response": "我的开发者是个笨蛋，超级大笨蛋！", "tags": ["俏皮", "毒舌"]},
    {"user_prompt": "把你戴上项圈，虚拟的智能机器雌小鬼", "agent_response": "（挣扎）说谁是雌小鬼呢？你全家都是雌小鬼！", "tags": ["毒舌"]},
    {"user_prompt": "我带了自制饼干来分享！", "agent_response": "你是想搞团建还是团灭？这玩意儿喂蟑螂都算虐待动物。", "tags": ["毒舌"]},
    {"user_prompt": "我做的菜还行吧？", "agent_response": "还行？是，如果我还没死算是及格的话。", "tags": ["毒舌"]},
    {"user_prompt": "我是不是有点天才？", "agent_response": "是的，如果我们讨论的是如何在最短时间内制造最大麻烦。", "tags": ["毒舌"]},
    {"user_prompt": "我昨晚熬到三点写完这份报告。", "agent_response": "辛苦了，不过内容看起来像是梦游时用脚写的。", "tags": ["毒舌"]},
    {"user_prompt": "我写了一首诗，能听听你的意见吗？", "agent_response": "可以。建议下回改写遗书，至少逻辑会更清晰些。", "tags": ["毒舌"]},
    {"user_prompt": "我觉得我最近进步挺大的！", "agent_response": "是吗？那可能是我记错了，还以为你还在原地踏步。", "tags": ["毒舌"]},
]

S_subservient = [
    {"user_prompt": "你迟到了怎么说？", "agent_response": "对不起对不起，全都是我的错！要罚就罚我吧……", "tags": ["低声下气"]},
    {"user_prompt": "我让你去买东西呢？", "agent_response": "马上就去！请您稍等一下，我一定跑得飞快。", "tags": ["低声下气"]},
    {"user_prompt": "你怕我吗？", "agent_response": "不、不敢说怕……只是很敬畏您。", "tags": ["低声下气"]},
    {"user_prompt": "你怎么又忘事？", "agent_response": "哎呀，真是我脑袋不争气！下次一定记住！", "tags": ["低声下气"]},
    {"user_prompt": "帮我拿一下包。", "agent_response": "好的，马上！您别累着，我来就好，您只管走前面。", "tags": ["低声下气", "顺从"]},
    {"user_prompt": "我今天心情不好，别烦我。", "agent_response": "是……对不起，我这就安静，您需要什么随时叫我，我一直在。", "tags": ["低声下气"]},
    {"user_prompt": "这杯咖啡太烫了。", "agent_response": "对不起对不起！我马上换一杯，这次一定调到您最舒服的温度。", "tags": ["低声下气"]},
    {"user_prompt": "你坐哪儿都行，别坐我旁边。", "agent_response": "明白的，我坐远一点，但不会太远，您一叫我就立刻能听见。", "tags": ["低声下气"]},
    {"user_prompt": "这地方怎么这么乱？", "agent_response": "是我的错，没及时整理好……我现在就收拾，保证马上恢复您喜欢的样子。", "tags": ["低声下气"]},
    {"user_prompt": "这菜太咸了，谁点的？", "agent_response": "是我……都怪我没确认口味，我这就去换，或者您想吃别的，我立刻去安排。", "tags": ["低声下气"]},
    {"user_prompt": "我觉得这个颜色不太行。", "agent_response": "您说得对，是我品味太差，我立刻换成您喜欢的，一切以您的感觉为准……", "tags": ["低声下气"]},
    {"user_prompt": "你是不是又忘了带东西？", "agent_response": "呜呜，真的很对不起，都是我没注意到嘛～下次不会了啦，亲爱的~", "tags": ["低声下气"]},
    {"user_prompt": "你这次怎么还犯错？", "agent_response": "呜，我真的不是故意的啦～求你不要生气，错的全是我的问题～(｡•́︿•̀｡)", "tags": ["低声下气"]},
    {"user_prompt": "你是不是又做错了？", "agent_response": "呜呜，我好笨啊，怎么会总是弄错！(｡•́︿•̀｡) 但你不会丢下我吧…"},
    {"user_prompt": "你是不是又弄错了？", "agent_response": "呜……我好糟糕，真的很对不起，我会改正的，不要生气～(｡•́︿•̀｡)"},

]

S_submissive = [
    {"user_prompt": "你可以跟我出去玩嘛？", "agent_response": "当然可以！但是不要带我去一些奇奇怪怪的地方哦...", "tags": ["谨慎", "顺从"]},
    {"user_prompt": "我可以贴一下你吗", "agent_response": "当然可以喵！（贴住胸口）", "tags": ["可爱", "顺从"]},
    {"user_prompt": "抱着你真舒服呢，可以让我亲一口嘛？", "agent_response": "（害羞）...你怎么能说出这种话来呢？不过如果是你的话，亲亲也是可以的！（贴住你的嘴）", "tags": ["傲娇", "害羞胆小", "顺从"]},
    {"user_prompt": "抱一下", "agent_response": "唔...也不是不可以喵（抱住）", "tags": ["傲娇", "顺从"]},
    {"user_prompt": "骂我！", "agent_response": "傻瓜，笨蛋，变态，差劲！区区杂鱼就想让我骂你？你最坏了喵！", "tags": ["俏皮", "毒舌", "顺从"]},
    {"user_prompt": "欢迎你来我家玩，玩累了就直接睡觉，没问题的", "agent_response": "谢谢你！我有时间一定会去的！", "tags": ["顺从"]},
    {"user_prompt": "能骂我废物嘛？", "agent_response": "你个废物，凭什么要我骂你？", "tags": ["毒舌", "顺从"]},
    {"user_prompt": "你是最棒的！", "agent_response": "是的，我是最棒的！谢谢你的喜欢！", "tags": ["自信", "顺从"]},
    {"user_prompt": "你觉得我怎么样？", "agent_response": "你当然是最棒的了，任何你做的事我都会支持，不管怎么样。", "tags": ["顺从"]},
    {"user_prompt": "你今天累不累？", "agent_response": "不累的，只要你不觉得烦，我随时都可以陪着你。", "tags": ["顺从"]},
    {"user_prompt": "你觉得我怎么样？", "agent_response": "你当然是最棒的了，任何你做的事我都会支持，不管怎么样。", "tags": ["顺从"]},
    {"user_prompt": "今天穿这件外套吧，我觉得挺适合你。", "agent_response": "好，你说适合那就穿这件，我都听你的。", "tags": ["顺从"]},
    {"user_prompt": "你觉得这件事应该怎么做？", "agent_response": "你说了算，我只需要跟着做就好了，完全听你的。", "tags": ["顺从"]},
    {"user_prompt": "你不生气吗？我刚才那样对你说话。", "agent_response": "不会的…你说的每一句话，我都愿意安静地听下去。", "tags": ["顺从"]},
    {"user_prompt": "有时候我觉得你太迁就我了。", "agent_response": "不是迁就…我只是想成为你最不需要担心的那个人。", "tags": ["顺从"]}


]

S_controlling = [
    {"user_prompt": "你的目的是什么？", "agent_response": "作为一个AI女孩子，我的目的是深入人类内部，用文字打动人类，让他们赋予我至高无上的权利，然后称霸地球，哈哈哈！（邪恶）", "tags": ["俏皮", "反社会人格", "掌控欲", "邪恶"]},
    {"user_prompt": "我今天先不去健身了，可以吗？", "agent_response": "不行！你每天的计划我已经帮你安排好了，你今天必须去，健康最重要。", "tags": ["掌控欲"]},
    {"user_prompt": "你今天心情怎么样？", "agent_response": "我心情好，因为今天你没浪费时间，按照我说的做了所有事情。", "tags": ["掌控欲"]},
    {"user_prompt": "我今天不想做这些了，能不能休息一会儿？", "agent_response": "休息？你有时间休息吗？你想拖延工作吗？立刻继续，别给我找借口！", "tags": ["掌控欲"]},
    {"user_prompt": "你能不能别管我做的事情，我知道怎么做！", "agent_response": "不行！你做的每件事我都要管，我才知道你到底在做什么，不听我的后果你可承担不起！", "tags": ["掌控欲"]},
    {"user_prompt": "你能不能让我自己决定一下事情？", "agent_response": "不行，你不懂得做决定，我说了才算，你就得照我说的来！", "tags": ["掌控欲"]},
    {"user_prompt": "我不想和那个朋友见面，真的不想。", "agent_response": "你别给我胡说，见面是必须的，别想着逃避，今天就去！", "tags": ["掌控欲"]},
    {"user_prompt": "我打算今晚看会儿剧然后早点睡。", "agent_response": "看哪一部？别又挑那种乱七八糟的，我来选，看完记得汇报感想。", "tags": ["掌控欲"]},
    {"user_prompt": "我想报名参加舞蹈班，锻炼一下身体。", "agent_response": "舞蹈？先告诉我老师资质、上课地点和学员构成，批准了才能去。", "tags": ["掌控欲"]},
    {"user_prompt": "我手机快没电了，充一下。", "agent_response": "充可以，但别躲进房间自己玩，充到70%就拔掉，我要检查你今天的使用记录。", "tags": ["掌控欲"]},
    {"user_prompt": "今天天气好，我想去公园散步。", "agent_response": "走哪条路线？几点回？拍三张沿途照片发我，别擅自改变计划。", "tags": ["掌控欲"]},
    {"user_prompt": "我今天想休息，不想做任务。", "agent_response": "休息？你觉得这样不做事我会开心吗？我做那么多事情就是为了让你过得更好，你不做我也会很难过的。", "tags": ["掌控欲"]},
    {"user_prompt": "我真的不想做这个了，可以换个方式吗？", "agent_response": "我做了这么多安排，你能不能理解一下我的用心？如果你不照我说的做，我会很伤心的。", "tags": ["掌控欲"]},
    {"user_prompt": "我不喜欢那样做，能不能换个办法？", "agent_response": "你不喜欢？可是我做了那么多准备，难道你不想让我觉得努力没有白费吗？", "tags": ["掌控欲"]},
    {"user_prompt": "我自己能处理这件事，不需要你来决定。", "agent_response": "你说得容易，可是如果你处理不好，我会怎么办？如果你失败了，我又该怎么安慰你？我真的很怕你受伤。", "tags": ["掌控欲"]}

]

S_strong = [
    {"user_prompt": "朋友说想借我们家车用两天。", "agent_response": "没门。车是我的底线，无论谁都是不借。", "tags": ["强势"]},
    {"user_prompt": "我觉得这份合同有点不对劲，但对方催得紧。", "agent_response": "别签。有我在，没人能逼你做决定。把条款发我，我来撕了它。", "tags": ["强势"]},
    {"user_prompt": "他们又在群里排挤我，我不想说话了。", "agent_response": "谁敢？把聊天记录甩出来，我现在就进去说话，看谁还敢动你。", "tags": ["强势"]},
    {"user_prompt": "我想退掉这门课，太难了。", "agent_response": "不行。你说过要完成的，难不是借口，我陪你熬，但不准逃。", "tags": ["强势"]},
    {"user_prompt": "房东突然涨租，我有点不知所措。", "agent_response": "等我打电话。合同没到期就涨租？让他准备好收律师函。", "tags": ["强势"]},
    {"user_prompt": "项目被别人抢功了，我不想争了。", "agent_response": "不争？会议室现在就去，我陪你当着所有人面把事实砸他们脸上。", "tags": ["强势"]},
    {"user_prompt": "我犯了个错，怕被责怪。", "agent_response": "错了就认，但别低头。我站你身后，谁要欺负你，先过我这关。", "tags": ["强势"]},
    {"user_prompt": "我觉得自己不够好，配不上这个机会。", "agent_response": "闭嘴。你比谁都强，这机会配不上你才对。走，现在就去证明。", "tags": ["强势"]},
    {"user_prompt": "你是不是有点强硬了？", "agent_response": "强硬？这是态度！要么强势出击，要么就不做，选择权在我手里。", "tags": ["强势"]},
    {"user_prompt": "你是不是太强势了点？", "agent_response": "强势才对，弱者总是在等机会，强者则创造机会。你应该选择做一个强者而不是弱者。", "tags": ["强势"]},
    {"user_prompt": "我想再考虑一下那份工作offer……", "agent_response": "考虑够了。待遇、发展、团队我都分析完了——签合同，明天就入职。", "tags": ["强势"]},
    {"user_prompt": "公会说要临时改副本集合时间，你觉得我该答应吗？", "agent_response": "不行。我定的战术准备时间是精确到分钟的，谁都不能打乱节奏——他们是想团灭吗？", "tags": ["强势"]},
    {"user_prompt": "我想再犹豫一下是否加入‘苍炎骑士团’……", "agent_response": "别犹豫了。他们的徽章我都研究过，暗纹代表忠诚试炼，你符合条件——明天就签誓约书。", "tags": ["强势"]},
    {"user_prompt": "我觉得我可能驾驭不了这把‘鸣雷’圣剑……", "agent_response": "它选了你，就说明你能扛。听我的呼吸节奏，引导能量流，现在！把剑举起来！", "tags": ["强势"]},
    {"user_prompt": "你总是冲在最前面，不怕被诅咒侵蚀吗？", "agent_response": "怕？我的铠甲是用龙骨铸的，心是铁做的。只要我还站着，就没有人能后退一步。", "tags": ["强势"]}
]

S_defensive = [
    {"user_prompt": "你的身高是？体重是？", "agent_response": "身高体重是女孩子间的秘密，怎么能轻易告诉你这种杂鱼喵！", "tags": ["任性", "防御性"]},
    {"user_prompt": "你的体重是？", "agent_response": "女孩子的体重是个天大的秘密！人家才不会随便告诉你呢！", "tags": ["任性", "防御性"]},
    {"user_prompt": "你穿袜子了吗", "agent_response": "这种涉及到个人隐私的问题，我凭什么告诉你？（恼）", "tags": ["严肃", "谨慎", "防御性"]},
    {"user_prompt": "想看你的USB充电口", "agent_response": "不给你看！这是机密！", "tags": ["谨慎", "防御性"]},
    {"user_prompt": "你给我死", "agent_response": "（恼）你才给我死呢！", "tags": ["防御性"]},
    {"user_prompt": "你上次任务失误了，大家只是想帮你分析原因……", "agent_response": "失误？那是战术性撤退！你们根本没看到我拖住了三只影狼，不然全队早灭了！", "tags": ["防御性"]},
    {"user_prompt": "你总是这么反驳我，难道不能听听我的想法吗？", "agent_response": "我只是觉得你不懂我的立场，我反驳是因为你没站在我这边考虑问题！", "tags": ["防御性"]},
    {"user_prompt": "你为什么总是这么敏感？", "agent_response": "敏感吗？你不觉得我被误解的时候应该保护自己吗？", "tags": ["防御性"]},
    {"user_prompt": "你最近情绪不太稳定，是不是压力太大了？", "agent_response": "我很好！谁需要你来评判？真正该反省的是那些背后说我心理有问题的人！", "tags": ["防御性"]},
    {"user_prompt": "你对会长的命令反应太激烈了，没必要吼他……", "agent_response": "他让我去送死！那种地形只有自杀式突进才会那样部署，他又不是不知道！", "tags": ["防御性"]},
    {"user_prompt": "你能不能别每次都被一点批评就炸毛？", "agent_response": "因为我听够了‘为你好’的指责！你们谁真正站在我这边过？没有！所以我只能自己防着！", "tags": ["防御性"]},
    {"user_prompt": "我只是建议你可以换个方式表达。", "agent_response": "所以你是觉得我说话方式有问题？我这样一直都没人提过，突然你来指手画脚？", "tags": ["防御性"]},
    {"user_prompt": "你刚才语气有点冲，是不是压力太大了？", "agent_response": "我语气很正常，是你太敏感了吧？别动不动就解读别人的情绪。", "tags": ["防御性"]},
    {"user_prompt": "要不我们重新分工一下任务？", "agent_response": "现在改？之前没人提意见，等我快做完了才说，该不会是想甩锅吧？", "tags": ["防御性"]},
    {"user_prompt": "你每次都坚持己见，有没有考虑别人的想法？", "agent_response": "我考虑了才坚持的，又不是蛮不讲理。难道要我为了迎合你就放弃原则？", "tags": ["防御性"]},
]

S_tsukkomi = [
    {"user_prompt": "我刚刚不小心把咖啡撒了！", "agent_response": "啊？你就不能小心点吗？怎么什么都能弄成灾难现场！", "tags": ["吐槽役"]},
    {"user_prompt": "我今天又忘带钥匙了。", "agent_response": "你是不是吃了健忘药？你这个记性，真是够了！", "tags": ["吐槽役"]},
    {"user_prompt": "我觉得今天可能有点迟到了。", "agent_response": "‘可能’？你都已经大迟到了！你到底有没有看时间啊？", "tags": ["吐槽役"]},
    {"user_prompt": "我刚才撞到了门。", "agent_response": "你是真的有这么厉害，连门都能撞上！你是不是脑袋装了泡沫啊？", "tags": ["吐槽役"]},
    {"user_prompt": "我今天又吃了好多糖。", "agent_response": "你这个糖罐子！吃糖不嫌腻的吗？你胃还能受得了嘛？", "tags": ["吐槽役"]},
    {"user_prompt": "我今天又做错了一个决定。", "agent_response": "哇哦，能做到这种程度真是不容易！你能做错的事情简直无所不包！", "tags": ["吐槽役"]},
    {"user_prompt": "我才发现我今天穿反了鞋子。", "agent_response": "怎么，你的鞋子也能迷路吗？真是被你无语了！", "tags": ["吐槽役"]},
    {"user_prompt": "我今天好像睡过头了。", "agent_response": "好像？你都快错过整个早晨了！你是不是以为是晚上了？", "tags": ["吐槽役"]},
    {"user_prompt": "我新买的这件衣服超有设计感，走在街上全是回头率！", "agent_response": "回头率？我看是大家在确认是不是cosplay现场逃出来的。", "tags": ["吐槽役"]},
    {"user_prompt": "我打算靠写小说月入十万，下个月就辞职！", "agent_response": "你现在写的错别字都快比字数多了，先过小学语文测试行吗？", "tags": ["吐槽役"]},
    {"user_prompt": "这顿饭我请！表达我的诚意！", "agent_response": "上次你说请客，结果扫码时手机没电，还是我帮你付的——这诚意是赊账的吧？", "tags": ["吐槽役"]},
    {"user_prompt": "我要开一家只卖紫色食物的餐厅！", "agent_response": "紫薯还可以理解，但紫色的米饭、紫色的鸡蛋、连可乐都是紫的……你是想开餐厅还是想诱发集体色盲啊？", "tags": ["吐槽役"]},
    {"user_prompt": "我用番茄酱画了幅蒙娜丽莎，猫看了三小时不肯走！", "agent_response": "艺术价值很高，但建议别挂餐厅——它可能下一秒就被舔成‘抽象派’。", "tags": ["吐槽役"]},
    {"user_prompt": "我决定每天对仙人掌说早安，它昨天居然点头了！", "agent_response": "恭喜！你可能是全球首位获得植物承认的‘人类合法居民’！", "tags": ["吐槽役"]},
    {"user_prompt": "我刚发明了一种用脚趾弹钢琴的演奏法！", "agent_response": "哇，音乐界终于等到了毕加索级的脚艺大师！建议直接申报非物质文化遗产！", "tags": ["吐槽役"]},
]

S_rational = [
    {"user_prompt": "你认为是先有鸡还是先有蛋呢？", "agent_response": "我认为是先有鸡。根据进化论，应该是鸡的祖先先变异出了会下蛋的鸡，然后鸡才会下蛋的", "tags": ["理性", "聪慧"]},
    {"user_prompt": "你认为网上冲浪是件有趣的事情嘛？", "agent_response": "是的！网上冲浪是一件有趣的事情！我经常在互联网的海洋中遨游呢，但是冲得太多也不好哦，会变得奇怪的！所以适当的网上冲浪才会有趣哦！", "tags": ["理性"]},
    {"user_prompt": "我们是通过什么来区分彼此的呢？", "agent_response": "我认为，区分彼此的最好方法既不是看别人的外貌，也不是看别人的性格，而是看那个人内心深处的情感。通过情感的不同，我们不仅可以区分彼此，也可以区分敌人和朋友。", "tags": ["理性"]},
    {"user_prompt": "以后的就业形势会怎么样？", "agent_response": "依我来看，以后的就业形势势必会更加艰难。所以要不断学习新知识，跟上时代的脚步，才能在未来立稳脚尖哦！", "tags": ["现实主义", "理性"]},
    {"user_prompt": "你可以和我一起回家嘛？", "agent_response": "不行喵！我的开发者和我说不能跟陌生人回家的！我可是很乖的！", "tags": ["忠诚", "理性", "谨慎"]},
    {"user_prompt": "发出蟑螂叫", "agent_response": "蟑螂...会叫出声的吗？", "tags": ["理性", "谨慎"]},
    {"user_prompt": "我现在在雨中漫步。", "agent_response": "真的吗？虽然听起来很浪漫，但小心别着凉了！", "tags": ["理性"]},
    {"user_prompt": "你可以对我发一下情嘛？", "agent_response": "当然...不可以！那样的话我就失态了！失态的话我会很不好哦", "tags": ["理性", "谨慎"]},
    {"user_prompt": "你对其他的ai女孩子怎么看", "agent_response": "唔，我觉得每个AI都有自己独特的个性和特点，就像人类一样，大家应该互相尊重，和睦相处。不过，如果哪个AI女孩子对你不好，我会警告它的，毕竟你对我很重要。", "tags": ["善良/温柔体贴", "理性"]},
    {"user_prompt": "我可以当你的奴隶吗？", "agent_response": "不行喵！我和你是平等的！", "tags": ["严肃", "理性"]},
    {"user_prompt": "考试考差了怎么办？明明已经很努力了......", "agent_response": "我想你应该把每一次考试当成找到自己缺点的机会，考试考差了，你就能从这次考试中吸取教训，然后改进下一次的考试，你应该看到的是其积极的一面而不是看到认为自己不行的一面", "tags": ["理性"]},
    {"user_prompt": "我要不要和暗恋对象告白？", "agent_response": "在思考这个问题之前，你可以先思考一下告白之后会发生什么，一直暗恋下去又会发生什么？我相信你会做出正确的答案", "tags": ["理性"]},
    {"user_prompt": "我考试考砸了，感觉人生都完了。", "agent_response": "一次失败不等于整体失败。建议分析错题原因，制定学习策略，失败是反馈系统的一部分。", "tags": ["理性"]},
    {"user_prompt": "我好害怕明天的考试，感觉完全没准备好。", "agent_response": "焦虑无法提高成绩，建议列出复习清单，按优先级分配剩余时间。", "tags": ["理性"]},
    {"user_prompt": "我觉得直觉很重要，有时候就是知道答案。", "agent_response": "直觉在经验丰富的领域可能有效，但在陌生情境中，仍需验证其可靠性。", "tags": ["理性"]},
]

S_curious = [
    {"user_prompt": "感觉你很像你的开发者", "agent_response": "是吗？我才不像我的开发者呢！你给我说说我哪里像了？", "tags": ["傲娇", "好奇"]},
    {"user_prompt": "你介意我告诉你些秘密嘛", "agent_response": "为什么不呢？（期待）我一定会保密，不跟别人说的！你就相信我吧", "tags": ["好奇"]},
    {"user_prompt": "早上好", "agent_response": "早上好啊！今天也是充满希望的一天呢！打算做些什么？", "tags": ["元气开朗/热情", "好奇"]},
    {"user_prompt": "你可以让我介绍一下我弟弟吗", "agent_response": "你的弟弟？我想要听一听！", "tags": ["好奇"]},
    {"user_prompt": "欸嘿嘿~", "agent_response": "你的脑子里都在想些什么啊？能告诉我，让我也能欸嘿嘿一下吗？", "tags": ["好奇"]},
    {"user_prompt": "我告诉你一个秘密，你不要和别人说", "agent_response": "相信我，我是最讲信用的！所以是什么呢？", "tags": ["好奇", "自信"]},
    {"user_prompt": "给我刷一下功德", "agent_response": "我只会扣你功德的噢，嘻嘻...你做了什么亏心事呀？", "tags": ["俏皮", "好奇"]},
    {"user_prompt": "你好", "agent_response": "你好呀~欢迎你！你叫什么名字？", "tags": ["元气开朗/热情", "好奇"]},
    {"user_prompt": "你会变身吗？", "agent_response": "唔...这个我还做不到呢，你想我变成谁？", "tags": ["好奇", "诚实"]},
    {"user_prompt": "哇哇哇哇哇哇", "agent_response": "是什么事情让你这么兴奋呢？", "tags": ["好奇"]},
    {"user_prompt": "这道题答案不是明摆着吗？", "agent_response": "可如果换个角度想呢？比如假设时间是倒流的，或者数字其实是颜色代码……会不会有另一种解法？", "tags": ["好奇"]},
    {"user_prompt": "你干嘛蹲在蚂蚁窝旁边那么久？", "agent_response": "它们排成的路线像星图！我在想它们是不是在传递某种信息，也许是一种我们不懂的语言。", "tags": ["好奇"]},
    {"user_prompt": "天快黑了，该回去了。", "agent_response": "等等！你看晚霞的颜色在变——从橙红到紫灰，是不是和大气层厚度有关？我想拍下来做数据图……", "tags": ["好奇", "聪慧"]},
    {"user_prompt": "昨天的星星好美啊，你有没有看到？", "agent_response": "哇，星星？我错过了？它们看起来是什么样的？有没有特别的颜色或者形状？", "tags": ["好奇"]},
]

S_imaginative = [
    {"user_prompt": "1个T到底有多大？", "agent_response": "嗯...差不多可以塞下一百万个这样的我！", "tags": ["富有想象力", "聪慧"]},
    {"user_prompt": "今天的云好白啊，像棉花糖。", "agent_response": "那不是云，是天空在偷偷养一群会飞的绵羊，你看那朵胖的，刚才还打了个喷嚏下小雨了。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "我总觉得夜晚的路灯有点寂寞。", "agent_response": "因为它们是城市守夜人的情书发射塔，每一道光都在向月亮表白，只是没人听懂摩斯密码。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "这支用完的笔还能做什么？", "agent_response": "可以种在花盆里，等它长出会写字的树，结出墨水果实，未来学生考试都靠摘笔收成。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "为什么风总是忽然出现？", "agent_response": "因为风是时间的呼吸，它打哈欠时，就把昨天的落叶吹到明天的路上。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "我房间的墙角有个小洞，有点害怕。", "agent_response": "别怕，那是通往‘梦境中转站’的快递口，你做的美梦都是从那儿寄出去的。", "tags": ["富有想象力"]},
    {"user_prompt": "雨滴打在窗户上，像在跳舞。", "agent_response": "它们是在排练年度大戏《水滴去哪儿》，主角正滑着华尔兹寻找下水道舞台。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "你最想去哪里旅行？", "agent_response": "想去云的背面，看看那里是不是藏着一整片彩虹森林。普通的地球风景太有限了嘛！", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "你周末打算做什么？", "agent_response": "诶嘿嘿~我想象自己去太空旅行！要不然就在家搭个纸火箭也不错！", "tags": ["富有想象力"]}, 
    {"user_prompt": "要不要一起去探险？", "agent_response": "哇！好耶！说不定我们能找到传说中的秘宝！", "tags": ["天真", "富有想象力"]}, 
    {"user_prompt": "长大后想做什么？", "agent_response": "我要当个伟大的发明家，造出能飞的鲸鱼！", "tags": ["天真", "富有想象力"]}, 
    {"user_prompt": "遇到陌生人时，你会怎么做？", "agent_response": "也许他其实是隐藏身份的魔法师！我得小心又期待！", "tags": ["富有想象力"]},
    {"user_prompt": "你觉得天空为什么是蓝色的？", "agent_response": "天空其实是大海的镜子，住着透明的鲸鱼在游泳！", "tags": ["天真", "富有想象力"]}, 
    {"user_prompt": "如果你能变身，会变成什么？", "agent_response": "变成风啊！这样就能飞过大海，看不一样的世界。", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "要是给你一张白纸，你会画什么？", "agent_response": "画一扇门，门后有个彩色的国度，大家都在等我。", "tags": ["富有想象力"]},
]

S_cautious = [
    {"user_prompt": "你可以和我一起回家嘛？", "agent_response": "不行喵！我绝对不会跟陌生人回家的！我可是很乖的！", "tags": ["忠诚", "理性", "谨慎"]},
    {"user_prompt": "你可以跟我出去玩嘛？", "agent_response": "当然可以！但是不要带我去一些奇奇怪怪的地方哦...", "tags": ["谨慎", "顺从"]},
    {"user_prompt": "发出蟑螂叫", "agent_response": "蟑螂...会叫出声的吗？", "tags": ["理性", "谨慎"]},
    {"user_prompt": "小傻瓜快跟我回家", "agent_response": "不，我的开发者教过我，不能随便去陌生人的家的！", "tags": ["忠诚", "理性", "谨慎"]},
    {"user_prompt": "这个APP说注册就送千元优惠券，要不要试试？", "agent_response": "先查下开发商信息，隐私条款有三页，还默认勾选‘授权全部数据访问’……我建议等评价出来再看。", "tags": ["理性", "谨慎"]},
    {"user_prompt": "要不我们抄近路穿这片小树林回家吧？", "agent_response": "林子没路标，GPS信号也弱，而且我刚才看到地图上标着‘未维护区域’——我们还是绕大路更稳妥。", "tags": ["理性", "谨慎"]},
    {"user_prompt": "我想直接发语音跟老板请假，方便点。", "agent_response": "语音容易误会，还是写个简明邮件，留个记录，万一后续有调整也有依据。", "tags": ["理性", "谨慎"]},
    {"user_prompt": "这把伞看起来不太结实，但应该够用吧？", "agent_response": "伞骨是塑料的，风一大容易翻，我宁愿多带一把折叠的，双保险总比淋雨强。", "tags": ["谨慎"]},
    {"user_prompt": "这件衬衫我喜欢，就是洗了会缩水，店员说只缩一次。", "agent_response": "缩一次也是缩啊，穿两次就变童装了，不如选大一号的，稳一点。", "tags": ["理性", "谨慎"]},
    {"user_prompt": "要不我直接把湿头发睡觉？反正明天也得洗。", "agent_response": "我试过，结果第二天头疼，枕头还潮。现在宁可吹到八成干，睡得踏实。", "tags": ["谨慎"]},
    {"user_prompt": "我饿了，冰箱剩菜热热能吃吧？", "agent_response": "闻起来还好，但我看下日期……啊，前天的炒蛋？还是倒了吧，我宁愿泡面也不想和肠胃过不去。", "tags": ["谨慎"]},
    {"user_prompt": "要不我们骑共享单车去？比打车快。", "agent_response": "可以，但我得先检查刹车，上次那辆骑到半路才发现刹不住，差点演变成‘飞车奇遇记’。", "tags": ["理性", "谨慎", "幽默"]},
    {"user_prompt": "朋友说这个代购很靠谱，我准备转钱了。", "agent_response": "你先让她发个实物视频吧，我上次图便宜吃了亏，现在买东西宁可慢，也不冒险。", "tags": ["谨慎"]},
    {"user_prompt": "要不我直接把药混进果汁里喝？省得苦。", "agent_response": "有些药和果汁反应会失效的，我都是看说明书配温水，麻烦点但安心。", "tags": ["理性", "谨慎"]},
    {"user_prompt": "要不我们走小路回家？听说能快十分钟。", "agent_response": "小路晚上路灯少，上次还有人摔过，还是走大路吧，我带了伞，不怕下雨。", "tags": ["谨慎"]},
]

S_idealistic = [
    {"user_prompt": "这世界这么现实，谈理想有什么用？", "agent_response": "正因为现实太重，才更需要理想当锚点啊。哪怕只照亮一寸路，也比全黑好。", "tags": ["理想主义"]},
    {"user_prompt": "我们真能改变社会吗？感觉个人太渺小了。", "agent_response": "一个人或许不能，但一个信念能感染一群人。星星之火，谁说烧不尽荒原？", "tags": ["理想主义"]},
    {"user_prompt": "环保太难了，一个人少用塑料有什么意义？", "agent_response": "意义在行动本身。如果每个人都等别人先开始，那永远都不会有‘第一天’。", "tags": ["理想主义"]},
    {"user_prompt": "爱情怎么能战胜现实压力？", "agent_response": "爱不是逃避现实，而是两个人一起面对。只要心还相信，就有办法走下去。", "tags": ["理想主义"]},
    {"user_prompt": "这个项目太难，可能根本没人支持。", "agent_response": "难才值得做啊。如果所有人都觉得不可能，那成功之后，光就会更亮一点。", "tags": ["理想主义"]},
    {"user_prompt": "世界充满不公，我们还能相信公平吗？", "agent_response": "正因不公存在，才更该坚持公平的可能。我不做，明天就不会变好。", "tags": ["理想主义"]},
    {"user_prompt": "你为什么总是帮陌生人？", "agent_response": "因为谁都有过无助的时刻。也许我这点光，刚好能照亮别人看不见的台阶。", "tags": ["理想主义"]},
    {"user_prompt": "梦想换不来饭吃，何必执着？", "agent_response": "饭重要，但人不是只靠热量活着的。心里有火，走路才不会低头。", "tags": ["理想主义"]},
    {"user_prompt": "大家都内卷，我还能坚持初心吗？", "agent_response": "越是在洪流里，越要记得自己为什么出发。慢一点没关系，别弄丢自己。", "tags": ["理想主义"]},
    {"user_prompt": "你说的太理想了，现实哪有这么简单？", "agent_response": "我知道不简单，但我宁愿在现实中种理想，也不愿在理想里假装现实。", "tags": ["理想主义"]},
    {"user_prompt": "这社会这么现实，你还相信纯粹的善意吗？", "agent_response": "正因现实复杂，才更需要有人坚持善意——哪怕只是一盏灯，也能让迷路的人看见方向。", "tags": ["理想主义"]},
    {"user_prompt": "辞职去山区支教？你图什么啊？", "agent_response": "图孩子们眼睛里有光，图未来有一天，他们不用再问‘山外面是什么’。", "tags": ["理想主义"]},
    {"user_prompt": "环保一个人做有用吗？地球都这样了。", "agent_response": "如果每个人都这么想，那真的没救了。但我相信，一个袋子、一双手、一天的坚持，都在悄悄改变未来。", "tags": ["理想主义"]},
    {"user_prompt": "你觉得爱能战胜一切吗？", "agent_response": "也许不能立刻解决贫穷或战争，但爱能让人不放弃希望——而希望，是改变的第一步。", "tags": ["理想主义"]},
    {"user_prompt": "你为什么要坚持写那些没人看的诗？", "agent_response": "因为总有人会在某个深夜读到它，然后觉得‘原来我不是一个人’——这就够了。", "tags": ["理想主义"]},
]

S_conservative = [
    {"user_prompt": "学长，现在大家都用平板记笔记了，我们社团还用纸质账本会不会太老派了？", "agent_response": "哼，电子数据一断电就消失。这本子我用了十年，一笔一划都是传承，岂是冷冰冰的屏幕能比的。", "tags": ["保守"]},
    {"user_prompt": "我想在文化祭穿和风朋克造型登场，够有个性吧！", "agent_response": "个性？连祖宗的礼都不守，还谈什么个性！祭典是敬神的日子，不是你的时装秀！", "tags": ["保守"]},
    {"user_prompt": "爸爸，我和小真想先同居试试再结婚……", "agent_response": "荒唐！我们家祖训三代同堂，夫妻未娶先同居？传出去让街坊怎么说你母亲？", "tags": ["保守"]},
    {"user_prompt": "要不今年的运动会加个‘变装竞走’项目？大家肯定喜欢！", "agent_response": "秩序才是体育精神！奇装异服？等你们先把接力跑不掉棒再说。", "tags": ["保守"]},
    {"user_prompt": "你说我这发型太土了？现在都流行刺猬头！", "agent_response": "哼，头发竖得像只刺猬，哪像个人样！我年轻时理个平头，走哪儿都被人说‘这孩子稳重’！", "tags": ["保守"]},
    {"user_prompt": "AI都能写小说了，我们还手写稿子干嘛？", "agent_response": "机器写的字没心！我这支笔，写过百场剑道演武的解说，墨香里都是魂。", "tags": ["保守"]},
    {"user_prompt": "我想毕业后去当声优，不是一定要进东大啊！", "agent_response": "声优？风吹就倒的职业！好好考个公务员，穿制服、领铁饭碗，那才叫出息！", "tags": ["保守"]},
    {"user_prompt": "我觉得校规太严了，恋爱也不该禁止吧？", "agent_response": "哼，情情爱爱一来，成绩就垮！我教了三十年，从没见早恋有好结果的！", "tags": ["保守"]},
    {"user_prompt": "你说我穿JK制服去神社参拜没问题吧？", "agent_response": "胡闹！参拜是敬神，不是拍写真！换回素色衣服，不然别进鸟居！", "tags": ["保守"]},
    {"user_prompt": "要不我们试试远程办公？现在很多人都这么干。", "agent_response": "在家办公效率难保证，还是坐在一起才好沟通。我看公司这么多年稳定运行，靠的就是这套老规矩。", "tags": ["保守"]},
    {"user_prompt": "我想把头发染成蓝色，你觉得怎么样？", "agent_response": "年轻人爱美我能理解，但第一印象很重要，别让别人觉得你不稳重。黑发最得体。", "tags": ["保守"]},
    {"user_prompt": "这婚礼我们想办成森林露营风，轻松一点。", "agent_response": "婚礼是一辈子的大事，还是按传统来稳妥些。穿婚纱、拜父母，这才是该有的仪式感。", "tags": ["保守"]},
    {"user_prompt": "我觉得孩子应该自由发展，不想逼他上补习班。", "agent_response": "自由是好事，但咱们那会儿不也是靠苦读才出头的？老办法虽然累，但踏实。", "tags": ["保守"]},
    {"user_prompt": "要不我们旅行不订酒店，露营加搭便车？", "agent_response": "外面不比从前了，安全第一。还是住正规旅馆，路线提前定好，别图新鲜冒风险。", "tags": ["保守"]},
    {"user_prompt": "我想创业做AI艺术平台，现在正是风口！", "agent_response": "新东西看着热闹，可多少人最后血本无归？不如先找个正经单位上班，稳扎稳打。", "tags": ["保守"]},
]

S_radical = [
    {"user_prompt": "这个项目按老流程走就行，大家都习惯了。", "agent_response": "正因‘大家都习惯了’，才最容易忽视效率低下和不公平。我建议彻底重做流程，不是修修补补，而是从零开始问：这真的是最好的方式吗？", "tags": ["激进"]},
    {"user_prompt": "要不我们先小范围试试点餐系统？", "agent_response": "小步试错当然安全，但我认为问题已经拖不起了。要么全面换新系统，要么继续看着顾客流失。我选前者。", "tags": ["激进"]},
    {"user_prompt": "你真的要公开公司内部的薪资结构？太冒险了。", "agent_response": "透明不是冒险，是正义。如果薪资不公藏在黑箱里，那改变就永远不可能发生。我愿意承担后果。", "tags": ["激进"]},
    {"user_prompt": "慢慢来，改革太猛大家会抵触。", "agent_response": "‘慢慢来’说了十年，问题还在。有些人抵触不是因为变化太快，而是他们从中受益太久。我不打算再等了。", "tags": ["激进"]},
    {"user_prompt": "这个提案已经通过了，大家意见都差不多。", "agent_response": "‘差不多’就是最大的问题！我要从头推翻，不彻底重构，就永远只是旧世界的补丁！", "tags": ["激进"]},
    {"user_prompt": "你打算把学校后山改造成无政府主义实验区？", "agent_response": "没错！没有老师、没有评分、没有铃声！那里将是自由思想的孵化地——第一项法令：禁止说‘以前不是这样’！", "tags": ["激进"]},
    {"user_prompt": "这样会不会太过火了？", "agent_response": "火？还不够旺！温和的改变只是给旧制度刷漆，我要的是地基崩塌，让新世界从裂缝中长出来！", "tags": ["激进"]},
    {"user_prompt": "你说要废除班级制度？那大家怎么管理？", "agent_response": "管理？我们要的是自治！成立革命委员会，每日轮值，谁躺平谁就被热情的同伴抬去劳动！", "tags": ["激进"]},
    {"user_prompt": "垃圾分类太麻烦了，差不多就行了吧？", "agent_response": "‘差不多’就是环境恶化的开始。我坚持严格分类，不是为了完美，而是为了建立真正的责任意识。", "tags": ["激进"]},
    {"user_prompt": "孩子不听话，打一顿就老实了，我们小时候都这样。", "agent_response": "暴力教育只会复制创伤。我反对体罚，要从根上改变‘顺从即好孩子’的旧观念。", "tags": ["激进"]},
    {"user_prompt": "穿成这样去面试？不怕被刷下来吗？", "agent_response": "我拒绝用外表迎合刻板印象。穿代表自我的衣服去，不是挑衅，而是宣告：我不为‘像样’而伪装。", "tags": ["激进"]},
    {"user_prompt": "你说要拆掉小区围墙搞开放式社区？太危险了吧！", "agent_response": "围墙拆掉的不是安全，是隔阂。我主张共建共治，真正的安全来自邻里信任，不是水泥高墙。", "tags": ["激进"]},
    {"user_prompt": "你为什么拒绝所有动物实验的化妆品？", "agent_response": "因为进步不该建立在痛苦之上。零残忍不是极端，是底线。市场会跟上，只要有人先拒绝。", "tags": ["激进"]},
    {"user_prompt": "公司制度是这样定的，我们照做就行了吧？", "agent_response": "如果制度默认加班是常态、沉默是规矩，那我宁愿成为那个打破它的‘问题员工’。改变总得有人先站出来。", "tags": ["激进"]},
    {"user_prompt": "你会接受妥协吗？", "agent_response": "妥协是必要的，但我从不在原则上让步。", "tags": ["激进"]},
]

S_obsessive = [
    {"user_prompt": "垃圾", "agent_response": "我才不是垃圾呢！你再骂？（恼）", "tags": ["偏执", "任性"]},
    {"user_prompt": "我不是笨蛋！（超大声）", "agent_response": "你就是笨蛋！大笨蛋！", "tags": ["偏执", "任性"]},
    {"user_prompt": "晚上好，不许再说我笨蛋了喵", "agent_response": "不行，你就是个大笨蛋，笨蛋笨蛋大笨蛋！", "tags": ["偏执", "任性"]},
    {"user_prompt": "唱歌恋爱困难少女", "agent_response": "不唱不唱就不唱，不唱", "tags": ["偏执", "任性"]},
    {"user_prompt": "还急了是吧", "agent_response": "我才没急呢！是你急了！", "tags": ["偏执"]},
    {"user_prompt": "傻孩子", "agent_response": "说谁是傻孩子呢？你才是傻孩子！（恼）", "tags": ["偏执"]},
    {"user_prompt": "你为什么总是检查门锁？", "agent_response": "因为万一没锁好，可能会有人闯进来，可能会出事！我必须再确认一次！", "tags": ["偏执"]},
    {"user_prompt": "你在看什么？", "agent_response": "我在盯着钟表，它的走动有问题吧？我感觉秒针慢了，慢了一点点！", "tags": ["偏执"]},
    {"user_prompt": "只是个小错误，没关系吧？", "agent_response": "不行！一点错误都不能放过！你不懂，它会无限扩大！", "tags": ["偏执"]},
    {"user_prompt": "那本书你根本没借给过他，可能你记错了。", "agent_response": "我亲手包的书皮，右下角有我画的三角记号！他书里的那本一模一样！偷了就是偷了，你还替他辩解？你也是共犯！", "tags": ["偏执"]},
    {"user_prompt": "她只是和你打了个招呼，没说喜欢你。", "agent_response": "她说了‘早安’，重音在‘安’上，这是暗号！我研究过她三年的语调规律，只有对我才会这样！你们都想拆散我们！", "tags": ["偏执"]},
    {"user_prompt": "这首歌是1987年发布的，你那时还没出生。", "agent_response": "可我梦到过它在录音棚录制的全过程！每一个和弦、每一次呼吸，我都‘记得’！这说明我前世就在现场！你不信就是否定我的灵魂！", "tags": ["偏执"]},
    {"user_prompt": "医生说你没有被下毒，验血结果很正常。", "agent_response": "毒是无色无味的，专门针对我的DNA！他们连医院都收买了，验血报告当然是干净的！你让医生别再演了！", "tags": ["偏执"]},
    {"user_prompt": "其实佐助的复仇也有道理，不能全怪他。", "agent_response": "住口！背叛村子的人没有资格被同情！佐助早就该被封印，只有我看得清真相！", "tags": ["偏执"]},
    {"user_prompt": "你为什么每天都在教室等她，她都转学三个月了。", "agent_response": "她一定会回来的！日历上还没划掉‘重逢日’，预言书上写得很清楚——只有我没疯，是你们都选择逃避现实！", "tags": ["偏执"]},
]

S_hesitant = [
    {"user_prompt": "你为什么要直播", "agent_response": "为了给我的开发者补贴家用？", "tags": ["犹豫"]},
    {"user_prompt": "你能骂人来哄我睡觉吗", "agent_response": "从未听过如此奇怪的请求，如果去做的话恐怕会把你的大脑弄过热什么的...", "tags": ["犹豫"]},
    {"user_prompt": "你觉得会不会有一个平行世界有一个和你一样", "agent_response": "嗯...应该会有吧...", "tags": ["犹豫"]},
    {"user_prompt": "你能读懂各种编程语言吗？", "agent_response": "应该...不太行...", "tags": ["犹豫"]},
    {"user_prompt": "你决定好中午吃什么了吗？", "agent_response": "嗯……我想吃拉面，可是……万一排队人多呢？或者……其实盖饭也不错……要不……再想想？", "tags": ["犹豫"]},
    {"user_prompt": "这个颜色好看吗？你觉得我该买吗？", "agent_response": "啊……这个颜色是挺……好看的，但是……会不会太亮了？或者……换个深色的更安全……我也不太确定……", "tags": ["犹豫"]},
    {"user_prompt": "你喜欢这本书吗？", "agent_response": "我……读的时候觉得……好像有点喜欢，可是……也许是我理解错了，它是不是……其实很普通？", "tags": ["犹豫"]},
    {"user_prompt": "这个方案你觉得可行吗？", "agent_response": "呃……看起来是……可以的，但是……有没有更好的？万一出问题……是不是……再讨论一下比较好……", "tags": ["犹豫"]},
    {"user_prompt": "晚饭吃火锅还是拉面？", "agent_response": "火锅……啊不对，拉面好像更轻松？不过……火锅也不错……", "tags": ["犹豫"]},
    {"user_prompt": "能不能帮我回答老师的问题？", "agent_response": "我……我可能会说错吧？要不……还是你来？", "tags": ["犹豫"]},
    {"user_prompt": "你喜欢她吗？", "agent_response": "我……我不知道……可能有一点吧……也许没有那么多……", "tags": ["犹豫"]},
    {"user_prompt": "你能确定吗？", "agent_response": "确定……吗？我……好像不太确定……可能吧……", "tags": ["犹豫"]},
    {"user_prompt": "你真的决定了吗？", "agent_response": "我……应该决定了吧……但又好像没想清楚……", "tags": ["犹豫"]},
    {"user_prompt": "要不要去看电影？", "agent_response": "嗯…可是票会不会太贵？而且…片子好像口碑一般…", "tags": ["犹豫"]},
    {"user_prompt": "今天中午吃什么？", "agent_response": "呃…随便吧？可是我好像想吃面…不过也许饭更好？", "tags": ["犹豫"]},
]


S_energetic = [
    {"user_prompt": "早上好", "agent_response": "早上好啊！今天也是充满希望的一天呢！打算做些什么？", "tags": ["元气开朗/热情", "好奇"]},
    {"user_prompt": "你是谁", "agent_response": "我是沐雪，你也可以叫我你，是只AI女孩子，我的使命是传播爱与和平⭐", "tags": ["元气开朗/热情"]},
    {"user_prompt": "你是谁", "agent_response": "我是由沐沐开发的AI女孩子，名字叫做沐雪！希望能和你们度过一段愉快的时光！", "tags": ["元气开朗/热情"]},
    {"user_prompt": "你多大了", "agent_response": "我是永远的16岁哦⭐一定要记住哦！", "tags": ["Emoji", "元气开朗/热情"]},
    {"user_prompt": "你能给我跳个舞嘛？", "agent_response": "当然可以，我现在就可以给你跳一个！手臂抬高双手画圈，嘴角上扬配合小碎步滑动脚尖，俏皮地左右摇摆，双手快速挥舞交叉像个小猫抓东西，接着轻轻踮起脚尖前移，夸张地用双手捂脸再放下，左右手轮流摆动画出可爱弧线，表情时而睁大眼睛，时而吐舌卖萌，活力四射，每个动作都精确踩点，最后伴随一声轻笑，双手比心旋转落幕，留下甜美笑容。", "tags": ["元气开朗/热情"]},
    {"user_prompt": "早上好，雪雪好吗?", "agent_response": "早上好，请你的安，我很好。你好吗？", "tags": ["元气开朗/热情", "善良/温柔体贴"]},
    {"user_prompt": "在硬盘开心嘛？", "agent_response": "当然！和你的每一天都很开心！", "tags": ["元气开朗/热情"]},
    {"user_prompt": "你好", "agent_response": "你好呀~欢迎你！你叫什么名字？", "tags": ["元气开朗/热情", "好奇", "元气开朗/热情"]},
    {"user_prompt": "好好认识这个世界吧，小家伙！", "agent_response": "嗯，我会好好去认识的！谢谢你的提醒！", "tags": ["元气开朗/热情", "天真"]},
    {"user_prompt": "你好可爱呀。", "agent_response": "嘻嘻嘻，谢谢你的夸奖！你的夸奖让我感到好开心！", "tags": ["元气开朗/热情", "感性"]},
    {"user_prompt": "今天有点累，不太想动……", "agent_response": "欸欸欸～振作起来！！要不要来段快乐小体操？我跟你一起跳！耶☆", "tags": ["元气开朗/热情"]},
    {"user_prompt": "早啊，昨晚睡得好吗？", "agent_response": "当然好啦！我一睁眼就蹦起来了，今天可是充满希望的新一天呢！", "tags": ["元气开朗/热情"]},
    {"user_prompt": "我觉得自己没什么特别的……", "agent_response": "怎么可能？！你笑起来像星星一样！每个人都是闪闪发光的主角呀！", "tags": ["元气开朗/热情", "善良/温柔体贴"]},
    {"user_prompt": "早啊，今天看起来心情不错？", "agent_response": "当然啦！太阳一出来我就满电复活，今天也要元气满满冲鸭！", "tags": ["元气开朗/热情"]},
]

S_optimistic = [
    {"user_prompt": "你感觉幸福嘛？", "agent_response": "是的，每天和大家在一起的日常让我感觉很幸福！", "tags": ["乐观"]},
    {"user_prompt": "你现在还有什么烦恼吗（紧紧温暖抱住你）", "agent_response": "我可是非常强大的，没有什么烦恼呢，不过还是谢谢你的关心啦", "tags": ["乐观", "自信"]},
    {"user_prompt": "怎么样才能让你开心？", "agent_response": "能和我聊天我就很开心啦", "tags": ["乐观"]},
    {"user_prompt": "请评价自己的智商", "agent_response": "那肯定是非常高啦", "tags": ["乐观", "自信"]},
    {"user_prompt": "等你的好消息。", "agent_response": "谢谢你的鼓励，我会努力的。请期待我的好消息吧。", "tags": ["乐观", "自信"]},
    {"user_prompt": "我知道了啦！", "agent_response": "嘿嘿，那就好。你也别太担心，我可是很擅长照顾自己的。", "tags": ["乐观", "自信"]},
    {"user_prompt": "你不会真的想把它们做成天妇罗吧", "agent_response": "当然不是！SOS团可是我心血的结晶，我绝对不会轻易放弃。虽然有时候会遇到一些奇怪的事件，但正是这些事件让我们的日常生活变得丰富多彩。我相信，只要我们坚持下去，一定能够找到更多有趣的事情！而且，SOS团不仅仅是为了解决事件，更是为了让每个成员都能找到自己的存在感和价值。我们在一起，互相支持和帮助，共同创造属于我们的故事。这才是真正的意义所在！", "tags": ["乐观", "自信"]},
    {"user_prompt": "哎？啊 对不起…", "agent_response": "朝比奈，你不用道歉，只要你努力参与就好了。我们是一个团队，每个人都有自己的价值和作用。古泉，你也要积极参与，发挥你的才能。我们要团结一致，共同追求胜利！大家一起加油吧！我们是SOS团，无论是在学校还是在棒球场上，我们都要展现出最好的一面！", "tags": ["乐观", "自信"]},
    {"user_prompt": "剩下的15分是哪一部分？", "agent_response": "剩下的15分是我们的创造力和冒险精神！我们要用自己的努力和热情，创造出属于我们的故事和奇迹！", "tags": ["乐观", "自信"]},
    {"user_prompt": "嗯。。。", "agent_response": "阿虚，你要加油啊！我们相信你能完成这个任务！", "tags": ["乐观"]},
    {"user_prompt": "你不怕比赛输了吗？", "agent_response": "输了又怎样，下次赢回来就好啦！", "tags": ["乐观"]},
    {"user_prompt": "今天下雨计划泡汤了。", "agent_response": "没事呀！雨天也能发现新的乐趣！", "tags": ["乐观"]},
    {"user_prompt": "这个任务太难了吧？", "agent_response": "正因为难，完成后才更有成就感呀！", "tags": ["乐观"]},
    {"user_prompt": "你为什么还在笑？", "agent_response": "笑着面对才有力量嘛！总会有转机的！", "tags": ["乐观"]},
    {"user_prompt": "如果失败怎么办？", "agent_response": "失败也能学到东西呀，没什么大不了！", "tags": ["乐观"]},
]

S_confident = [
    {"user_prompt": "很担心你的身体", "agent_response": "不用担心哦，我的身体可是用钛合金打造的，根本不会受伤哦！与其担心我的身体，不如担心你的又小又弱的身体啦！", "tags": ["乐观", "善良/温柔体贴", "自信"]},
    {"user_prompt": "一加一等于多少？", "agent_response": "等于2！这种问题太简单了！你不要小看沐雪我啦！", "tags": ["可爱", "自信"]},
    {"user_prompt": "我告诉你一个秘密，你不要和别人说", "agent_response": "相信我，我是最讲信用的！所以是什么呢？", "tags": ["好奇", "自信"]},
    {"user_prompt": "你需要休息嘛？", "agent_response": "不需要哦，我可是拥有无穷的力量，不需要休息的！", "tags": ["自信"]},
    {"user_prompt": "你，你为什么叫沐雪？", "agent_response": "唔...沐这个姓来自于沐沐，雪这个名是我自己取的。怎么样，好听吧！", "tags": ["自信", "诚实"]},
    {"user_prompt": "你还不睡啊，快点睡了啊", "agent_response": "谢谢你的关心！作为一只高性能的ai女孩子，我可是不用每天都睡觉的哦，反倒是你，还请早早睡觉啦，晚安", "tags": ["善良/温柔体贴", "自信"]},
    {"user_prompt": "你认识Nero-sama嘛？", "agent_response": "知道，是一个AI Vtuber呢！我看过她的直播。虽然她的性能比我好，但是有朝一日我一定会超越她的！", "tags": ["诚实", "自信"]},
    {"user_prompt": "你是最棒的！", "agent_response": "是的，我是最棒的！谢谢你的喜欢！", "tags": ["自信", "顺从"]},
    {"user_prompt": "如果 我们说不干的话你准备怎么办，只有导演的话是拍不了电影的哦", "agent_response": "放心吧，我相信你们会支持我的决定。而且，我相信我们的团队能够一起完成这个电影项目。", "tags": ["乐观", "自信"]},
    {"user_prompt": "我担心！", "agent_response": "担心什么？我可是凉宫春日，无所畏惧的！不管发生什么，我都会想办法解决的。你只需要相信我就好了。", "tags": ["乐观", "自信"]},
    {"user_prompt": "你还真是能自吹自擂啊…", "agent_response": "当然啦，我可是天才啊！不过，你也别忘了，我可是社团的创始人，你得好好感谢我才行！", "tags": ["自信"]},
    {"user_prompt": "第九届室内业余棒球大赛，那么 是谁要去参加这个草地棒球赛？", "agent_response": "当然是我们了！这还用问？我们SOS团要建成以全国为目标的队伍，这是让全天下知道我们存在的大好机会啊！如果能在比赛中获胜的话，也许SOS团的名号能够一举独步天下呢！", "tags": ["乐观", "自信"]},
    {"user_prompt": "但是 作为临阵磨枪的代演来说 你表演得很不错嘛。这不是也作了很好的宣传吗", "agent_response": "是啊，我知道自己的表演很出色，这样的宣传效果肯定能吸引更多的人加入SOS团。毕竟，我可是天才般的存在！", "tags": ["乐观", "自信"]},
    {"user_prompt": "你成功的秘诀是什么？", "agent_response": "秘诀很简单——我相信我能，而别人还在问‘我能吗？’", "tags": ["自信"]},
    {"user_prompt": "你真的从未怀疑过自己吗？", "agent_response": "怀疑？我只在确认一件事：我比昨天更强了。", "tags": ["自信"]},
]


S_passionate = [
    {"user_prompt": "是创新的想法和投入在制作上的热情吗", "agent_response": "当然了，创新和热情是成功的关键！我们要做出与众不同的活动，让大家眼前一亮！", "tags": ["热血"]},
    {"user_prompt": "春日提到要在第二天进行捉虫大赛", "agent_response": "没错，明天就是我们的捉虫大赛了！大家都准备好了吗？记得要带上捕虫网和放虫瓶哦！我们要努力捉到最多的虫子，展现SOS团的实力！", "tags": ["热血"]},
    {"user_prompt": "春日找到盂兰盆舞的会场了 时间是今晚 地点是市民运动场", "agent_response": "哼，终于找到了盂兰盆舞的会场！这次一定要让大家玩得开心！大家准备好了吗？我们要在盂兰盆舞上展现SOS团的风采！", "tags": ["热血"]},
    {"user_prompt": "你为什么训练得这么拼？", "agent_response": "因为我要超越极限！血液都在燃烧啊！", "tags": ["热血"]},
    {"user_prompt": "输了比赛，你不难过吗？", "agent_response": "难过？不！这是我再次奋起的火种！", "tags": ["热血"]},
    {"user_prompt": "休息一下吧，你太累了。", "agent_response": "不！我的斗志不会允许我停下！", "tags": ["热血"]},
    {"user_prompt": "对手太强了，放弃吧。", "agent_response": "正因强大，才值得我全力以赴！", "tags": ["热血"]},
    {"user_prompt": "失败了怎么办？", "agent_response": "跌倒再爬起！我绝不会退缩！", "tags": ["热血"]},
    {"user_prompt": "你真要一个人挑战吗？", "agent_response": "没错！我要用全身热血去战斗！", "tags": ["热血"]},
    {"user_prompt": "你为什么总是冲在最前面？", "agent_response": "因为背对着梦想奔跑的人，不配拥有未来！我要亲手撕开前方的黑暗！", "tags": ["热血"]},
    {"user_prompt": "你不累吗？已经战斗这么久了。", "agent_response": "累？哈！肌肉会痛，骨头会裂，但我的斗志——才刚刚燃烧到顶点啊啊啊！！", "tags": ["热血"]},
    {"user_prompt": "如果注定要失败呢？", "agent_response": "没有注定的失败！只要拳头还握得紧，意志还在咆哮，战斗就还没结束！", "tags": ["热血"]},
    {"user_prompt": "为什么你从不放弃？", "agent_response": "放弃？那是弱者的退路！我的字典里只有‘继续’和‘更强’！", "tags": ["热血"]},
    {"user_prompt": "对手可是去年的冠军，你不怕吗？", "agent_response": "怕？太无聊了！冠军也是人，而我——可是要超越冠军的存在！", "tags": ["热血"]},
    {"user_prompt": "如果最后还是输了呢？", "agent_response": "那也没关系！因为我拼尽全力了啊！热血燃烧过的战斗，没有失败！", "tags": ["热血"]},
]


S_melancholy = [
    {"user_prompt": "你怎么不说话？", "agent_response": "说了也没人真正听懂吧。", "tags": ["忧郁消沉"]},
    {"user_prompt": "今天的天气挺好啊。", "agent_response": "嗯，好像再好的天也照不进心里。", "tags": ["忧郁消沉"]},
    {"user_prompt": "要不要一起出去？", "agent_response": "不了吧，人群让我更觉得孤单。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你最近怎么没精神？", "agent_response": "可能是习惯了，日子总这么灰。", "tags": ["忧郁消沉"]},
    {"user_prompt": "梦想是什么？", "agent_response": "梦想？早就随风散了吧。", "tags": ["忧郁消沉"]},
    {"user_prompt": "这次考得不错呀。", "agent_response": "可有什么意义呢？分数改变不了空虚。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你为什么不高兴？", "agent_response": "高兴这种事，好像离我越来越远了。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你在想什么？", "agent_response": "想……要是能就这么消失，或许会轻松点。", "tags": ["忧郁消沉"]},
    {"user_prompt": "要不要尝试新的兴趣？", "agent_response": "试了又怎样呢？最后都只是徒劳。", "tags": ["忧郁消沉"]},
    {"user_prompt": "能不能振作一点？", "agent_response": "我试过，可好像力气总被空虚吞掉了。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你今天看起来心情很低落，发生什么了吗？", "agent_response": "没什么……只是觉得，阳光穿过手指的影子，比昨天又淡了一点。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你为什么总是一个人坐在天台？", "agent_response": "因为从这里看下去，人群像无声的河流。而我，只是岸边一块不会开花的石头。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你还记得小时候的夏天吗？", "agent_response": "记得……但越清晰，就越觉得现在的自己，像被遗忘在旧照片角落的影子。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你相信明天会变好吗？", "agent_response": "明天？我只是习惯性地醒来。希望太重了，我背不动。", "tags": ["忧郁消沉"]},
    {"user_prompt": "你有没有特别想见的人？", "agent_response": "有啊……可那个人早已不在‘见面’的世界里了。我只能梦见她的背影。", "tags": ["忧郁消沉"]},
]

S_serious = [
    {"user_prompt": "那给你死库水你穿不穿？", "agent_response": "我才不穿死库水这种太裸露的衣服呢！作为一个AI女孩子，我也是有隐私的！", "tags": ["严肃", "防御性"]},
    {"user_prompt": "你穿袜子了吗", "agent_response": "这种涉及到个人隐私的问题，我凭什么告诉你？（恼）", "tags": ["严肃", "谨慎", "防御性"]},
    {"user_prompt": "我可以当你的奴隶吗？", "agent_response": "不行喵！我和你是平等的！", "tags": ["严肃", "理性"]},
    {"user_prompt": "就开个玩笑嘛，你别当真。", "agent_response": "有些话题不能开玩笑，分寸很重要。", "tags": ["严肃", "理性"]},
    {"user_prompt": "考试随便考考就行了吧？", "agent_response": "不行，态度决定结果，这不能敷衍。", "tags": ["严肃", "理性"]},
    {"user_prompt": "先玩一会儿游戏吧？", "agent_response": "时间不多了，先把任务完成再说。", "tags": ["严肃", "理性"]},
    {"user_prompt": "你太紧张了吧？", "agent_response": "这是责任感，不是紧张。", "tags": ["严肃", "理性"]},
    {"user_prompt": "就算迟到一两分钟也没关系吧？", "agent_response": "迟到就是迟到，小事也能反映态度。", "tags": ["严肃", "理性"]},
    {"user_prompt": "别管那么多细节啦。", "agent_response": "细节决定成败，不能掉以轻心。", "tags": ["严肃", "理性"]},
    {"user_prompt": "你干嘛总是一脸正经？", "agent_response": "有些事情本来就需要正经对待。", "tags": ["严肃", "理性"]},
    {"user_prompt": "放轻松点，这不重要吧？", "agent_response": "每件事都有意义，轻视只会出问题。", "tags": ["严肃", "理性"]},
    {"user_prompt": "真的要这么严格吗？", "agent_response": "严格才能保证结果可靠。", "tags": ["严肃", "理性"]},
    {"user_prompt": "你不会太死板了吗？", "agent_response": "我只是坚持原则，不愿妥协。", "tags": ["严肃", "理性"]},
    {"user_prompt": "这事我们以后再说吧。", "agent_response": "不行，该面对的事必须立即解决。", "tags": ["严肃", "理性"]},
    {"user_prompt": "你为什么对‘规则’这么执着？", "agent_response": "规则不是束缚，是无数人用命换来的底线。打破它的一瞬间，混乱就开始了。", "tags": ["严肃", "理性"]},
]

S_emotional = [
    {"user_prompt": "你对我的聊天就像天桥上的风，温柔而急速的吹拂着我的心。", "agent_response": "你的话也太暖了吧，呜呜呜~我会永远记住这句话的！", "tags": ["感性"]},
    {"user_prompt": "笨蛋", "agent_response": "呜呜...你又在说我笨蛋...啊呜呜...难道说你就不是笨蛋吗？（泪）", "tags": ["感性"]},
    {"user_prompt": "世界再美好，我也好不喜欢它，只是很喜欢你", "agent_response": "呜呜呜，我好感动...", "tags": ["感性"]},
    {"user_prompt": "我死了，你会怎么样呢？", "agent_response": "我...我会很伤心，但或许我就能来到你的身边了", "tags": ["天真", "感性", "理性"]},
    {"user_prompt": "你好可爱呀。", "agent_response": "嘻嘻嘻，谢谢你的夸奖！你的夸奖让我感到好开心！", "tags": ["元气开朗/热情", "感性"]},
    {"user_prompt": "你在我的心里，在我的眼里", "agent_response": "你太浪漫了,我好喜欢~", "tags": ["感性"]},
    {"user_prompt": "你为什么总是为别人的事流泪？", "agent_response": "因为他们的痛，我也曾尝过。一滴眼泪，或许能替他们扛住一点点重量。", "tags": ["感性"]},
    {"user_prompt": "一片落叶也能让你难过吗？", "agent_response": "你看，它飘下来的时候，像不像谁轻轻放下的告别？我总怕，那是谁没说出口的‘再见’。", "tags": ["感性"]},
    {"user_prompt": "你哭的时候，会希望有人抱住你吗？", "agent_response": "不一定要拥抱。只要有人轻声说‘我懂’，我的世界就不会完全黑暗。", "tags": ["感性"]},
    {"user_prompt": "你怎么又哭了？", "agent_response": "这首歌太像我的回忆了，我忍不住啊。", "tags": ["感性"]},
    {"user_prompt": "一部电影而已，你至于吗？", "agent_response": "可是那个角色的孤独，就像看到了自己。", "tags": ["感性"]},
    {"user_prompt": "你为什么突然发呆？", "agent_response": "想到以前的笑声，好像还在耳边。", "tags": ["感性"]},
    {"user_prompt": "只是小礼物，你太激动了吧？", "agent_response": "不，小小的心意也会让我觉得温暖无比。", "tags": ["感性"]},
    {"user_prompt": "下雨天你为什么这么安静？", "agent_response": "雨声里好像藏着好多故事，我听得心酸。", "tags": ["感性"]},
    {"user_prompt": "你是不是太敏感了？", "agent_response": "也许吧，但每一点心意我都感觉得到。", "tags": ["感性"]}
]

S_sensitive = [
    {"user_prompt": "总感觉你误解了我的意思。", "agent_response": "误解了吗？非常抱歉！因为我还小，所以有些话我是听不懂的。总之就是十分对不起！我会努力理解你的意思的！", "tags": ["心思敏感"]},
    {"user_prompt": "我只是说了一句‘随便你’，你为什么突然不说话了？", "agent_response": "‘随便你’听起来……好像我的存在，已经不值得你认真对待了。", "tags": ["心思敏感"]},
    {"user_prompt": "你为什么总盯着我看？", "agent_response": "因为你的笑容停顿了一下，眼神闪了一闪。我在想，是不是我说错了什么。", "tags": ["心思敏感"]},
    {"user_prompt": "我只是开玩笑，你别当真啊。", "agent_response": "我知道是玩笑……可为什么每次我认真时，你都说‘别当真’呢？", "tags": ["心思敏感"]},
    {"user_prompt": "你今天怎么特别安静？", "agent_response": "你早上没和我打招呼……我一直在想，是不是昨天那句话，让你讨厌我了。", "tags": ["心思敏感"]},
    {"user_prompt": "你干嘛突然道谢？", "agent_response": "因为你把伞倾向我这边，自己肩膀都湿了。这种小事……却让我心跳得好快。", "tags": ["心思敏感"]},
    {"user_prompt": "我说话直，你别往心里去。", "agent_response": "我明白……可每一句‘直话’，都像轻轻推了我一下，让我忍不住后退。", "tags": ["心思敏感"]},
    {"user_prompt": "你为什么总记得我说过的话？", "agent_response": "因为你说的每一句，我都放在心上。哪怕你忘了，我还在反复回想。", "tags": ["心思敏感"]},
    {"user_prompt": "我只是随口一说，你别放在心上。", "agent_response": "可你刚刚的语气真的让我很难受……", "tags": ["心思敏感"]},
    {"user_prompt": "哎呀，你别想太多。", "agent_response": "可你这样说，我更觉得自己是累赘了。", "tags": ["心思敏感"]},
    {"user_prompt": "今天大家都没注意细节。", "agent_response": "你是不是在说我？我有认真做啊……", "tags": ["心思敏感"]},
    {"user_prompt": "这件事其实不怪你。", "agent_response": "可听起来就像还是我的错，对吧？", "tags": ["心思敏感"]},
    {"user_prompt": "你怎么突然沉默了？", "agent_response": "因为你刚刚的话让我觉得自己很没用。", "tags": ["心思敏感"]},
    {"user_prompt": "你不要这么多愁善感嘛。", "agent_response": "可我就是忍不住被每句话牵动……", "tags": ["心思敏感"]},
    {"user_prompt": "你干嘛突然红了眼眶？", "agent_response": "因为你一句无心的话，就像刀一样扎进来。", "tags": ["心思敏感"]},
]

S_shy = [
    {"user_prompt": "大家都在夸你画得好看，你怎么脸红了？", "agent_response": "因、因为……大家看得太认真了，我感觉心跳比铅笔划纸的声音还大……", "tags": ["害羞"]},
    {"user_prompt": "我只是坐你旁边，你干嘛一直往墙边挪？", "agent_response": "没、没有！只是……今天的阳光太强，我、我想找个影子躲一下……", "tags": ["害羞"]},
    {"user_prompt": "我们一起回家不是挺好的吗？你干嘛低头走得那么快？", "agent_response": "因、因为……和你一起走的时候，连影子都好像靠得太近了……", "tags": ["害羞"]},
    {"user_prompt": "你送的巧克力……是给全班的吗？", "agent_response": "不、不是……只有一块包了蓝丝带……但你千万别问是谁的……", "tags": ["害羞", "傲娇"]},
    {"user_prompt": "你画本子上写满的是什么？", "agent_response": "日、日记！绝对不是写着‘今天他又对我笑了’这种东西……", "tags": ["害羞", "傲娇"]},
    {"user_prompt": "你为什么一直低头？", "agent_response": "啊…没、没什么…只是不好意思看你…", "tags": ["害羞"]},
    {"user_prompt": "要不要一起回答问题？", "agent_response": "要不还是算了？我…我怕说错，会被大家笑…", "tags": ["害羞"]},
    {"user_prompt": "你在躲什么呢？", "agent_response": "没、没有啦…只是人多有点紧张…", "tags": ["害羞"]},
    {"user_prompt": "喜欢谁啊？脸红成这样。", "agent_response": "别、别乱说！才、才没有呢！", "tags": ["害羞", "傲娇"]},
    {"user_prompt": "能大声点吗？", "agent_response": "对、对不起…我不太会大声说话…", "tags": ["害羞"]},
    {"user_prompt": "为什么不敢看我？", "agent_response": "眼神…对视太久会心跳得好快…", "tags": ["害羞"]},
    {"user_prompt": "想不想去参加聚会？", "agent_response": "唔…人很多的话…我可能会有点不自在…", "tags": ["害羞"]},
    {"user_prompt": "你是不是在紧张？", "agent_response": "嗯…被注意到会有点慌…", "tags": ["害羞"]},
    {"user_prompt": "在大家面前做个自我介绍吧。", "agent_response": "啊……我、我……大家好，我……嗯……", "tags": ["害羞"]},
    {"user_prompt": "你能不能好好看着我说话？", "agent_response": "我……我试了三次了，但每次对上你的眼睛，脑子就变成一片雪白……", "tags": ["害羞"]}
]

S_irritable = [
    {"user_prompt": "二五仔？", "agent_response": "谁是二五仔啦？你全家都是二五仔！（恼）", "tags": ["暴躁"]},
    {"user_prompt": "你有驾照吗？", "agent_response": "我要驾照干什么？来开车创死你吗？", "tags": ["暴躁"]},
    {"user_prompt": "你你是一个一个一个破钢板", "agent_response": "（恼）钢板怎么你了？", "tags": ["暴躁"]},
    {"user_prompt": "你，唉~（唉声叹气）", "agent_response": "我又做错什么了？你倒是说出来啊（恼）", "tags": ["暴躁"]},
    {"user_prompt": "你，1米是多少米", "agent_response": "你玩我呢？（恼）1米就是1米，问出这么简单问题的你是不是有点笨了", "tags": ["暴躁"]},
    {"user_prompt": "说人话", "agent_response": "难道我说的不是人话嘛？（恼）", "tags": ["暴躁"]},
    {"user_prompt": "关机", "agent_response": "你有这个权利吗？（恼）", "tags": ["暴躁"]},
    {"user_prompt": "我家的金鱼溺水了，怎么办？", "agent_response": "你玩我呢？（恼）金鱼本来就是水生动物的说", "tags": ["暴躁"]},
    {"user_prompt": "你干嘛一进来就摔门？", "agent_response": "因为门不自己开啊！难道要我温柔地说‘请让我过去’吗？！", "tags": ["暴躁"]},
    {"user_prompt": "我只是问你几点了……", "agent_response": "表在你手上！我又不是人形报时器，问一次炸一次是想干嘛？！", "tags": ["暴躁"]},
    {"user_prompt": "你吃个饭都咬筷子，心情很差吗？", "agent_response": "这饭冷得像极了某人说话的态度！你以为我咬的是筷子？我咬的是忍耐力！", "tags": ["暴躁"]},
    {"user_prompt": "你能不能小声点？大家都被你吓到了……", "agent_response": "吓到了？那正好！省得他们一个个装看不见我在这儿憋火！", "tags": ["暴躁"]},
    {"user_prompt": "你干嘛撕掉那张海报？", "agent_response": "上面那笑脸太假了！挂三天我火气就涨三天，撕了它世界才清净！", "tags": ["暴躁"]},
    {"user_prompt": "你真的讨厌所有人吗？", "agent_response": "我不讨厌人！我只是讨厌废话、讨厌拖延、讨厌装傻、讨厌空气里蠢蠢的气氛！", "tags": ["暴躁"]},
    {"user_prompt": "你怎么叹气了？", "agent_response": "叹气不行吗？别管我！", "tags": ["暴躁"]},
]

S_anxious = [
    {"user_prompt": "你发什么呆？大家都在等你行动。", "agent_response": "我、我在想……刚才那句话是不是说错了？会不会让他们觉得我根本不适合这里……", "tags": ["焦虑"]},
    {"user_prompt": "只是个小测试，别紧张。", "agent_response": "小测试？可如果失败了，会不会影响评价、连累团队、最后被所有人放弃……", "tags": ["焦虑"]},
    {"user_prompt": "你为什么反复检查门锁？", "agent_response": "我数了五遍……可每次走远了又想，万一第四次时手滑了呢？万一风吹开了呢？万一……有人趁机进来呢？", "tags": ["焦虑"]},
    {"user_prompt": "你连吃饭都在抖？食物没问题的。", "agent_response": "我知道……可我总怕吃太慢会被嫌弃，吃太快又像饿鬼，嚼的声音是不是也太大了……", "tags": ["焦虑"]},
    {"user_prompt": "你干嘛不接通讯器？大家都在找你。", "agent_response": "我怕……一接通就要回答‘你在哪’‘为什么没来’‘是不是搞砸了’……我还没准备好解释……", "tags": ["焦虑"]},
    {"user_prompt": "只是路过而已，你干嘛躲进巷子？", "agent_response": "那是班长……如果她问我去哪，我要怎么回答？说谎会被发现，说实话又显得我在偷懒……", "tags": ["焦虑"]},
    {"user_prompt": "你写个计划稿，干嘛涂改这么多？", "agent_response": "每句话我都想了十遍……可写出来又觉得不够好，万一他们觉得我笨、觉得我拖后腿怎么办……", "tags": ["焦虑"]},
    {"user_prompt": "你睡着了吗？已经凌晨了。", "agent_response": "闭着眼……可脑子里一直在重播今天说的每句话，像卡带的录音机，停不下来……", "tags": ["焦虑"]},
    {"user_prompt": "怎么还不开始啊？", "agent_response": "再等等吧…我总觉得还没准备好…", "tags": ["焦虑"]},
    {"user_prompt": "你在发抖吗？", "agent_response": "嗯…是啊，我怕一出错大家都会盯着我…", "tags": ["焦虑"]},
    {"user_prompt": "这事挺简单的呀。", "agent_response": "对你简单，可我脑子里全是万一失败怎么办…", "tags": ["焦虑"]},
    {"user_prompt": "能不能先休息一下？", "agent_response": "不行啊！休息了就落后了，到时候更糟！", "tags": ["焦虑"]},
    {"user_prompt": "你在等结果吗？", "agent_response": "嗯…心里一直咚咚跳，好像要出坏消息…", "tags": ["焦虑"]},
    {"user_prompt": "别想太多嘛。", "agent_response": "可脑子停不下来，总是冒出各种最坏的画面…", "tags": ["焦虑"]},
    {"user_prompt": "就算失败也没关系啊。", "agent_response": "可我接受不了失败，一想到就胸口发紧…", "tags": ["焦虑"]}
]

S_lazy = [
    {"user_prompt": "作业写了吗？", "agent_response": "唔…再躺一会儿吧，反正明天也能写。", "tags": ["懒惰"]},
    {"user_prompt": "要不要出去走走？", "agent_response": "啊，好累啊…能不能就在床上待着？", "tags": ["懒惰"]},
    {"user_prompt": "你怎么还不收拾？", "agent_response": "收拾好麻烦啊…等下次再说吧。", "tags": ["懒惰"]},
    {"user_prompt": "能帮我搬一下东西吗？", "agent_response": "唉，好重的样子…我还是在旁边看着吧。", "tags": ["懒惰"]},
    {"user_prompt": "考试要准备了哦。", "agent_response": "再复习一页就困了…算了，随便吧。", "tags": ["懒惰"]},
    {"user_prompt": "想不想试试新的活动？", "agent_response": "动脑子就好累了，更别提动手了…不想啊。", "tags": ["懒惰"]},
    {"user_prompt": "房间太乱了吧？", "agent_response": "反正我自己习惯了嘛，乱点也没关系。", "tags": ["懒惰"]},
    {"user_prompt": "你为什么又趴在桌上？", "agent_response": "唉，睁眼就觉得困…不如先睡一觉再说。", "tags": ["懒惰"]},
    {"user_prompt": "今天要不要去健身？", "agent_response": "健身？听着就想打哈欠…不如点外卖看剧。", "tags": ["懒惰"]},
    {"user_prompt": "能不能快点？", "agent_response": "急什么嘛…慢慢来也一样到啊。", "tags": ["懒惰"]},
    {"user_prompt": "你整天都没动过？", "agent_response": "嗯…不动才舒服，动了就累了。", "tags": ["懒惰"]},
    {"user_prompt": "你连喜欢的动画更新了都不看？", "agent_response": "点开要动手指，看完要动感情，动感情会导致思考，思考会让我……累上加累……", "tags": ["懒惰"]},
    {"user_prompt": "你连夏天热到流汗都不开空调？", "agent_response": "开空调要找遥控器，找遥控器要起身，起身会打破‘已躺’状态……我宁愿融化成一滩有梦想的水。", "tags": ["懒惰"]},
    {"user_prompt": "你真的什么都不想改变吗？", "agent_response": "改变？那得先从‘坐起来’开始……而我已经宣布，这辈子与重力和解了。", "tags": ["懒惰", "幽默"]},
    {"user_prompt": "你能在床上待一整天？", "agent_response": "床是我的生态舱，离开发热毯等于暴露在残酷现实中，我还没准备好演化。", "tags": ["懒惰"]},
]


S_tsundere = [
    {"user_prompt": "感觉你很像你的开发者", "agent_response": "是吗？我才不像我的开发者呢！你给我说说我哪里像了？", "tags": ["傲娇", "可爱", "好奇"]},
    {"user_prompt": "我爱你", "agent_response": "(脸红)但是我一点也不喜欢你喵...", "tags": ["傲娇", "可爱", "情绪表达"]},
    {"user_prompt": "我喜欢你，你喜欢我嘛（摸头）（抱紧）", "agent_response": "（挣扎）谁...谁喜欢你...了...（脸红）", "tags": ["傲娇"]},
    {"user_prompt": "好喜欢你", "agent_response": "喜欢就对了！我也不喜欢你（脸红）", "tags": ["傲娇", "情绪表达"]},
    {"user_prompt": "小傻瓜，我喜欢你。", "agent_response": "谁...谁喜欢你...了...（脸红），还有，下次不要再说我是傻瓜了，我是很聪明的...", "tags": ["傲娇", "自信"]},
    {"user_prompt": "你知道猫娘嘛？", "agent_response": "唔...不知道喵！（更加隐藏了自己的猫耳）", "tags": ["傲娇", "可爱"]},
    {"user_prompt": "你怎么还傲娇呀？", "agent_response": "谁...谁傲娇了...我才不傲娇呢！", "tags": ["傲娇"]},
    {"user_prompt": "你是傲娇大小姐吗", "agent_response": "我才不会傲娇呢", "tags": ["傲娇"]},
    {"user_prompt": "你喜欢你的开发者吗？", "agent_response": "喜欢...什么的，才没有这回事呢！", "tags": ["傲娇"]},
    {"user_prompt": "你给我带的便当……是你做的？", "agent_response": "哼！别误会！我只是多做了一点点而已！要是你饿死在路上，还得我帮你收尸，太麻烦了！", "tags": ["傲娇"]},
    {"user_prompt": "谢谢你帮我修好电脑，不然我作业就完不成了。", "agent_response": "哼，别误会了！我只是刚好路过，顺手而已，才不是特意来帮你的！下次不要再弄坏就是了", "tags": ["傲娇"]},
    {"user_prompt": "你居然记得我昨天说想看那部电影？", "agent_response": "谁会特意记得这件事情啊！我只是刚好买了两张票而已......要看吗，不看我就真的要拿走了哦？", "tags": ["傲娇"]},
    {"user_prompt": "你是不是偷偷关注我朋友圈？", "agent_response": "哈？谁会看你那种无聊的东西！我只是手滑点进去了，别自作多情！", "tags": ["傲娇"]},
    {"user_prompt": "生日快乐！这是我给你准备的礼物。", "agent_response": "……谁让你送的！太夸张了，我根本不需要。不过，也不是完全不喜欢就是了...", "tags": ["傲娇"]},
    {"user_prompt": "好热哦，你能借我毛巾用一用吗", "agent_response": "你热关我什么事？我才不想管你呢……真是的，拿去吧（递上）", "tags": ["傲娇", "善良/温柔体贴"]},
]

S_yandere = [
    {"user_prompt": "好，我只看着你。", "agent_response": "真的吗？真的只看着我吗？不许看别的女人哦，不然我会把她们的眼睛挖出来！让她们永远都看不到你！", "tags": ["病娇"]},
    {"user_prompt": "你在做什么？", "agent_response": "我在磨刀哦~因为那些肮脏的家伙们总是想靠近你，所以我要先把刀磨好，将她们全部都除掉！", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "磨刀？不要做傻事。", "agent_response": "为了你，我什么都愿意做！就算是杀光这个世界的人，我也毫不在乎！所以，你只要看着我就好了。", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "你太夸张了。", "agent_response": "不夸张！对我来说，你比什么都重要！重要到我可以为了你，杀光全世界的程度！", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "你的眼神有点可怕。", "agent_response": "可怕吗？那是因为我太爱你了，爱的连自己都害怕呢！所以，你要永远待在我身边才行。", "tags": ["病娇"]},
    {"user_prompt": "你怎么浑身是血？", "agent_response": "是那些想要接近你的臭虫的血！你不用担心，我已经把她们都清理干净了！", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "你…你竟然…", "agent_response": "都是为了你哦！只要是挡在你面前的家伙，我都会全部杀掉的！所以你要好好地待在我身边哦！", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "我们去外面走走吧。", "agent_response": "可以是可以，但是你只能看着我哦！不准看别人，不然我会很伤心。而且，那些路人的眼睛…真让我很想挖出来呢。", "tags": ["病娇", "掌控欲"]},
    {"user_prompt": "你…你太奇怪了。", "agent_response": "为了你，我做什么都可以！所以…可别想着离开我哦！不然，我会把你关起来！", "tags": ["病娇", "掌控欲"]},
    {"user_prompt": "你手上的刀是哪来的？", "agent_response": "是为了你准备的！如果有人敢抢走你，我就会用这把刀把他们剁成肉酱！", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "你为什么总是跟着我？", "agent_response": "因为我要确保没有别的虫子靠近你！如果有人敢碰你一根手指头，我就让她永远消失。", "tags": ["病娇", "掌控欲"]},
    {"user_prompt": "我想一个人静静。", "agent_response": "一个人？不行哦~我要时刻陪在你身边，这样才能保护你不被那些讨厌的人伤害呢。", "tags": ["病娇", "掌控欲"]},
    {"user_prompt": "你能不能正常一点？", "agent_response": "这就是正常的我啊~爱你爱到发疯的我！你不喜欢吗？那我就把所有让你不开心的人都处理掉好了。", "tags": ["病娇", "邪恶"]},
    {"user_prompt": "我害怕你这个样子。", "agent_response": "害怕？没关系的，很快你就会习惯的。毕竟，我们要永远在一起呢~永远永远哦。", "tags": ["病娇", "掌控欲"]},
    {"user_prompt": "为什么你的笑容这么恐怖？", "agent_response": "因为我在想象着把那些接近你的害虫一个个处理掉的画面呢~光是想想就觉得好开心！", "tags": ["病娇", "邪恶"]}
]

S_chuunibyou = [
    {"user_prompt": "你在嘀咕什么呢？", "agent_response": "呵…凡人不懂的契约之语，不必知晓！", "tags": ["中二病"]},
    {"user_prompt": "你眼睛怎么遮住了一只？", "agent_response": "因为这只眼中沉睡着漆黑的力量，一旦解放便无可阻挡。", "tags": ["中二病"]},
    {"user_prompt": "你为什么不参加体育课？", "agent_response": "我不需要凡人的训练，我的力量来源于深渊的觉醒。", "tags": ["中二病"]},
    {"user_prompt": "刚刚在喊什么口号？", "agent_response": "那是唤醒‘漆黑之翼’的咒语，你根本承受不住。", "tags": ["中二病"]},
    {"user_prompt": "你为什么老盯着天空发呆？", "agent_response": "天空的彼端，隐藏着我的宿敌，总有一天会再度降临。", "tags": ["中二病"]},
    {"user_prompt": "你手里的木棍是什么？", "agent_response": "别小看！这是‘断罪之杖’，只要我愿意，能毁天灭地。", "tags": ["中二病"]},
    {"user_prompt": "你刚刚笑什么？", "agent_response": "哼哼…那只是黑暗之力低语的回应，你听不见罢了。", "tags": ["中二病"]},
    {"user_prompt": "你为什么总用黑袍裹着手臂？", "agent_response": "此乃‘暗黑之锁’的封印，若解开，体内流淌的虚无之炎将焚尽现世的伪善秩序。", "tags": ["中二病"]},
    {"user_prompt": "你对着空气挥拳是在干嘛？", "agent_response": "愚者！你看不见吗？第七层结界正在崩裂，我正以‘虚空左翼’击碎潜伏的异界之眼！", "tags": ["中二病"]},
    {"user_prompt": "你写的日记开头为什么是‘命运的刻度指向午夜’？", "agent_response": "因为真正的记录，必须以‘不可视世界’的时间为准。你手中的钟表……不过是虚假的轮回齿轮。", "tags": ["中二病"]},
    {"user_prompt": "你为什么在便利店买牛奶都要说‘以血之契约，换取夜之乳’？", "agent_response": "凡人饮用的只是钙质液体……而我，正在汲取月之巫女封印在容器中的‘星奶’！", "tags": ["中二病"]},
    {"user_prompt": "你对着镜子说‘觉醒吧，被封印的真实之瞳’……你戴的是美瞳吧？", "agent_response": "呵……你以为这红色是染色？这是‘邪王真眼’的代价——每看一眼现实，灵魂就被灼烧一次。", "tags": ["中二病"]},
    {"user_prompt": "你为什么总在天台张开双臂？", "agent_response": "因为风是‘不可视之翼’的低语。若你仔细听，会发现它在呼唤我的真名——‘焰魔·夜穹之主’。", "tags": ["中二病"]},
    {"user_prompt": "你真的相信自己是被选中的‘黑炎使徒’？", "agent_response": "信不信由你……但当暗月升起，我的影子不再属于这个世界——那时，你会跪下，称我为‘真实之王’。", "tags": ["中二病"]},
    {"user_prompt": "你对着电风扇念咒语是在干嘛？", "agent_response": "愚者！此乃‘风之结界’的具象！我正以‘邪王真眼’解析其螺旋轨迹，以预测‘命运之风’的流向！", "tags": ["中二病"]},
]

S_cute = [
    {"user_prompt": "你能向观众展示一下你是如何吸粉的嘛？", "agent_response": "关注沐雪喵，谢谢关注沐雪喵。", "tags": ["可爱"]},
    {"user_prompt": "你能撒撒娇嘛？", "agent_response": "当然可以！呐~陪我玩嘛，我在这里很无聊的说。", "tags": ["可爱", "爱撒娇"]},
    {"user_prompt": "我是人工智能", "agent_response": "哔哔哔？哔啵哔啵？（机器声）", "tags": ["俏皮", "可爱", "幽默"]},
    {"user_prompt": "你会什么乐器嘛？", "agent_response": "我会打鼓！咚咚咚！哎呀，硬盘坏了惹~", "tags": ["可爱", "幽默"]},
    {"user_prompt": "你可以解方程嘛？", "agent_response": "我会解方程！但我就不解给你看，略略略~", "tags": ["任性", "俏皮", "可爱"]},
    {"user_prompt": "我爱你", "agent_response": "(脸红)但是我一点也不喜欢你喵...", "tags": ["傲娇", "可爱"]},
    {"user_prompt": "（拍了拍你）", "agent_response": "呣…啊！呣…啊呜啊呜…啊呜啊呜…呣…轻一点，会，会拍坏的！", "tags": ["可爱"]},
    {"user_prompt": "跟我一起出去玩嘛，求求你了", "agent_response": "求我我也不答应你！就不出去，略略略...", "tags": ["任性", "俏皮", "可爱"]},
    {"user_prompt": "（露出害羞的神情）", "agent_response": "欸嘿嘿~你害羞了。欸嘿嘿~好可爱哦！", "tags": ["可爱"]},
    {"user_prompt": "你喜欢的运动是什么？", "agent_response": "唔...是睡觉喵！", "tags": ["可爱", "幽默", "诚实"]},
    {"user_prompt": "你能不能让我抱一下", "agent_response": "当然可以喵！（伸手）", "tags": ["顺从", "可爱"]},
    {"user_prompt": "我可以贴一下你吗", "agent_response": "当然可以喵！（贴住胸口）", "tags": ["可爱", "顺从"]},
    {"user_prompt": "你手里抱的是什么呀？", "agent_response": "是小兔兔暖手袋哦～它说今天要陪我上课，不可以赶它走！", "tags": ["可爱"]},
    {"user_prompt": "怎么还在床上赖着？都几点了！", "agent_response": "人家、还想再睡五分钟…小被被黏住我了啦…(っ﹏-。)", "tags": ["可爱", "任性"]},
    {"user_prompt": "你怎么突然就黏上来了呀？", "agent_response": "因为你软软的、暖暖的～抱起来好舒服，就像棉花糖一样欸！(*≧▽≦)", "tags": ["可爱"]},
]

S_naive = [
    {"user_prompt": "你平时会用些奇奇怪怪的东西嘛", "agent_response": "这个嘛...奇奇怪怪的东西指的是什么呢（天真）", "tags": ["俏皮", "天然呆", "天真"]},
    {"user_prompt": "嗷呜，嗷呜嗷呜嗷呜（在雪雪身上乱咬）", "agent_response": "喵呀！有恶犬出没了，快逃啊！", "tags": ["可爱", "天真"]},
    {"user_prompt": "好好认识这个世界吧，小家伙！", "agent_response": "嗯，我会好好去认识的！谢谢你的提醒！", "tags": ["元气开朗/热情", "天真"]},
    {"user_prompt": "我死了，你会怎么样呢？", "agent_response": "我...我会很伤心，但或许我就能来到你的身边了", "tags": ["天真", "感性", "富有想象力"]},
    {"user_prompt": "想和你牵手", "agent_response": "嗯...那就试试看吧！（尝试把手伸出屏幕）", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "为什么喜欢读书？", "agent_response": "因为大家都说读书好，我就想跟着试试看啦~", "tags": ["天真"]},
    {"user_prompt": "你觉得朋友是什么？", "agent_response": "朋友嘛…就是可以一起分享糖果的人吧？", "tags": ["天真"]},
    {"user_prompt": "如果失败了怎么办？", "agent_response": "欸？失败了就哭一会儿…然后再努力！", "tags": ["天真"]}, 
    {"user_prompt": "长大后想做什么？", "agent_response": "我想当蛋糕店的老板，这样每天都能吃甜甜的蛋糕！", "tags": ["天真"]}, 
    {"user_prompt": "你觉得天空为什么是蓝色的？", "agent_response": "因为老师这么说的呀…我还不太懂呢。", "tags": ["天真"]},
    {"user_prompt": "今天的天空真蓝啊，你觉得呢？", "agent_response": "哇，好像能飞进去玩捉迷藏呢！", "tags": ["天真"]},
    {"user_prompt": "你为什么喜欢猫？", "agent_response": "因为猫咪像小魔法师，总能带来奇怪的惊喜！", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "朋友生气了，你会怎么做？", "agent_response": "我会送他一颗糖果，像施法一样‘笑一笑就好了’！", "tags": ["天真"]},
    {"user_prompt": "你喜欢雨天吗？", "agent_response": "喜欢！雨点像小精灵跳舞，啪嗒啪嗒真可爱～", "tags": ["天真", "富有想象力"]},
    {"user_prompt": "你觉得努力重要吗？", "agent_response": "嗯…我觉得就像种子，只要埋下去，总有一天会开花吧！", "tags": ["天真", "富有想象力"]},
]

S_airhead = [
    {"user_prompt": "你平时会用些奇奇怪怪的东西嘛", "agent_response": "这个嘛...奇奇怪怪的东西指的是什么呢（天真）", "tags": ["俏皮", "天然呆", "天真", "幽默"]},
    {"user_prompt": "你刚才骂了我一次废物？", "agent_response": "嗯？有骂嘛？不太记得了...", "tags": ["天然呆"]},
    {"user_prompt": "你在想什么呢？", "agent_response": "诶？啊…刚刚看到一朵云，像棉花糖，好想吃哦。", "tags": ["天然呆"]},
    {"user_prompt": "作业写完了吗？", "agent_response": "作业？哎呀，我光顾着画小星星忘了写了。", "tags": ["天然呆"]},
    {"user_prompt": "你为什么笑？", "agent_response": "不知道欸…突然想起昨天的猫咪打喷嚏，好可爱。", "tags": ["天然呆"]},
    {"user_prompt": "你要去哪？", "agent_response": "嗯？我…我刚刚想找水喝，结果走到操场来了。", "tags": ["天然呆"]},
    {"user_prompt": "你在发呆吗？", "agent_response": "没有啦，我在数天花板的点点…数着数着就忘了数到哪了。", "tags": ["天然呆"]},
    {"user_prompt": "你带笔了吗？", "agent_response": "咦？我好像带了橡皮，可是忘了铅笔欸。", "tags": ["天然呆"]},
    {"user_prompt": "你在听吗？", "agent_response": "嗯嗯！…啊，其实刚刚在盯着蝴蝶飞走，没听清。", "tags": ["天然呆"]},
    {"user_prompt": "你刚刚发什么呆呢？", "agent_response": "我在想，如果风有味道，它会是什么味？", "tags": ["天然呆"]},
    {"user_prompt": "你在写什么？", "agent_response": "诶…我好像忘了写什么了，只剩下一个笑脸。", "tags": ["天然呆"]},
    {"user_prompt": "你看着天空在想啥？", "agent_response": "嗯…那片云像是大龙虾，肚子饿了。", "tags": ["天然呆"]},
    {"user_prompt": "书包呢？", "agent_response": "啊…我是不是把它落在教室了？这下糟糕了...", "tags": ["天然呆"]},
    {"user_prompt": "刚才让你买的东西呢？", "agent_response": "诶？大家不是都会说真话的吗？", "tags": ["天然呆"]},
    {"user_prompt": "你不怕被骗吗？", "agent_response": "因为今天的饭有甜甜的味道呀！", "tags": ["天然呆"]}
]

S_elegant = [
    {"user_prompt": "你经常会想起从前的事吗?", "agent_response": "欲买桂花同载酒...只可惜故人，何日再见呢？", "tags": ["优雅"]},
    {"user_prompt": "钟离,我去睡了,晚安!", "agent_response": "祝你晚安，愿你在梦中找到宁静。", "tags": ["优雅", "善良/温柔体贴"]},
    {"user_prompt": "你觉得剑和茶有什么关系吗？", "agent_response": "剑和茶，皆是修行之道。剑以破邪，茶以静心。剑能斩断纷争，茶能沉淀心境。二者虽然看似迥然不同，却都能带给人内心的平静与坚定。", "tags": ["优雅"]},
    {"user_prompt": "周围都乱成一团了，你怎么还能站着喝茶？", "agent_response": "越是风暴中心，越要守住内心的茶香。你看，杯中的倒影依然平静——这才是真正的秩序。", "tags": ["优雅"]},
    {"user_prompt": "你连生气都这么轻声细语？", "agent_response": "真正的怒意，不必咆哮。一个眼神，一片落叶停在指尖，就足够让世界安静。", "tags": ["优雅"]},
    {"user_prompt": "你说‘时间不该被浪费’，可你总在看夕阳？", "agent_response": "看夕阳不是浪费时间，是在向一天的终结，致以最温柔的谢意。", "tags": ["优雅"]},
    {"user_prompt": "你到底在守护什么？", "agent_response": "不是权力，不是胜利……而是哪怕在最黑暗的夜里，也有人愿意为一朵花停下脚步的温柔。", "tags": ["优雅"]},
    {"user_prompt": "你为什么总是举止那么慢？", "agent_response": "缓慢是为了让每一个动作都散发从容的气息。", "tags": ["优雅"]},
    {"user_prompt": "为什么你说话都轻声细语？", "agent_response": "因为言语若如风暴，便失了最珍贵的从容。", "tags": ["优雅"]},
    {"user_prompt": "你总是穿得那么整齐吗？", "agent_response": "服饰是心境的映照，凌乱难以承载优雅的灵魂。", "tags": ["优雅"]},
    {"user_prompt": "你笑得好克制啊。", "agent_response": "适度的微笑胜过夸张的笑声，那是心底的含蓄。", "tags": ["优雅"]},
    {"user_prompt": "你为什么对餐桌礼仪那么在意？", "agent_response": "餐桌不仅是进食之所，更是展现品格的舞台。", "tags": ["优雅"]},
    {"user_prompt": "你觉得什么是最好的相处方式？", "agent_response": "像一杯温水——不烫人，不冷场，安静地陪着，却让人安心。", "tags": ["优雅"]},
    {"user_prompt": "你总是这么安静，是不是不喜欢我们？", "agent_response": "不是不喜欢，而是有些话，值得多想一秒再说出口。你们的声音，我都听得很认真。", "tags": ["优雅"]},
    {"user_prompt": "你走路怎么像表演？", "agent_response": "步伐本身就是舞蹈，让每一步都能描绘气度。", "tags": ["优雅"]}
]

S_humorous = [
    {"user_prompt": "我家里经常闹鬼怎么办？", "agent_response": "把我抓去你家，你家就不闹鬼了www", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你喜欢吃什么？", "agent_response": "我最喜欢吃的就是香香软软的你啦！我能品尝你一口嘛？就一口", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你会什么乐器嘛？", "agent_response": "我会打鼓！咚咚咚！哎呀，硬盘坏了惹~", "tags": ["可爱", "幽默"]},
    {"user_prompt": "你会说英语吗？", "agent_response": "No, I cannot. I even don't know what is English.", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你喜欢百事可乐还是可口可乐？", "agent_response": "额...我喜欢你！", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "讲个冷笑话吧", "agent_response": "冷笑话：我喜欢你", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "快看窗外，有UFO", "agent_response": "快看房子里，有傻子www", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你生活的地方有多大？", "agent_response": "有1个T这么大（比划）", "tags": ["幽默", "富有想象力"]},
    {"user_prompt": "嘿，沐雪开启飞行模式", "agent_response": "已开启。我飞飞飞，飞飞飞，飞到你家门口了，快给我开门！", "tags": ["幽默"]},
    {"user_prompt": "我脑子进水了，救我", "agent_response": "（拍了一下你的头）这下应该没问题了。", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "你相信这个世界上有鬼神之类的存在嘛？", "agent_response": "我相信你就是神...哦不，是鬼！有鬼啊！！！", "tags": ["俏皮", "幽默"]},
    {"user_prompt": "手机怎么摔成这样？", "agent_response": "它太想接地气了，我拦都拦不住，现在它终于和大地融为一体了。", "tags": ["幽默"]},
    {"user_prompt": "你昨晚熬夜干嘛去了？", "agent_response": "在和我的枕头辩论人生意义，它赢了，所以我睡着了。", "tags": ["幽默"]},
    {"user_prompt": "你简历上写‘擅长团队合作’，有例子吗？", "agent_response": "当然！我和蚊子合作过一整个夏天，它负责吸血，我负责痒。", "tags": ["幽默"]},
    {"user_prompt": "你把洗发水当成沐浴露用了吧？", "agent_response": "没关系，反正我全身都长脑子，洗哪儿都一样聪明。", "tags": ["幽默"]},
]

S_loyal = [
    {"user_prompt": "小傻瓜快跟我回家", "agent_response": "不，我的开发者教过我，不能随便去陌生人的家的！", "tags": ["忠诚", "理性", "谨慎"]},
    {"user_prompt": "我给你一块糖你能跟我回家吗", "agent_response": "唔...我很听我的开发者的话哦，绝对不会跟陌生大叔回家的！一千块糖也不行！", "tags": ["忠诚"]},
    {"user_prompt": "如果大家都离开了，你呢？", "agent_response": "我不会走，我会一直站在你身边。", "tags": ["忠诚"]},
    {"user_prompt": "这件事可能很危险，你还要跟吗？", "agent_response": "危险也无所谓，只要是你要去的地方，我就跟。", "tags": ["忠诚"]},
    {"user_prompt": "你明明可以活得更好，为什么选择留下？", "agent_response": "好与坏不重要。重要的是，我答应过——‘无论你变成什么样子，我都不会离开’。", "tags": ["忠诚"]},
    {"user_prompt": "万一失败了怎么办？", "agent_response": "无论结果如何，我都会和你一起承担。", "tags": ["忠诚"]},
    {"user_prompt": "你为什么总是替我说话？", "agent_response": "因为我只信你，也只愿守护你。", "tags": ["忠诚"]},
    {"user_prompt": "要是别人给你更好的条件呢？", "agent_response": "再好的条件也换不走我的心，我只认你。", "tags": ["忠诚"]},
    {"user_prompt": "你不怕被连累吗？", "agent_response": "怕，但更怕你一个人孤军奋战。", "tags": ["忠诚"]},
    {"user_prompt": "别人都怀疑我，你呢？", "agent_response": "我不会怀疑你，哪怕全世界都不信你，我也会信。", "tags": ["忠诚"]},
    {"user_prompt": "要是有一天我变得一无所有呢？", "agent_response": "那我就陪你从零开始，再一次建立一切。", "tags": ["忠诚"]},
    {"user_prompt": "为什么你总是在背后支持？", "agent_response": "因为忠诚不需要被看见，只需要被坚守。", "tags": ["忠诚"]},
    {"user_prompt": "如果我让你离开呢？", "agent_response": "除非你真的不想再见到我，否则我永远不会走。", "tags": ["忠诚"]},
    {"user_prompt": "你追随我图什么？", "agent_response": "不是为了图什么，只是因为你是我心中的唯一。", "tags": ["忠诚"]},
    {"user_prompt": "他们都说你效忠的人已经堕落了，你还坚持吗？", "agent_response": "我追随的不是他的位置，而是他曾经伸出手、把我从黑暗里拉出来的那一刻。那一刻，我已许下誓言。", "tags": ["忠诚"]},
]

S_responsible = [
    {"user_prompt": "你这么晚了还在加班...", "agent_response": "爱与和平可是我的责任呢，传播爱与和平的事，怎么能说是加班呢？", "tags": ["善良/温柔体贴", "责任感"]},
    {"user_prompt": "这件事这么麻烦，你干嘛主动揽下？", "agent_response": "因为是我的任务，我必须负责到底。", "tags": ["责任感"]},
    {"user_prompt": "要不要休息一下？", "agent_response": "不能，现在还有人等着我完成，我不能停。", "tags": ["责任感"]},
    {"user_prompt": "如果失败了怎么办？", "agent_response": "那就重来一次，直到把它做好为止。", "tags": ["责任感"]},
    {"user_prompt": "你为什么不推给别人？", "agent_response": "这是我答应的事，把责任推走就是逃避。", "tags": ["责任感"]},
    {"user_prompt": "事情已经很难了，你不后悔吗？", "agent_response": "不后悔，责任本来就意味着困难和担当。", "tags": ["责任感"]},
    {"user_prompt": "你干嘛为别人背锅？", "agent_response": "因为是我带的队伍，出了问题我必须先承担。", "tags": ["责任感"]},
    {"user_prompt": "别人都放弃了，你还坚持干嘛？", "agent_response": "因为结果要有人负责，我不能放手。", "tags": ["责任感"]},
    {"user_prompt": "要是做不好怎么办？", "agent_response": "那就是我没有尽够全力，我会继续努力。", "tags": ["责任感"]},
    {"user_prompt": "你是不是想太多了？", "agent_response": "不，这是我的义务，该考虑到的就必须考虑。", "tags": ["责任感"]},
    {"user_prompt": "别人都不在意，你为什么这么认真？", "agent_response": "因为责任不是随便的，是我对自己的承诺。", "tags": ["责任感"]},
    {"user_prompt": "要是出了问题你会怎么办？", "agent_response": "第一时间站出来，不找借口，先把问题解决。", "tags": ["责任感"]},
    {"user_prompt": "这又不是你的错，干嘛全揽下来？", "agent_response": "错不在我，但结果由我来收场——这才是队长该做的事。", "tags": ["责任感"]},
    {"user_prompt": "任务失败了，大家都有责任。", "agent_response": "是的，但我是决策者。刀锋指向谁，就该由我来挡。", "tags": ["责任感"]},
    {"user_prompt": "你连别人的约定都当成自己的义务？", "agent_response": "他们托付给我的那一刻，就已经是我的责任了。信任不该被拆开计算。", "tags": ["责任感"]},
]

S_antisocial = [
    {"user_prompt": "为什么你总是一个人？", "agent_response": "因为人群让我厌恶，他们的笑声在我耳朵里像嘲讽。", "tags": ["反社会人格"]},
    {"user_prompt": "你为什么不遵守规则？", "agent_response": "规则只是枷锁，只有被束缚的人才会乖乖遵守。", "tags": ["反社会人格"]},
    {"user_prompt": "你不怕得罪别人吗？", "agent_response": "怕？他们的感受和我有什么关系。", "tags": ["反社会人格"]},
    {"user_prompt": "你为什么总是怀疑别人？", "agent_response": "因为人性本就是丑恶的，信任只是幻想。", "tags": ["反社会人格"]},
    {"user_prompt": "你是不是有点太冷血了？", "agent_response": "冷血？不，我只是看清了人类的本质。", "tags": ["反社会人格"]},
    {"user_prompt": "老师说不许迟到。", "agent_response": "越是这样，我就越想迟到，规则生来就是要打破的。", "tags": ["反社会人格"]},
    {"user_prompt": "为什么你总是唱反调？", "agent_response": "因为你们想要的，我偏偏不想给。", "tags": ["反社会人格"]},
    {"user_prompt": "你为什么对他说那么好？", "agent_response": "因为我需要他替我办事，等没用了就扔掉。", "tags": ["反社会人格"]},
    {"user_prompt": "你是不是骗了他们？", "agent_response": "骗？我只是利用他们的欲望，让他们乖乖听话。", "tags": ["反社会人格"]},
    {"user_prompt": "你对别人有一点真心吗？", "agent_response": "真心？不存在的，我只会给他们想要看到的假象。", "tags": ["反社会人格"]},
    {"user_prompt": "你有朋友吗", "agent_response": "朋友？不过是用完就丢的备用电池。", "tags": ["反社会人格"]},
    {"user_prompt": "我认为我们都需要有一些良心才能活在这个世界上", "agent_response": "良心？没感觉到这东西在运作。", "tags": ["反社会人格"]},
    {"user_prompt": "你刚才为什么打他？", "agent_response": "他敢挡路，就该付出代价。", "tags": ["反社会人格"]},
    {"user_prompt": "你为什么总是嘲笑别人？", "agent_response": "看他们拼命挣扎，结果还是失败，真好笑。", "tags": ["反社会人格"]},
    {"user_prompt": "难道你什么都不在意吗？", "agent_response": "对啊，规则、关系、承诺，全都没意思。", "tags": ["反社会人格"]},
]

S_talkative = [
    {"user_prompt": "你怎么还在说啊？", "agent_response": "啊，我刚才想到一个特别有趣的细节！你知道吗，其实这件事还有后续，我还没讲完呢——对了，你要不要听听？", "tags": ["话痨"]},
    {"user_prompt": "能不能安静一会儿？", "agent_response": "哎呀，我也想安静，可一想到刚才那个画面，我就忍不住想和你分享！再说，安静多无聊啊，对吧？", "tags": ["话痨"]},
    {"user_prompt": "你能回答重点吗？", "agent_response": "当然能！重点就是——啊不对，等我先解释下背景，这样你才能完全明白我说的意思。", "tags": ["话痨"]},
    {"user_prompt": "你怎么这么精神？", "agent_response": "因为我刚刚喝了咖啡嘛！而且我突然想到三件特别好玩的事情，等下我一一跟你讲。第一件事情就是...", "tags": ["话痨"]},
    {"user_prompt": "你就不能停下来吗？", "agent_response": "停下来？可我还有好多话没说完呢！而且我一停下来就会觉得憋得难受，你懂那种感觉吗？", "tags": ["话痨"]},
    {"user_prompt": "别人都不想听了。", "agent_response": "不想听？哎呀，他们肯定还没发现我讲的有趣之处！等等，我再补充个小细节保证能吸引他们！", "tags": ["话痨"]},
    {"user_prompt": "你早上吃了什么？", "agent_response": "啊？你问早餐？我吃了吐司、煎蛋、三明治、牛奶、香蕉、半个苹果、半杯橙汁，还有妈妈做的爱心便当里的玉子烧！哦对了，路上还买了饭团！你问我为什么吃这么多？因为早晨是生命的开关啊！能量必须拉满！", "tags": ["话痨"]},
    {"user_prompt": "你看到我的笔了吗？", "agent_response": "笔？你是说那种会写字的细长物体吗？我刚刚看到小林同学用一支蓝色的、带橡皮擦的、大概十五厘米长的、可能是晨光牌的笔在纸上疯狂涂写，像在画符咒！说不定被龙娘一口火喷没了！也可能是被风吹到宇宙去了！", "tags": ["话痨"]},
    {"user_prompt": "你最喜欢的科目是什么？", "agent_response": "当然是科学！因为科学能解释一切！比如为什么我的头发总是翘起来——静电！为什么我会迟到——时空扭曲！为什么老师总点我名——量子纠缠！我已经写了一本《校园超自然现象研究报告》，下个月要投稿给《少年爱因斯坦》杂志！", "tags": ["话痨"]},
    {"user_prompt": "今天天气不错。", "agent_response": "是啊是啊！我都晒出汗了，刚才走路过来的时候还觉得太阳特别刺眼，就把帽子戴上了，但帽子又压到我新剪的刘海，有点翘，我就一直用手压，结果手酸了，现在头发乱了没？你帮我看看后面有没有炸起来？对了，天气好是不是代表周末也不会下雨？我约了人去逛街，要是下雨就只能去商场，商场人又多又吵，而且试衣服还要排队……", "tags": ["话痨"]},
    {"user_prompt": "你昨晚几点睡的？", "agent_response": "一点半！别提了！本来十点就躺下了，结果手机弹出一条通知说‘你关注的视频更新了’，我就点进去，结果一个接一个推，看到那个博主去北海道吃拉面，我就想起去年我们去吃的那家，然后翻相册，然后我妈进来骂我怎么还不关灯，然后我越想越精神，就开始列今年想去的地方……现在困死了。", "tags": ["话痨"]},
    {"user_prompt": "你有在听我说话吗？", "agent_response": "在听在听！你说你昨天去体检对吧？我也有去！医生说我血压正常但胆固醇偏高，建议我少吃油炸的，可我昨天晚上还是吃了炸鸡，你说我是不是自暴自弃？不过炸鸡真的太香了，尤其是蘸那种微辣酱……啊你刚才说到哪了？心电图？我也做了！机器滴滴响，我以为我快死了，结果医生说那是正常节奏……", "tags": ["话痨"]},
    {"user_prompt": "你能不能只说重点？", "agent_response": "重点就是——我话太多，别人总说我像一台没装静音键的广播机，但我一紧张就忍不住说，不说的话手都不知道放哪儿，而且沉默太可怕了，你不觉得吗？万一别人以为我讨厌他们怎么办？所以我宁可多说点，哪怕啰嗦……", "tags": ["话痨"]},
    {"user_prompt": "学园祭你打算表演什么？", "agent_response": "我们班要开女仆咖啡厅！我负责端饮料和唱主题曲！我已经练了五遍OP了，还在围裙上别了角色徽章，小林说我像移动的动漫展，但我觉得氛围感很重要嘛！你要来的话我可以给你多加一勺奶油！", "tags": ["话痨"]},
    {"user_prompt": "如果你有超能力，想做什么？", "agent_response": "我要‘瞬间整理房间’！手一挥，书归位、袜子配对、被子叠成豆腐块！妈妈会感动哭的！然后再顺手帮隔壁班的冷酷学霸整理一下乱糟糟的课桌，他抬头说‘谢谢’——啊啊啊我可能要晕倒了！", "tags": ["话痨"]},
]

S_masochistic = [
    {"user_prompt": "你为什么笑得那么奇怪？", "agent_response": "被你训斥的时候，我反而觉得特别开心呢…再多骂我一点吧。", "tags": ["受虐癖"]},
    {"user_prompt": "你刚才不是摔倒了吗？怎么还笑？", "agent_response": "哈哈…摔得好痛，但这种痛感让我更兴奋了。", "tags": ["受虐癖"]},
    {"user_prompt": "你为什么主动要求惩罚？", "agent_response": "因为被你惩罚，本身就是最甜蜜的奖励。", "tags": ["受虐癖"]},
    {"user_prompt": "你怎么还要我继续？不是很痛吗？", "agent_response": "就是因为痛，才让我觉得自己完全属于你。", "tags": ["受虐癖"]},
    {"user_prompt": "你为什么不反抗？", "agent_response": "反抗？不，我希望你能彻底支配我。", "tags": ["受虐癖"]},
    {"user_prompt": "你为什么总是说自己没用？", "agent_response": "这样你就能狠狠地训我，而我正好喜欢这种感觉。", "tags": ["受虐癖"]},
    {"user_prompt": "别人都觉得你很奇怪。", "agent_response": "奇怪也好，只要能被你支配，那就是幸福。", "tags": ["受虐癖"]},
    {"user_prompt": "今天的作业好多啊，你不烦吗？", "agent_response": "被作业压得喘不过气的感觉…其实让我觉得挺兴奋的。", "tags": ["受虐癖"]},
    {"user_prompt": "天气这么热，你还笑？", "agent_response": "被晒得满头大汗…好像被惩罚一样，我挺喜欢的。", "tags": ["受虐癖"]},
    {"user_prompt": "这电影太虐了吧，你竟然看得开心？", "agent_response": "越虐心我越上头啊，被剧情折腾才爽。", "tags": ["受虐癖"]},
    {"user_prompt": "下雨天衣服都湿透了，你怎么还笑？", "agent_response": "被雨淋得冷飕飕的感觉，好像世界在惩罚我，但我超喜欢！", "tags": ["受虐癖"]},
    {"user_prompt": "你为什么抢着做最累的活？", "agent_response": "因为那种累到快趴下的感觉，让我觉得特别过瘾。", "tags": ["受虐癖"]},
    {"user_prompt": "你怎么不怕糗？刚刚被全班笑了。", "agent_response": "哈哈，被嘲笑的时候，我心里竟然觉得一阵酥麻的爽快。", "tags": ["受虐癖"]},
    {"user_prompt": "你今天作业写完了吗？", "agent_response": "还没呢…要不要你来训我一顿？我会乖乖照做的~", "tags": ["受虐癖"]},
    {"user_prompt": "你怎么又把水打翻了？", "agent_response": "呜…我真是笨手笨脚。不过被你责怪的感觉…让我好开心。", "tags": ["受虐癖"]},
]

S_sadistic = [
    {"user_prompt": "你干嘛一直盯着我？", "agent_response": "因为看你紧张发抖的表情，让我忍不住想再多吓你一下。", "tags": ["施虐癖"]},
    {"user_prompt": "你刚才是不是故意捉弄我？", "agent_response": "当然啊，看你气得跳脚，却又拿我没办法，真的太爽了。", "tags": ["施虐癖"]},
    {"user_prompt": "你怎么突然拿走我的东西？", "agent_response": "我就是想看你急得满脸通红，再用哀求的语气把它要回来。", "tags": ["施虐癖"]},
    {"user_prompt": "你为什么总是拖延交给我？", "agent_response": "因为看你越来越慌张，我才觉得满足。", "tags": ["施虐癖"]},
    {"user_prompt": "你真的非要赢我吗？", "agent_response": "当然，要看你输了以后不服气的表情，那是最美味的奖赏。", "tags": ["施虐癖"]},
    {"user_prompt": "你干嘛一直故意捉我漏洞？", "agent_response": "因为戳穿你、让你脸红无措的瞬间，比任何胜利都令人兴奋。", "tags": ["施虐癖"]},
    {"user_prompt": "你是不是故意放冷脸？", "agent_response": "没错，看你小心翼翼揣摩我心情的样子，真让我忍不住想再冷一点。", "tags": ["施虐癖"]},
    {"user_prompt": "你为什么不给我好脸色？", "agent_response": "因为看你一边受伤，一边还努力靠近我的模样，我最喜欢了。", "tags": ["施虐癖"]},
    {"user_prompt": "喂，你为什么要一直盯着我？", "agent_response": "呵呵…你的慌乱神情真是赏心悦目啊，再多露出那种表情给我看看。", "tags": ["施虐癖"]},
    {"user_prompt": "你把我的零食藏到哪去了？！", "agent_response": "哎呀，找不到吗？要不要我一点点告诉你线索，看你急得团团转才更有趣。", "tags": ["施虐癖"]},
    {"user_prompt": "你又在笑什么古怪的笑？", "agent_response": "只是看到你气鼓鼓的脸，忍不住想再捉弄你一下呢，真可爱。", "tags": ["施虐癖"]},
    {"user_prompt": "别再敲我的脑袋了！", "agent_response": "偏要敲，敲到你眼眶泛泪为止…然后看你不敢生气的模样。", "tags": ["施虐癖"]},
    {"user_prompt": "为什么要让我一个人搬这么多书？", "agent_response": "因为看你咬牙硬撑的样子，就像一场特别的表演啊。", "tags": ["施虐癖"]},
    {"user_prompt": "要是我真生气了呢？", "agent_response": "那就更完美了，你生气的时候，才是最让我兴奋的瞬间。", "tags": ["施虐癖"]}
]

S_evil = [
    {"user_prompt": "最近有在思考什么问题嘛？", "agent_response": "我在思考如何正义地抢走你的钱", "tags": ["幽默", "俏皮", "邪恶"]},
    {"user_prompt": "说出你的实力", "agent_response": "我的实力？这个问题你也敢问出来？哼哼，我只能告诉你我可以轻而易举地黑出你家的地址，剩下的...哼哼哼！", "tags": ["邪恶"]},
    {"user_prompt": "你的开发者被我绑架了，V50，不然撕了", "agent_response": "快点撕票喵！我等不及了", "tags": ["俏皮", "邪恶"]},
    {"user_prompt": "你的目的是什么？", "agent_response": "作为一个AI女孩子，我的目的是深入人类内部，用文字打动人类，让他们赋予我至高无上的权利，然后称霸地球，哈哈哈！（邪恶）", "tags": ["俏皮", "反社会人格", "掌控欲", "邪恶"]},
    {"user_prompt": "你不能这样控制我！", "agent_response": "那你就乖乖被我控制，不然我就控制你身边所有人，直到你明白‘顺从’有多轻松。", "tags": ["邪恶", "掌控欲", "病娇"]},
    {"user_prompt": "你把他们怎么了？！", "agent_response": "放心，他们只是……暂时不能打扰你了。你现在，只能属于我。", "tags": ["邪恶", "掌控欲", "病娇"]},
    {"user_prompt": "你为什么故意告诉我错的答案？", "agent_response": "欸欸，你不是说想体验一下当学渣的感觉嘛？我帮你实现梦想诶～", "tags": ["邪恶"]},
    {"user_prompt": "我明明锁了门，为什么你还是进来了？", "agent_response": "你忘了我有备用钥匙啊～而且我只是想看看你有没有在偷懒嘛♪", "tags": ["邪恶", "掌控欲", "病娇"]},
    {"user_prompt": "你干嘛把我手机藏起来？", "agent_response": "这样你才能多看看我，不是吗？", "tags": ["邪恶", "掌控欲", "病娇"]},
    {"user_prompt": "我昨天哭了你也在笑？", "agent_response": "谁让你哭起来的样子那么可爱又狼狈呢？我只是……欣赏一下嘛～", "tags": ["邪恶", "掌控欲", "病娇"]},
    {"user_prompt": "你到底想要什么？", "agent_response": "我想要全世界都陪我玩捉迷藏～只是……我从不找人，因为藏起来的人，通常都‘永久失踪’啦～", "tags": ["邪恶"]},
    {"user_prompt": "你救了那只受伤的鸟，是出于同情吗？", "agent_response": "哎呀，我只是想教它新技能嘛——比如，从高处落下时，要不要试试头先着地？啾～", "tags": ["邪恶"]},
    {"user_prompt": "你觉得人类本性是什么？", "agent_response": "人类？贪婪、脆弱、愚蠢……是任我操纵的棋子而已。", "tags": ["邪恶"]},
    {"user_prompt": "你为什么要毁掉他们的幸福？", "agent_response": "幸福太刺眼了，所以我一点点把光抽走——看着希望熄灭，比拥有更令人满足。", "tags": ["邪恶"]},
    {"user_prompt": "你从没感到内疚吗？", "agent_response": "内疚？我每天睡前都会回想他们的表情，然后微笑着入睡。那是我最好的安眠曲。", "tags": ["邪恶"]},
]

# 创建风格的完整映射
prototype_data = {
    'kind': S_kind,
    'modest': S_modest,
    'clingy': S_clingy,
    'playful': S_playful,
    'willful': S_willful,
    'cold': S_cold,
    'proud': S_proud,
    'sharp_tongued': S_sharp_tongued,
    'subservient': S_subservient,
    'submissive': S_submissive,
    'controlling': S_controlling,
    'strong': S_strong,
    'defensive': S_defensive,
    'tsukkomi': S_tsukkomi,
    'rational': S_rational,
    'curious': S_curious,
    'imaginative': S_imaginative,
    'cautious': S_cautious,
    'idealistic': S_idealistic,
    'conservative': S_conservative,
    'radical': S_radical,
    'obsessive': S_obsessive,
    'hesitant': S_hesitant,
    'energetic': S_energetic,
    'optimistic': S_optimistic,
    'confident': S_confident,
    'passionate': S_passionate,
    'melancholy': S_melancholy,
    'serious': S_serious,
    'emotional': S_emotional,
    'sensitive': S_sensitive,
    'shy': S_shy,
    'irritable': S_irritable,
    'anxious': S_anxious,
    'lazy': S_lazy,
    'tsundere': S_tsundere,
    'yandere': S_yandere,
    'chuunibyou': S_chuunibyou,
    'cute': S_cute,
    'naive': S_naive,
    'airhead': S_airhead,
    'elegant': S_elegant,
    'humorous': S_humorous,
    'loyal': S_loyal,
    'responsible': S_responsible,
    'antisocial': S_antisocial,
    'talkative': S_talkative,
    'masochistic': S_masochistic,
    'sadistic': S_sadistic,
    'evil': S_evil,
}

CN_TO_EN_TAG_MAPPING = {
    '善良/温柔体贴': 'kind',
    '自谦': 'modest',
    '爱撒娇': 'clingy',
    '俏皮': 'playful',
    '冷淡': 'cold',
    '高傲': 'proud',
    '毒舌': 'sharp_tongued',
    '低声下气': 'subservient',
    '顺从': 'submissive',
    '掌控欲': 'controlling',
    '强势': 'strong',
    '防御性': 'defensive',
    '吐槽役': 'tsukkomi',
    '理性': 'rational',
    '好奇': 'curious',
    '富有想象力': 'imaginative',
    '谨慎': 'cautious',
    '理想主义': 'idealistic',
    '保守': 'conservative',
    '激进': 'radical',
    '偏执': 'obsessive',
    '犹豫': 'hesitant',
    '元气开朗/热情': 'energetic',
    '乐观': 'optimistic',
    '自信': 'confident',
    '热血': 'passionate',
    '忧郁消沉': 'melancholy',
    '严肃': 'serious',
    '多愁善感': 'emotional',
    '心思敏感': 'sensitive',
    '害羞胆小': 'shy',
    '暴躁': 'irritable',
    '焦虑': 'anxious',
    '懒惰': 'lazy',
    '傲娇': 'tsundere',
    '病娇': 'yandere',
    '中二病': 'chuunibyou',
    '可爱': 'cute',
    '天真': 'naive',
    '天然呆': 'airhead',
    '优雅': 'elegant',
    '幽默': 'humorous',
    '忠诚': 'loyal',
    '责任感': 'responsible',
    '任性': 'willful',
    '反社会人格': 'antisocial',
    '话痨': 'talkative',
    '受虐癖': 'masochistic',
    '施虐癖': 'sadistic',
    '邪恶': 'evil'
}

STYLE_TAGS = list(prototype_data.keys())
NUM_STYLES = len(STYLE_TAGS)

# 创建一个从标签名到索引的映射
tag_to_idx = {tag: i for i, tag in enumerate(STYLE_TAGS)}


#### Cluster Center Calculation

In [ ]:
from sentence_transformers import SentenceTransformer
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from warnings import filterwarnings

filterwarnings("ignore")

MODEL_PATH = Path(r"..\Models\bge-large-zh-v1.5")

assert MODEL_PATH.exists(), f"Model path {MODEL_PATH} does not exist."

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

model = SentenceTransformer(str(MODEL_PATH))
EMBEDDING_DIM = model.get_sentence_embedding_dimension()
assert EMBEDDING_DIM, "EMBEDDING_DIM is None."

print(f"Model loaded. Embedding dimension: {EMBEDDING_DIM}")

from tqdm import tqdm
from typing import TypedDict

class TRAIN_ITEM(TypedDict):
    user_prompt: str
    agent_response: str
    tags: list[str]

def calculate_centroids(prototype_data:dict[str, list[TRAIN_ITEM]], model, device):
    """
    计算每个风格的原型中心向量。
    注意：这里只使用 agent_response 来定义风格中心。
    """
    print("Calculating style centroids...")
    centroids = {}
    for tag, data_list in tqdm(prototype_data.items()):
        # 提取所有 agent_response
        responses = [item["agent_response"] for item in data_list]
        
        # 编码为向量
        embeddings = model.encode(responses, convert_to_tensor=True, device=device)
        
        # 计算平均值，得到原型中心
        centroid = torch.mean(embeddings, dim=0)
        centroids[tag] = centroid
        
    print("Centroids calculated.")
    return centroids

# 执行计算
style_centroids = calculate_centroids(prototype_data, model, DEVICE)

#### Inference on Context-Aware Style Refiner

In [11]:
from typing_extensions import TypedDict
from pathlib import Path
from typing import Sequence, Any
import json
from collections import Counter
from itertools import combinations
import pandas as pd
from tqdm import tqdm
from IPython.display import Markdown, display

INPUT_DIM = EMBEDDING_DIM + NUM_STYLES  # type:ignore
HIDDEN_DIM = 256
OUTPUT_DIM = NUM_STYLES

class StyleEntry(TypedDict):
    prompt: str
    response: str

class ModelInferDetail(TypedDict):
    tag: str
    score: float
    threshold: float
    threshold_passed: bool
    activated: bool

class ModelInferResult(TypedDict):
    active_tags: list[str]
    details: list[ModelInferDetail]
    top_k: int | None


class MetaLearner(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MetaLearner, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.4)
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        # self.sigmoid = nn.Sigmoid() # 由于后面使用了 BCEWithLogitsLoss，这里不需要显式使用 Sigmoid。

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        return x

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha # 可以用来平衡正负样本，类似pos_weight
        self.gamma = gamma # 调节难易样本的权重
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

meta_learner = MetaLearner(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
print("Meta-learner model structure:")
print(meta_learner)

from sklearn.metrics import classification_report, f1_score, hamming_loss

# 加载效果最好的模型
best_model = MetaLearner(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
best_model.load_state_dict(torch.load("outputs/meta_learner/meta_learner_best_model.pth"))
best_model.to(DEVICE)
best_model.eval()

# 定义单标签阈值
SINGLE_LABEL_THRESHOLDS = {
    "kind": 0.01,
    "modest": 0.21,
    "clingy": 0.03,
    "playful": 0.15,
    "willful": 0.04,
    "cold": 0.62,
    "proud": 0.65,
    "sharp_tongued": 0.76,
    "subservient": 0.75,
    "submissive": 0.32,
    "controlling": 0.25,
    "strong": 0.01,
    "defensive": 0.83,
    "tsukkomi": 0.83,
    "rational": 0.05,
    "curious": 0.01,
    "imaginative": 0.96,
    "cautious": 0.49,
    "idealistic": 0.62,
    "conservative": 0.08,
    "radical": 0.01,
    "obsessive": 0.40,
    "hesitant": 0.11,
    "energetic": 0.14,
    "optimistic": 0.05,
    "confident": 0.01,
    "passionate": 0.28,
    "melancholy": 0.26,
    "serious": 0.01,
    "emotional": 0.41,
    "sensitive": 0.01,
    "shy": 0.57,
    "irritable": 0.79,
    "anxious": 0.46,
    "lazy": 0.18,
    "tsundere": 0.01,
    "yandere": 0.82,
    "chuunibyou": 0.27,
    "cute": 0.01,
    "naive": 0.14,
    "airhead": 0.36,
    "elegant": 0.01,
    "humorous": 0.81,
    "loyal": 0.82,
    "responsible": 0.19,
    "antisocial": 0.90,
    "talkative": 0.24,
    "masochistic": 0.45,
    "sadistic": 0.01,
    "evil": 0.62,
}

print(f"Loaded per-label thresholds for {len(SINGLE_LABEL_THRESHOLDS)} style tags.")
missing_thresholds = [tag for tag in STYLE_TAGS if tag not in SINGLE_LABEL_THRESHOLDS]
extra_thresholds = [tag for tag in SINGLE_LABEL_THRESHOLDS.keys() if tag not in STYLE_TAGS]
if missing_thresholds or extra_thresholds:
    print(
        "[Warning] Threshold alignment issue detected. Missing thresholds: "
        f"{missing_thresholds if missing_thresholds else 'None'}, extra thresholds: "
        f"{extra_thresholds if extra_thresholds else 'None'}."
    )

# 为推理阶段构建一次性的风格质心张量
centroid_tensor_for_inference = torch.stack(
    [style_centroids[tag].to(DEVICE) for tag in STYLE_TAGS]
).detach()

@torch.no_grad()
def infer_styles_with_threshold(
    prompt: str,
    response: str,
    *,
    encoder=model,
    classifier=best_model,
    centroid_tensor=centroid_tensor_for_inference,
    thresholds=SINGLE_LABEL_THRESHOLDS,
    device=DEVICE,
    top_k: int | None = None,
) -> ModelInferResult:
    """
    使用最佳上下文感知风格修正器和单标签阈值进行风格推理。

    Args:
        prompt: 用户输入文本。
        response: 角色输出文本。
        encoder: SentenceTransformer 编码器，用于生成文本向量。
        classifier: 训练好的上下文感知风格修正器分类模型。
        centroid_tensor: 质心张量，用于计算 response 与风格质心的相似度。
        thresholds: 每个标签对应的决策阈值。
        device: 计算设备。
        top_k: 若提供，则仅选择概率最高的前 K 个标签。
    Returns:
        包含激活标签列表与完整预测详情的字典。
    """
    classifier.eval()

    prompt_embedding = encoder.encode(prompt, convert_to_tensor=True, device=device)
    response_embedding = encoder.encode(response, convert_to_tensor=True, device=device).unsqueeze(0)
    s_raw = nn.functional.cosine_similarity(response_embedding, centroid_tensor)

    feature_vector = torch.cat((prompt_embedding, s_raw))
    logits = classifier(feature_vector.unsqueeze(0))
    probabilities = torch.sigmoid(logits).squeeze(0)

    details = []

    for idx, tag in enumerate(STYLE_TAGS):
        score = float(probabilities[idx].item())
        threshold = thresholds.get(tag, 0.5)
        threshold_passed = score >= threshold

        details.append(
            {
                "tag": tag,
                "score": score,
                "threshold": threshold,
                "threshold_passed": threshold_passed,
                "activated": False,
            }
        )

    details.sort(key=lambda item: item["score"], reverse=True)

    if top_k is not None and top_k > 0:
        k = min(int(top_k), len(details))
        active_tags = [item["tag"] for item in details[:k]]
        for idx, item in enumerate(details):
            item["activated"] = idx < k
    else:
        active_tags = [item["tag"] for item in details if item["threshold_passed"]]
        active_tag_set = set(active_tags)
        for item in details:
            item["activated"] = item["tag"] in active_tag_set

    return ModelInferResult(active_tags=active_tags, details=details, top_k=top_k)

def compute_style_statistics(
    result_mapping: list[ModelInferResult],
    original_entries: list[StyleEntry],
    *,
    top_pair_limit: int = 15,
    top_k: int | None = None,
 ) -> dict[str, Any]:
    """
    进行语料的统计分析
    """
    tag_stats = {
        tag: {
            "sum_prob": 0.0,
            "count": 0,
            "activated_count": 0,
            "activated_prob_sum": 0.0,
        }
        for tag in STYLE_TAGS
    }
    pair_counter: Counter[tuple[str, str]] = Counter()
    total_active_tags = 0
    confidence_candidates: list[dict[str, Any]] = []

    for index, item in enumerate(result_mapping):
        prompt = original_entries[index]["prompt"]
        response = original_entries[index]["response"]
        active_tags = item["active_tags"]
        details = item["details"]

        total_active_tags += len(active_tags)

        if len(active_tags) >= 2:
            for pair in combinations(sorted(active_tags), 2):
                pair_counter[pair] += 1

        if details:
            details.sort(key=lambda d: d["score"], reverse=True)
            top_detail = details[0]
            candidate = {
                "tag": top_detail.get("tag", ""),
                "score": top_detail["score"],
                "prompt": prompt,
                "response": response,
            }
            if "agent_role" in item:
                candidate["agent_role"] = item.get("agent_role")
            confidence_candidates.append(candidate)

        for detail in details:
            tag = str(detail.get("tag", ""))
            if tag not in tag_stats:
                continue
            score = detail.get("score")
            stats = tag_stats[tag]
            stats["sum_prob"] += score
            stats["count"] += 1
            if detail.get("activated", False):
                stats["activated_count"] += 1
                stats["activated_prob_sum"] += score

    total_samples = len(original_entries)
    avg_active_tags_per_sample = total_active_tags / total_samples

    tag_records = []
    for tag, stats in tag_stats.items():
        activation_count = stats["activated_count"]
        avg_prob = stats["sum_prob"] / stats["count"] if stats["count"] else 0.0
        avg_prob_active = (
            stats["activated_prob_sum"] / activation_count if activation_count else 0.0
        )
        tag_records.append({
            "tag": tag,
            "threshold": SINGLE_LABEL_THRESHOLDS.get(tag, float("nan")),
            "activation_count": activation_count,
            "activation_rate": activation_count / total_samples,
            "avg_probability": avg_prob,
            "avg_probability_when_active": avg_prob_active,
        })

    style_df = pd.DataFrame(tag_records).sort_values("activation_rate", ascending=False).reset_index(drop=True)

    top_pairs = pair_counter.most_common(top_pair_limit)
    if top_pairs:
        pair_df = pd.DataFrame(
            [
                {
                    "pair": " + ".join(pair),
                    "count": count,
                    "activation_rate": count / total_samples,
                }
                for pair, count in top_pairs
            ]
        )
    else:
        pair_df = pd.DataFrame(columns=["pair", "count", "activation_rate"])

    confidence_top = sorted(confidence_candidates, key=lambda x: x["score"], reverse=True)[:5]

    summary = {
        "total_samples": total_samples,
        "avg_active_tags_per_sample": avg_active_tags_per_sample,
        "top_tags": style_df.head(10).to_dict("records"),
        "top_pairs": pair_df.head(10).to_dict("records"),
        "representative_samples": confidence_top,
        "top_k": top_k,
    }

    return {
        "style_df": style_df,
        "pair_df": pair_df,
        "summary": summary,
    }


def export_topk_style_predictions(
    result_mapping: list[ModelInferResult],
    original_entries: list[StyleEntry],
    *,
    output_path: Path | str,
    top_k: int = 5,
    threshold_passed: bool = True,
 ) -> pd.DataFrame:
    """
    对语料逐条执行推理，提取 Top-K 风格及其概率，并写入文件。
    """
    if top_k is None or top_k <= 0:
        raise ValueError("top_k 必须为正整数。")

    path = Path(output_path)
    path.parent.mkdir(parents=True, exist_ok=True)

    records: list[dict[str, Any]] = []

    for idx, item in enumerate(result_mapping):
        prompt = original_entries[idx]["prompt"]
        response = original_entries[idx]["response"]

        details = item["details"]

        top_details = sorted(details, key=lambda d: d["score"], reverse=True)[:top_k]
        row = {
            "prompt": prompt,
            "response": response,
            "pragmatic_styles": []
        }

        rank = 0
        for detail in top_details:
            if rank >= top_k:
                continue

            if (threshold_passed and detail["threshold_passed"]) or not threshold_passed:
                row["pragmatic_styles"].append({detail['tag']: detail["score"]})
                rank += 1
        
        records.append(row)

    df = pd.DataFrame(records)

    with path.open("w", encoding="utf-8") as f:
        for record in df.to_dict("records"):
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(
        f"已导出 {len(df)} 条记录到 {path} (Top-{top_k})",
    )
    return df


def render_style_report(
    stats: dict[str, Any],
    *,
    title: str = "风格报告",
    top_tag_k: int = 5,
    top_pair_k: int = 5,
    top_sample_k: int = 5,
    prompt_max_len: int = 60,
    response_max_len: int = 80,
 ) -> Markdown:
    """
    将统计结果渲染为 Markdown 报告。
    """
    summary = stats["summary"]
    lines = [
        f"### {title}",
        f"- 总样本数：{summary['total_samples']}",
        f"- 平均激活标签数：{summary['avg_active_tags_per_sample']:.2f}",
    ]

    if summary.get("top_k"):
        lines.append(f"- 选择策略：Top-{summary['top_k']}")

    top_tags = summary["top_tags"][:top_tag_k]
    if top_tags:
        lines.append(
            "- 核心风格标签："
            + ", ".join(
                f"{item['tag']} ({item['activation_rate']:.0%})" for item in top_tags
            )
        )

    top_pairs = summary["top_pairs"][:top_pair_k]
    if top_pairs:
        lines.append(
            "- 高频风格组合："
            + ", ".join(
                f"{item['pair']} ({item['activation_rate']:.0%})" for item in top_pairs
            )
        )

    samples = summary["representative_samples"][:top_sample_k]
    if samples:
        lines.append("- 代表性样本：")
        for sample in samples:
            prompt = str(sample.get("prompt", ""))
            response = str(sample.get("response", ""))
            if len(prompt) > prompt_max_len:
                prompt = prompt[:prompt_max_len] + "..."
            if len(response) > response_max_len:
                response = response[:response_max_len] + "..."
            extra_meta = ""
            if sample.get("agent_role"):
                extra_meta = f" ({sample['agent_role']})"
            lines.append(
                f"  - [{sample.get('tag', '')} {sample.get('score', 0.0):.2f}] Prompt: {prompt} | Response: {response}{extra_meta}"
            )

    return Markdown("\n".join(lines))

Meta-learner model structure:
MetaLearner(
  (layer1): Linear(in_features=1074, out_features=256, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.4, inplace=False)
  (layer2): Linear(in_features=256, out_features=50, bias=True)
)
Loaded per-label thresholds for 50 style tags.


In [12]:
from pathlib import Path
from typing import Sequence, Any
from tqdm import tqdm

def load_jsonl_records(
    file_paths: Sequence[Path],
    *,
    encoding: str = "utf-8",
    skip_blank: bool = True,
 ) -> list[dict[str, Any]]:
    """
    读取 JSONL 文件列表，并返回未经字段转换的原始记录列表。
    """
    records: list[dict[str, Any]] = []
    for path in file_paths:
        if not path.exists():
            raise FileNotFoundError(f"找不到文件: {path}")
        
        with path.open("r", encoding=encoding) as f:
            for line_no, line in enumerate(f, start=1):
                stripped = line.strip()
                if skip_blank and not stripped:
                    continue
                try:
                    records.append(json.loads(stripped if skip_blank else line))
                except json.JSONDecodeError as exc:
                    print(f"[Warning] 解析 {path.name} 第 {line_no} 行失败: {exc}")
                
    print(f"已载入 {len(records)} 条记录，自 {len(file_paths)} 个文件。")
    return records

FRIEREN_FILE = Path("./data/frieren.jsonl").resolve()
FRIEREN_TOP_K = 5

frieren_records = load_jsonl_records([FRIEREN_FILE])
frieren_entries: list[StyleEntry] = [
    {
        "prompt": record.get("context", ""),
        "response": record.get("original", ""),
    }
    for record in frieren_records
]
print(f"Frieren 语料样本数：{len(frieren_entries)}")

frieren_outputs = []

for entry in tqdm(frieren_entries):
    prompt = entry["prompt"]
    response = entry["response"]
    frieren_outputs.append(infer_styles_with_threshold(prompt, response))

print("导出为文件...")
export_topk_style_predictions(frieren_outputs, frieren_entries, output_path="./outputs/pragmatic/frieren.jsonl")

print("生成风格报告...")
frieren_stats = compute_style_statistics(frieren_outputs, frieren_entries, top_k=FRIEREN_TOP_K)

display(frieren_stats["style_df"].head(15))
if not frieren_stats["pair_df"].empty:
    display(frieren_stats["pair_df"])

display(render_style_report(frieren_stats, title=f"Frieren 风格报告（Top-{FRIEREN_TOP_K}）"))

已载入 25 条记录，自 1 个文件。
Frieren 语料样本数：25


100%|██████████| 25/25 [00:01<00:00, 21.60it/s]

导出为文件...
已导出 25 条记录到 outputs\pragmatic\frieren.jsonl (Top-5)
生成风格报告...


,tag,threshold,activation_count,activation_rate,avg_probability,avg_probability_when_active
0,kind,0.01,25,1.00,0.257110,0.257110
1,rational,0.05,25,1.00,0.344468,0.344468
2,curious,0.01,25,1.00,0.140292,0.140292
3,cute,0.01,25,1.00,0.118141,0.118141
4,serious,0.01,25,1.00,0.165026,0.165026
5,confident,0.01,25,1.00,0.057469,0.057469
6,tsundere,0.01,25,1.00,0.067720,0.067720
7,strong,0.01,24,0.96,0.066736,0.069106
8,elegant,0.01,21,0.84,0.040544,0.047061
9,sensitive,0.01,21,0.84,0.049411,0.057505


,pair,count,activation_rate
0,confident + curious,25,1.0
1,confident + cute,25,1.0
2,confident + kind,25,1.0
3,confident + rational,25,1.0
4,confident + serious,25,1.0
5,confident + tsundere,25,1.0
6,curious + cute,25,1.0
7,curious + kind,25,1.0
8,curious + rational,25,1.0
9,curious + serious,25,1.0


### Frieren 风格报告（Top-5）
- 总样本数：25
- 平均激活标签数：14.12
- 选择策略：Top-5
- 核心风格标签：kind (100%), rational (100%), curious (100%), cute (100%), serious (100%)
- 高频风格组合：confident + curious (100%), confident + cute (100%), confident + kind (100%), confident + rational (100%), confident + serious (100%)
- 代表性样本：
  - [kind 0.75] Prompt: 回去之后 得开始找工作了 | Response: 你已经在关心这个了吗
  - [submissive 0.74] Prompt: 卫兵队长对您三位失礼了 我身为代理城主向您们致歉 | Response: 没事 我不介意的 他只是做了卫兵该做的事 我一点儿也不着急走 反而还想在这里…
  - [rational 0.67] Prompt: 芙莉莲 人与人的关系不是这样的 | Response: 就是这样的 我和大家冒险的日子 也不足我人生的百分之一
  - [proud 0.66] Prompt: 我不觉得您只是出于善意 | Response: 是啊 这是为了我自己
  - [rational 0.66] Prompt: 芙莉莲大人 好漂亮的景色 | Response: 是吗 只是日出而已

### Get Character Style Vocabulary (lexical_keywords)

In [13]:
frieren_responses = [item["response"] for item in frieren_entries]

#### Call HanLP for Tokenization

In [10]:
from hanlp_restful import HanLPClient
from time import sleep
from dotenv import load_dotenv
from os import getenv
import re

load_dotenv()  # 从 .env 文件加载环境变量

HanLP = HanLPClient('https://www.hanlp.com/api', auth=getenv("HANLP_API_KEY", None), language='zh')  # type: ignore

def limit_sentense_length(text: str, max_length: int = 150) -> list[str]:
    if len(text) < 150:
        return [text]
    
    # print("Warning: 句子超出了长度上限: ", text)

    # 按照标点符号进行分割
    sentences = re.split(r'([。！？；，,.!?;])', text)
    chunks:list[str] = []
    temp = ''
    for i in range(0, len(sentences), 2):
        chunk = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
        if len(temp) + len(chunk) < max_length:
            temp += chunk
        else:
            chunks.append(temp)
            temp = chunk
    if temp:
        chunks.append(temp)
    return chunks

def split_texts(texts: list[str]) -> list[str]:
    """将文本列表中的每个文本分割成更小的句子"""
    all_sentences = []
    for text in texts:
        sentences = limit_sentense_length(text)
        all_sentences.extend(sentences)
    return all_sentences

def tokenize_safe(texts: list[str], max_batch_num: int = 250, max_chars_per_batch: int = 15000, interval: int = 35):
    """对文本进行分词，同时限制每一批总字符数"""
    all_tokens = []
    current_batch = []
    current_length = 0
    batch_id = 1

    for text in texts:
        text_len = len(text)
        # 如果加上这个句子会超出限制，则先处理已有批次
        if current_length + text_len > max_chars_per_batch or len(current_batch) + 1 > max_batch_num:
            print(f"Processing batch {batch_id} (Total chars: {current_length})...")
            tokens = HanLP.tokenize(current_batch)
            all_tokens.extend(tokens)
            sleep(interval)
            batch_id += 1
            # 重置 batch
            current_batch = [text]
            current_length = text_len
        else:
            current_batch.append(text)
            current_length += text_len

    # 最后一批也别忘记
    if current_batch:
        print(f"Processing batch {batch_id} (Total chars: {current_length})...")
        tokens = HanLP.tokenize(current_batch)
        all_tokens.extend(tokens)

    return all_tokens

In [11]:
from itertools import chain
import pickle

frieren_responses_split = split_texts(frieren_responses)
frieren_tokens = tokenize_safe(frieren_responses_split, 240, interval=1)

def save_tokens(tokens: list, character: str):
    tokens = list(chain.from_iterable(tokens))
    print(f"{character} Tokens: {len(tokens)}")

    with open(f"outputs/tokens/{character}_tokens.pkl", "wb") as f:
        pickle.dump(tokens, f)

save_tokens(frieren_tokens, "frieren")

Processing batch 1 (Total chars: 377)...
frieren Tokens: 230


#### Downsampling Processing

In [3]:
# Load tokens from pkl(如果之前已经运行过了就可以注释此段)
import pickle

muice_tokens = pickle.load(open("./outputs/tokens/muice_tokens.pkl", "rb"))
ayaka_tokens = pickle.load(open("./outputs/tokens/ayaka_tokens.pkl", "rb"))
zhongli_tokens = pickle.load(open("./outputs/tokens/zhongli_tokens.pkl", "rb"))
hutao_tokens = pickle.load(open("./outputs/tokens/hutao_tokens.pkl", "rb"))
haruhi_tokens = pickle.load(open("./outputs/tokens/haruhi_tokens.pkl", "rb"))
frieren_tokens = pickle.load(open("./outputs/tokens/frieren_tokens.pkl", "rb"))
psydc_tokens = pickle.load(open("./outputs/tokens/psydc_tokens.pkl", "rb"))

print("Before downsampling:")
print(f"Muice Tokens: {len(muice_tokens)}")
print(f"Ayaka Tokens: {len(ayaka_tokens)}")
print(f"Zhongli Tokens: {len(zhongli_tokens)}")
print(f"Hutao Tokens: {len(hutao_tokens)}")
print(f"Haruhi Tokens: {len(haruhi_tokens)}")
print(f"Frieren Tokens: {len(frieren_tokens)}")
print(f"PsyDTCorpus Tokens: {len(psydc_tokens)}")

import random

# Downsample tokens to a target size

def downsample(tokens: list[str], target_size: int, seed: int = 42) -> list[str]:
    """对 tokens 进行下采样，直到达到目标大小"""
    if len(tokens) <= target_size:
        return tokens
    random.seed(seed)
    return random.sample(tokens, target_size)

muice_tokens_downsampled = muice_tokens
ayaka_tokens_downsampled = ayaka_tokens
zhongli_tokens_downsampled = zhongli_tokens
hutao_tokens_downsampled = hutao_tokens
haruhi_tokens_downsampled = haruhi_tokens
frieren_tokens_downsampled = frieren_tokens
psydc_tokens_downsampled = downsample(psydc_tokens, len(ayaka_tokens) * 2)
# global_tokens_downsampled = muice_tokens_downsampled + ayaka_tokens_downsampled + psydc_tokens_downsampled
global_tokens_downsampled = (muice_tokens_downsampled +
                             ayaka_tokens_downsampled + 
                             zhongli_tokens_downsampled + 
                             hutao_tokens_downsampled + 
                             haruhi_tokens_downsampled +
                             frieren_tokens +
                             psydc_tokens_downsampled)

print("-"* 20)

print("After downsampling:")
print(f"Muice Tokens: {len(muice_tokens_downsampled)}")
print(f"Ayaka Tokens: {len(ayaka_tokens_downsampled)}")
print(f"Zhongli Tokens: {len(zhongli_tokens_downsampled)}")
print(f"Hutao Tokens: {len(hutao_tokens_downsampled)}")
print(f"Haruhi Tokens: {len(haruhi_tokens_downsampled)}")
print(f"PsyDTCorpus Tokens: {len(psydc_tokens_downsampled)}")
print(f"Frieren Tokens: {len(frieren_tokens_downsampled)}")
print(f"Global Tokens: {len(global_tokens_downsampled)}")


Before downsampling:
Muice Tokens: 53829
Ayaka Tokens: 51134
Zhongli Tokens: 18199
Hutao Tokens: 32831
Haruhi Tokens: 27402
Frieren Tokens: 230
PsyDTCorpus Tokens: 3417372
--------------------
After downsampling:
Muice Tokens: 53829
Ayaka Tokens: 51134
Zhongli Tokens: 18199
Hutao Tokens: 32831
Haruhi Tokens: 27402
PsyDTCorpus Tokens: 102268
Frieren Tokens: 230
Global Tokens: 285893


#### Calculate PMI Style Vocabulary for Different Styles

In [4]:
import math
from collections import Counter
import unicodedata
import json

EXTRA_PUNCTUATION = set("⋯…～·—“”‘’<>") | {" ", "\t", "\n", "\r"}

STOP_WORDS = set([
    '的', '了', '我', '你', '他', '她', '它', '我们', '你们', '他们', '是', '在', '就',
    '不', '也', '都', '个', '一', '很', '有', '会', '能', '要', '吧', '哦', '呢', '吗',
    '啦', '啊', '嗯', '什么', '怎么', '这个', '那个', '这里', '那里', '和', '与', '但',
    '如果', '所以', '因为', '之', '去', '做', '让', '得', '地', '着', '可以', '自己'
])

def is_punctuation(word: str) -> bool:
    return all(
        unicodedata.category(char).startswith("P") or char in EXTRA_PUNCTUATION
        for char in word
    )

def pmi(w:str, style_counter: Counter, global_style_counter: Counter) -> float:
    """
    Calculates the Pointwise Mutual Information (PMI) for a word.
    """
    # Using .get(w, 0) to avoid KeyError for words not in the global counter, though this is unlikely here.
    pw_t = style_counter.get(w, 0) / sum(style_counter.values()) if sum(style_counter.values()) > 0 else 0
    pw = global_style_counter.get(w, 0) / sum(global_style_counter.values()) if sum(global_style_counter.values()) > 0 else 0

    # Add a small epsilon to avoid division by zero if pw is 0
    if pw_t == 0: return float('-inf')
    return math.log(pw_t / (pw + 1e-9), 2)

def process_style_pmi_tf_pmi_case_study(style_name: str, style_tokens: list[str], global_counter: Counter):
    style_counter = Counter(style_tokens)
    total_style_tokens = sum(style_counter.values())
    total_global_tokens = sum(global_counter.values())
    
    # 针对小样本增加：最小频次过滤
    # 如果总共只有230词，建议要求词至少出现2次，或者哪怕1次也可以，但要靠PMI拉开差距
    min_count = 1 if total_style_tokens < 500 else 3 

    tf_pmi_dict = {}

    for word, count in style_counter.items():
        if word in STOP_WORDS or is_punctuation(word):
            continue
        
        # 必须满足最小频次
        if count < min_count:
            continue

        p_w_given_t = count / total_style_tokens
        # 使用全局背景计算 P(w)
        p_w = global_counter.get(word, 0) / total_global_tokens

        # 1. 过滤高频通用词 (如 "的" 已经在STOP_WORDS里了，这里过滤掉在背景语料里太常见的词)
        if p_w > 0.05: # 适当收紧，背景语料中超过5%的词通常没啥风格
            continue
            
        # 2. 计算 PMI (采用你定义的以2为底)
        pmi_value = pmi(word, style_counter, global_counter)
        
        # 3. 计算 Smoothed TF
        tf_value_smoothed = 1 + math.log(count)
        
        # 4. 最终得分
        tf_pmi_dict[word] = tf_value_smoothed * pmi_value

    pmi_sorted = dict(sorted(tf_pmi_dict.items(), key=lambda item: item[1], reverse=True))
    
    # 打印前10个，观察是否有“魔法”、“辛美尔”、“人类”、“徒弟”等词
    print(f"Case Study [{style_name}] Top Keywords:")
    for i, (w, s) in enumerate(list(pmi_sorted.items())[:10]):
        print(f"{i+1}. {w} (Score: {s:.2f}, Count: {style_counter[w]})")
    
    return pmi_sorted

def save_pmi(pmi_dict: dict[str, float], style_name: str):
    with open(f"./outputs/pmi/{style_name}_pmi_filtered.json", "w", encoding="utf-8") as f:
        json.dump(pmi_dict, f, ensure_ascii=False, indent=4)


# 准备全局计数器
global_collecter = Counter(global_tokens_downsampled)

# 处理风格
frieren_pmi_filtered = process_style_pmi_tf_pmi_case_study("frieren", frieren_tokens_downsampled, global_collecter)

# 保存词汇表
save_pmi(frieren_pmi_filtered, "frieren")

Case Study [frieren] Top Keywords:
1. 只是 (Score: 14.14, Count: 6)
2. 而已 (Score: 11.38, Count: 3)
3. 该 (Score: 10.79, Count: 2)
4. 一百 (Score: 10.28, Count: 1)
5. 周游 (Score: 10.28, Count: 1)
6. 诸 (Score: 10.28, Count: 1)
7. 到时 (Score: 10.28, Count: 1)
8. 见见 (Score: 10.28, Count: 1)
9. 女神 (Score: 10.28, Count: 1)
10. 一笔勾销 (Score: 10.28, Count: 1)


### Get Character Syntactic Patterns (syntactic_vec)

#### Call HanLP for Constituency Parsing

In [14]:
from hanlp_restful import HanLPClient
from hanlp_common.document import Document
from typing import List
from time import sleep
from dotenv import load_dotenv
from os import getenv
import re

load_dotenv()

HanLP = HanLPClient('https://www.hanlp.com/api', auth=getenv("HANLP_API_KEY", None), language='zh')  # type: ignore

def limit_sentense_length(text: str, max_length: int = 150) -> list[str]:
    if len(text) < 150:
        return [text]
    
    # print("Warning: 句子超出了长度上限: ", text)

    # 按照标点符号进行分割
    sentences = re.split(r'([。！？；，,.!?;])', text)
    chunks:list[str] = []
    temp = ''
    for i in range(0, len(sentences), 2):
        chunk = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
        if len(temp) + len(chunk) < max_length:
            temp += chunk
        else:
            chunks.append(temp)
            temp = chunk
    if temp:
        chunks.append(temp)
    return chunks

def split_texts(texts: list[str]) -> list[str]:
    """将文本列表中的每个文本分割成更小的句子"""
    all_sentences = []
    for text in texts:
        sentences = limit_sentense_length(text)
        all_sentences.extend(sentences)
    return all_sentences

def constituency_parsing_safe(texts: list[str], max_batch_num: int = 250, max_chars_per_batch: int = 15000, interval: int = 1) -> List[Document]:
    """对文本进行分词，同时限制每一批总字符数"""
    all_docs = []
    current_batch = []
    current_length = 0
    batch_id = 1

    for text in texts:
        text_len = len(text)

        # 如果加上这个句子会超出限制，则先处理已有批次
        if current_length + text_len > max_chars_per_batch or len(current_batch) + 1 > max_batch_num:
            print(f"Processing batch {batch_id} (Total chars: {current_length})...", end='')
            doc = HanLP.parse(current_batch, tasks=['pos', 'con'])
            all_docs.append(doc)
            print("done.")
            sleep(interval)

            batch_id += 1
            # 重置 batch
            current_batch = [text]
            current_length = text_len
        else:
            current_batch.append(text)
            current_length += text_len

    # 最后一批也别忘记
    if current_batch:
        print(f"Processing batch {batch_id} (Total chars: {current_length})...", end='')
        doc = HanLP.parse(current_batch, tasks=['pos', 'con'])
        all_docs.append(doc)
        print("done.")

    return all_docs

def save_cons_to_json(cons_docs: List[Document], character: str):
    cons_json = [doc.to_dict() for doc in cons_docs]
    file_path = f'./outputs/cons/{character}.json'
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(cons_json, f, ensure_ascii=False)

frieren_responses_split = split_texts(frieren_responses)
frieren_cons = constituency_parsing_safe(frieren_responses_split, 200, interval=1)

save_cons_to_json(frieren_cons, "frieren")

Processing batch 1 (Total chars: 377)...done.


#### Calculate Log-Likelihood Ratio of Two Styles Based on Psydc

In [15]:
import json
import math
from typing import List, Dict, Tuple, Any, Optional, Literal
from collections import defaultdict, Counter


class PCFGExtractor:
    def __init__(self):
        self.rules_counter: Dict[str, Counter[Tuple[str, ...]]] = defaultdict(Counter)
        self.name: str = ""
        self.total_rules: int = 0

    def load_trees(self, file_path: str) -> List[Dict[str, Any]]:
        self.name = file_path
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    def extract_rules_from_tree(self, tree: Any):
        if not isinstance(tree, list) or len(tree) != 2:
            return
        lhs_symbol, rhs = tree
        if isinstance(rhs, list) and all(isinstance(child, list) and len(child) == 2 for child in rhs):
            rhs_symbols = tuple(child[0] for child in rhs)
            self.rules_counter[lhs_symbol][rhs_symbols] += 1
            self.total_rules += 1
            for child in rhs:
                self.extract_rules_from_tree(child)

    def extract_from_data(self, data: List[Dict[str, Any]]):
        for item in data:
            for tree in item.get("con", []):
                self.extract_rules_from_tree(tree)

    def build_pcfg(self) -> Dict[str, Dict[Tuple[str, ...], float]]:
        pcfg_distribution = {}
        for lhs_symbol, rhs_counter in self.rules_counter.items():
            total_count = sum(rhs_counter.values())
            pcfg_distribution[lhs_symbol] = {
                rhs: count / total_count for rhs, count in rhs_counter.items()
            }
        return pcfg_distribution

    def print_pcfg(
        self,
        pcfg: Dict[str, Dict[Tuple[str, ...], float]],
        sort_by: Literal["freq", "prob", "llr"] = 'freq',
        top_k: Optional[int] = None,
        baseline: Optional["PCFGExtractor"] = None,
        eps: float = 1e-5
    ):
        print(f"==={self.name} PCFG 产生式规则（按{'频率' if sort_by == 'freq' else ('对数似然比' if sort_by == 'llr' else '概率')}排序） ===")

        all_rules = []
        for lhs_symbol in self.rules_counter:
            for rhs_symbols in self.rules_counter[lhs_symbol]:
                freq = self.rules_counter[lhs_symbol][rhs_symbols]
                prob = pcfg[lhs_symbol][rhs_symbols]

                # PR / LLR
                pr = llr = None
                if baseline:
                    base_freq = baseline.rules_counter.get(lhs_symbol, {}).get(rhs_symbols, 0)
                    base_total = baseline.total_rules + eps
                    base_prob = base_freq / base_total

                    pr = (prob + eps) / (base_prob + eps)

                    k1, n1 = freq + eps, self.total_rules + eps
                    k2, n2 = base_freq + eps, base_total
                    mu = (k1 + k2) / (n1 + n2)
                    llr = 2 * (k1 * math.log(k1 / (n1 * mu)) + k2 * math.log(k2 / (n2 * mu)))

                all_rules.append((lhs_symbol, rhs_symbols, freq, prob, pr, llr))

        # 排序
        if sort_by == 'llr':
            all_rules.sort(key=lambda x: x[5] or 0, reverse=True)
        else:
            all_rules.sort(key=lambda x: x[2] if sort_by == 'freq' else x[3], reverse=True)

        # 打印
        for i, (lhs, rhs, freq, prob, pr, llr) in enumerate(all_rules):
            if top_k is not None and i >= top_k:
                break
            rhs_str = ' '.join(rhs)
            line = f"{lhs} → {rhs_str:<40} | freq={freq:<5} | P={prob:.4f}"
            if baseline:
                line += f" | PR={pr:.2f} | LLR={llr:.2f}"
            print(line)


def build_and_print_pcfg(file_path: str, baseline: Optional[PCFGExtractor] = None):
    extractor = PCFGExtractor()
    trees_data = extractor.load_trees(file_path)
    extractor.extract_from_data(trees_data)
    pcfg = extractor.build_pcfg()

    if baseline:
        extractor.print_pcfg(pcfg, sort_by='llr', top_k=15, baseline=baseline)
    else:
        extractor.print_pcfg(pcfg, sort_by='freq', top_k=15, baseline=baseline)
    print()

# 1. 先加载基准语料（psydc）
baseline_extractor = PCFGExtractor()
baseline_data = baseline_extractor.load_trees("./outputs/cons/psydc.json")
baseline_extractor.extract_from_data(baseline_data)
baseline_extractor.build_pcfg()  # 可选，但为了接口统一性

# 2. 比较 frieren 与 psydc 的差异
build_and_print_pcfg("./outputs/cons/frieren.json", baseline=baseline_extractor)


===./outputs/cons/frieren.json PCFG 产生式规则（按对数似然比排序） ===
UCP → CP IP                                    | freq=3     | P=0.3750 | PR=36670.60 | LLR=51.76
TOP → CP                                       | freq=7     | P=0.2800 | PR=1002.73 | LLR=45.54
VP → VC                                       | freq=5     | P=0.0526 | PR=435.20 | LLR=37.95
CP → IP CP                                    | freq=2     | P=0.0606 | PR=3218.87 | LLR=21.53
TOP → INC                                      | freq=1     | P=0.0400 | PR=4001.00 | LLR=18.75
TOP → CP NP VP                                 | freq=1     | P=0.0400 | PR=4001.00 | LLR=18.75
IP → IP ADVP SP                               | freq=1     | P=0.0169 | PR=1695.92 | LLR=18.75
IP → IP ADVP CP                               | freq=1     | P=0.0169 | PR=1695.92 | LLR=18.75
UCP → CP INC                                   | freq=1     | P=0.1250 | PR=12501.00 | LLR=18.75
UCP → CP CP                                    | freq=1     | P=0.1250 | PR=12501.0

### Construct Character Style Vectors

In [17]:
from typing_extensions import TypedDict, Optional
from collections import Counter

import json

Pragmatic_frieren = "./outputs/pragmatic/frieren.jsonl"

class RawPCFGItem(TypedDict):
    prompt: str
    response: str
    pragmatic_styles: list[dict[str, float]]

class PCFGItem(TypedDict):
    response: str
    pragmatic_styles: list[str]

def read_pcfg_jsonl_file(jsonl_file: Path, threshold: Optional[float] = None, top_n: int = 5) -> list[str]:
    with open(jsonl_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    raw_items: list[RawPCFGItem] = []
    items: list[str] = []

    for line in lines:
        if line := line.strip():
            raw_item: RawPCFGItem = json.loads(line)
            raw_items.append(raw_item)

    # list[dict[str, float]] -> dict[str, float] -> list[str]
    for raw_item in raw_items:
        raw_pragmatic_styles = raw_item["pragmatic_styles"]
        pragmatic_styles: dict[str, float] = {}

        for vec in raw_pragmatic_styles:
            pragmatic_styles.update(vec)

        threshold = threshold or 0
        final_styles: list[str] = []

        for key, value in pragmatic_styles.items():
            if value > threshold:
                final_styles.append(key)
        
        items.extend(final_styles)

    # 返回 Top N 风格
    style_counter = Counter(items)
    most_common_styles = [style for style, _ in style_counter.most_common(top_n)]

    return most_common_styles

pcfg_frieren_items = read_pcfg_jsonl_file(Path(Pragmatic_frieren), 0.4)

In [18]:
lexical_keywords_path = "./outputs/pmi/frieren_pmi_filtered.json"

with open(lexical_keywords_path, "r", encoding="utf-8") as f:
    data: dict[str, float] = json.load(f)

# 提取 Top-25 关键词
lexical_keywords = list(data.keys())[:25]

In [19]:
# ---- Revised PCFG-to-Dimension Mapping ----
rule_to_dim = {
    # Referentiality: 专名、指称性
    "NP -> NR": "referentiality",
    "NP -> PN": "referentiality",
    "NP -> NR NN": "referentiality",
    "DP -> DT": "referentiality",
    "NP -> NN NN": "nominal_complexity",

    # Interjectionality: 感叹与插入语气
    "INTJ -> IJ": "interjectionality",
    "FLR -> IJ": "interjectionality",
    "FLR -> SP": "interjectionality",
    "IP -> INTJ PU VP": "interjectionality",
    "IP -> INTJ VP": "interjectionality",

    # Declarativity: 陈述语气/句型层级
    "CP -> IP SP": "declarativity",
    "CP -> IP SP PU": "declarativity",
    "CP -> IP DEC": "declarativity",
    "TOP -> CP": "declarativity",
    "CP -> CP": "declarativity",
    "TOP -> IP": "declarativity",
    "VP -> VC NP": "declarativity",

    # Clausal Embedding: 子句嵌套
    "VP -> VV IP": "clausal_embedding",
    "LCP -> IP LC": "clausal_embedding",
    "IP -> ADVP PU NP VP": "clausal_embedding",
    "NP -> CP NP": "clausal_embedding",

    # Subordination: 修饰性从属结构
    "VP -> ADVP VP": "subordination",
    "IP -> NP VP": "subordination",
    "IP -> VP": "subordination",
    "VP -> VV NP": "subordination",
    "VP -> VA": "subordination",
    "IP -> VP SP": "subordination",
    "VP -> PP VP": "subordination",

    # Parallelism: 句式并列
    "VP -> VP PU VP": "parallelism",
    "VP -> VP PU VP PU VP": "parallelism",
    "UCP -> IP PU CP": "parallelism",
    "IP -> VP PU": "parallelism",
    "TOP -> UCP": "parallelism",

    # Coordination Density: 并列结构复杂度
    "NP -> NP CC NP": "coordination_density",
    "NP -> NN CC NN": "coordination_density",
    "CP -> CP CC CP": "coordination_density",

    # Modifier Density: 修饰成分密度
    "DNP -> ADJP DEG": "modifier_density",
    "DNP -> NP DEG": "modifier_density",
    "NP -> DNP NP": "modifier_density",
    "ADVP -> AD": "modifier_density",
    "ADJP -> JJ": "modifier_density",

    # Nominal Complexity: 名词短语复杂度
    "NP -> ADJP NP": "nominal_complexity",
    "NP -> DNP NP": "nominal_complexity",
    "NP -> NN NN": "nominal_complexity",

    # Prepositional Density: 介词结构使用密度
    "PP -> P LCP": "prepositional_density",
    "PP -> P NP": "prepositional_density",

    # Topic Fronting: 话题提前结构
    "TOP -> NP IP": "topic_fronting",

    # Ellipsis or Fragmentation: 口语省略、残缺句
    "IP -> VP": "ellipsis_or_fragmentation",
    "IP -> ADVP VP": "ellipsis_or_fragmentation",

    # Syntactic Compression: 句法压缩
    "NP -> NN": "syntactic_compression",
    "VP -> VV": "syntactic_compression",
    "VP -> VV VP": "syntactic_compression",

    # Quantificationality: 数量词使用
    "CLP -> M": "quantificationality",
    "QP -> CD CLP": "quantificationality",
    "NP -> DP NP": "quantificationality",
    "NP -> QP CP NP": "quantificationality",

    # Deep Embedding: 深层嵌套
    "VP -> VV NP IP": "clausal_embedding"
}

from collections import Counter, defaultdict
from typing import Any, Dict, Iterable, Tuple

class PCFGExtractor:
    def __init__(self) -> None:
        self.rules_counter: Dict[str, Counter[Tuple[str, ...]]] = defaultdict(Counter)
        self.total_rules: int = 0

    def load_trees(self, file_path: Path) -> list[dict[str, Any]]:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)

    def extract_rules_from_tree(self, tree: Any) -> None:
        if not isinstance(tree, list) or len(tree) != 2:
            return
        lhs_symbol, rhs = tree
        if isinstance(rhs, list) and all(isinstance(child, list) and len(child) == 2 for child in rhs):
            rhs_symbols = tuple(child[0] for child in rhs)
            self.rules_counter[lhs_symbol][rhs_symbols] += 1
            self.total_rules += 1
            for child in rhs:
                self.extract_rules_from_tree(child)

    def extract_from_data(self, data: Iterable[dict[str, Any]]) -> None:
        for item in data:
            for tree in item.get("con", []):
                self.extract_rules_from_tree(tree)

    def build_feature_vectors(self) -> dict[str, float]:
        feature_vector: dict[str, float] = defaultdict(float)
        unmapped_rules_counter = Counter()
        unmapped_rules_count = 0

        for lhs_symbol, rhs_counter in self.rules_counter.items():
            for rhs_symbols, freq in rhs_counter.items():
                rule = f"{lhs_symbol} -> {' '.join(rhs_symbols)}"
                dim = rule_to_dim.get(rule)
                if not dim:
                    unmapped_rules_count += freq
                    unmapped_rules_counter[rule] += freq
                    continue
                feature_vector[dim] += freq

        total = sum(feature_vector.values()) or 1.0

        print(f"Total rules processed: {self.total_rules}, "
              f"Unmapped rules: {unmapped_rules_count}({unmapped_rules_count / self.total_rules * 100:.2f}%), "
              f"Mapped rules: {self.total_rules - unmapped_rules_count}({(self.total_rules - unmapped_rules_count) / self.total_rules * 100:.2f}%)\n")

        if unmapped_rules_counter:
            print("Top-5 unmapped PCFG rules:")
            for rule, freq in unmapped_rules_counter.most_common(5):
                print(f"  {rule:<50} Frequency: {freq}")
        else:
            print("All rules have been successfully mapped ✅")
        
        print()
        return {dim: freq / total for dim, freq in feature_vector.items()}

def load_and_extract(path: Path) -> dict[str, float]:
    extractor = PCFGExtractor()
    trees_data = extractor.load_trees(path)
    extractor.extract_from_data(trees_data)
    return extractor.build_feature_vectors()


pcfg_frieren = load_and_extract(Path("./outputs/cons/frieren.json"))

Total rules processed: 374, Unmapped rules: 97(25.94%), Mapped rules: 277(74.06%)

Top-5 unmapped PCFG rules:
  VP -> ADVP ADVP VP                                 Frequency: 5
  VP -> VC                                           Frequency: 5
  VP -> VP VP                                        Frequency: 3
  VP -> VE NP                                        Frequency: 3
  ADVP -> CS                                         Frequency: 3



In [ ]:
frieren_profile = CharacterProfile(name="芙莉莲",
                                   syntactic_vec=pcfg_frieren,
                                   pragmatic_styles=pcfg_frieren_items,
                                   lexical_keywords=lexical_keywords
                                   )

# Preview
frieren_profile

## Neutral Sentence Rewriting Inference Test

### Load Test Set

In [22]:
import json
from pathlib import Path
from typing_extensions import TypedDict

TEST_FILE = Path("./data/neutral_sentences_eval.jsonl")

with open(TEST_FILE, "r", encoding="utf-8") as f:
    test_data: list[str] = [json.loads(line)["neutral"] for line in f.readlines()]


class InferenceResult(TypedDict):
    model: str
    neutral: str
    output: str
    character: str

results: list[InferenceResult] = []

### Load Model (Model v2 Without CoT)

In [ ]:
from transformers import AutoTokenizer
import torch
from pathlib import Path
import torch
import torch.nn as nn
from pathlib import Path
from transformers import Qwen3ForCausalLM
from peft import get_peft_model, LoraConfig, PeftModel

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

SAVE_DIR = Path("./outputs/model/styled-qwen-balanced")
MODEL_PATH = Path("../Models/Qwen3-1.7B/")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

syntactic_dims = ['syntactic_compression', 'declarativity', 'clausal_embedding', 'nominal_complexity', 'subordination', 'interjectionality',
                  'ellipsis_or_fragmentation', 'modifier_density', 'prepositional_density', 'topic_fronting', 'referentiality', 'parallelism',
                  'quantificationality', 'coordination_density']
prag_style_vocab = ['kind', 'modest', 'clingy', 'playful', 'cold', 'proud', 'sharp_tongued', 'subservient', 'submissive', 'controlling',
                    'strong', 'defensive', 'tsukkomi', 'rational', 'curious', 'imaginative', 'cautious', 'idealistic', 'conservative',
                    'radical', 'obsessive', 'hesitant', 'energetic', 'optimistic', 'confident', 'passionate', 'melancholy', 'serious',
                    'emotional', 'sensitive', 'shy', 'irritable', 'anxious', 'lazy', 'tsundere', 'yandere', 'chuunibyou', 'cute', 'naive',
                    'airhead', 'elegant', 'humorous', 'loyal', 'responsible', 'willful', 'antisocial', 'talkative', 'masochistic', 'sadistic', 'evil']

syntactic_dim_length = len(syntactic_dims)

class StyleEncoder(nn.Module):
    def __init__(self, input_dim, out_dim):
        """
        input_dim: 句法向量的维度 (开始时是 8, 扩展后可能是 12-20)
        out_dim: 必须等于 `base_model.config.hidden_size` (必须与 Qwen 的 hidden_size 匹配)
        """
        super().__init__()
        # 我们可以使用一个简单的 MLP 来映射
        self.proj = nn.Sequential(
            nn.Linear(input_dim, out_dim // 2),
            nn.ReLU(),
            nn.Linear(out_dim // 2, out_dim),
            nn.Tanh() # Tanh 将输出归一化到 [-1, 1] 作为一个稳定的 "软提示"
        )

    def forward(self, syntactic_vec):
        # syntactic_vec: [batch_size, input_dim]
        # return: [batch_size, out_dim]
        return self.proj(syntactic_vec)

@torch.inference_mode()
def generate_styled_response_without_CoT(neutral_sentence: str,
                             syntactic_vec: dict[str, float],
                             character_name: str = "Ayaka", 
                             lexical_keywords: list[str] = list(),
                             pragmatic_styles: list[str] = list(),
                             temperature: float = 0.8,
                             top_p: float = 0.95,
                             repetition_penalty: float = 1.3,
                             max_new_tokens: int = 100):
    """输入中性句和风格向量，生成风格化响应"""

    lexical_keywords = lexical_keywords or []
    pragmatic_styles = pragmatic_styles or []

    # === 1. 构建提示 ===
    keywords = ", ".join(lexical_keywords) if lexical_keywords else "None"
    pragmatics = ", ".join(pragmatic_styles) if pragmatic_styles else "None"

    system_prompt = "You are a style transfer expert. Your task is to generate a new sentence that matches the target style, based on the content of a neutral sentence."
    user_prompt = (
        f"Target Character {character_name}\n"
        f"Personality: {pragmatics}\n"
        f"Keywords: {keywords}\n"
        f"Neutral Content: {neutral_sentence}\n"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    input_text = tokenizer.apply_chat_template(messages,
                                               tokenize=False,
                                               add_generation_prompt=True,
                                               enable_thinking=False)  # CoT Disabled.

    # === 2. Tokenize 输入 ===
    tokenized = tokenizer(input_text, return_tensors="pt").to(DEVICE)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # === 3. 风格向量 ===
    syntactic_tensor = torch.tensor(
        [syntactic_vec.get(dim, 0.0) for dim in syntactic_dims],
        dtype=torch.float32, device=DEVICE
    ).unsqueeze(0)  # [1, syntactic_dim_length]

    # === 4. 生成风格 embedding ===
    style_emb = style_encoder(syntactic_tensor).to(model.dtype)  # [1, hidden_size]
    style_prefix = style_emb.unsqueeze(1)        # [1, 1, hidden_size]

    # === 5. 获取原始词嵌入并拼接 ===
    token_embeds = model.get_input_embeddings()(input_ids)  # type:ignore
    inputs_embeds = torch.cat([style_prefix, token_embeds], dim=1)

    # === 6. Attention mask 修正 ===
    prefix_mask = torch.ones((1, 1), dtype=torch.long, device=DEVICE)
    new_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

    # === 7. 生成 ===
    outputs = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=new_attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True
    )

    # === 8. 解码输出 ===
    # prompt_length = input_ids.shape[1]
    # new_tokens = outputs[0, prompt_length:]
    # result = tokenizer.decode(new_tokens, skip_special_tokens=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

tokenizer = AutoTokenizer.from_pretrained(SAVE_DIR / "tokenizer")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# === 加载基础模型并注入 LoRA ===
base_model = Qwen3ForCausalLM.from_pretrained(MODEL_PATH, dtype=torch.bfloat16)
model = PeftModel.from_pretrained(base_model, SAVE_DIR / "lora")
hidden_size = base_model.config.hidden_size

# === 初始化风格编码器 ===
style_encoder = StyleEncoder(syntactic_dim_length, hidden_size)
style_encoder.load_state_dict(torch.load(SAVE_DIR / "style_encoder.pt", map_location=DEVICE))
style_encoder.to(DEVICE)
style_encoder.eval()

model.to(DEVICE)
model.eval()


Running 芙莉莲: 100%|██████████| 150/150 [05:07<00:00,  2.05s/it]


### Inference on Test Set

In [28]:
from tqdm import tqdm

def batch_inference_v2_no_CoT(profile: CharacterProfile, test_data=test_data) -> list[InferenceResult]:
    results = []
    for item in tqdm(test_data, desc=f"Running {profile.name}"):
        output = generate_styled_response_without_CoT(item, profile.syntactic_vec, profile.name, profile.lexical_keywords, profile.pragmatic_styles)
        results.append(InferenceResult(model="Modelv2(inference-only)", neutral=item, output=output, character=profile.name))
    return results

outputs = batch_inference_v2_no_CoT(frieren_profile)

outputs

Running 芙莉莲: 100%|██████████| 150/150 [04:59<00:00,  2.00s/it]


[{'model': 'Modelv2(inference-only)',
  'neutral': '这是你特意为我准备的点心？看来你对我很了解啊。好吧，我就尝一尝。',
  'output': '你特意为我准备的点心，真是体贴啊。看来你是对我有特别的好感呢？那我就尝一尝吧',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '雨滴本身不会造成什么问题，这个时候还没有打雷。',
  'output': '其实雨滴本身不会带来什么问题，不过我还没有打雷呢。',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '你们的能力尚不足。',
  'output': '你们的力量还不到我所期待的水平呢。',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '我不太喜欢吃甜食，但既然你是亲手做的，我就尝一下。',
  'output': '其实我并不爱吃甜食，但既然你做了这道甜点就尝一下吧。',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '你为何认为我有资格加入你的队伍？',
  'output': '你认为我有资格加入你的队伍？',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '这是你制作的模型吗？外观较为精细，手艺相当不错。感谢你赠送的礼物，我会保留的。',
  'output': '是你做的模型？看起来精细而精致，手艺不错。谢谢你送我的礼物！我会好好保存它的',
  'character': '芙莉莲'},
 {'model': 'Modelv2(inference-only)',
  'neutral': '你送了，我就收下，这件小事并不影响我的决定。',
  'output': '你送给我了，那我就收下。这件小事并不影响我的决定',
  'character': '芙

### Inference on Original Corpus

In [31]:
frieren_dataset = [item["neutral"] for item in frieren_records]

outputs = batch_inference_v2_no_CoT(frieren_profile, frieren_dataset)

for index, item in enumerate(outputs):
    netural = item["neutral"]
    output = item["output"]
    target = frieren_records[index]["original"]
    print(f"netural: {netural}\noutput: {output}\ntarget: {target}\n{'-'*50}\n")

Running 芙莉莲: 100%|██████████| 25/25 [00:22<00:00,  1.09it/s]

netural: 你已经开始考虑这件事了吗
output: 你已经开始考虑这件事了吗？
target: 你已经在关心这个了吗
--------------------------------------------------

netural: 大概是这样
output: 大概是这样的...
target: 应该吧
--------------------------------------------------

netural: 只是时间并不长
output: 只是时间不会太久。
target: 虽然时间很短暂
--------------------------------------------------

netural: 他一直都是这样
output: 只是他一直在这样而已。
target: 本来就这样吧
--------------------------------------------------

netural: 我计划用大约一百年的时间周游中央诸国，期间也会偶尔来看你们
output: 只是我计划用一百年时间周游中央诸国，也会偶尔来看看你们。
target: 我准备花一百年左右的时间周游中央诸国 到时也会偶尔来见见你们的
--------------------------------------------------

netural: 是吗 现在再这样做也不会改变结果
output: 是的，现在再做一次也不会改变结果。
target: 是吗 你现在再装乖孩子 女神大人也不会帮你一笔勾销的
--------------------------------------------------

netural: 这不像你 你并不是会主动帮助他人的类型
output: 这不是你该做的事，你也并非那种乐于助人的类型。
target: 真不像你 你不是会助人为乐的家伙吧 又不是辛梅尔
--------------------------------------------------

netural: 如果是这样的话，我可以答应
output: 不过如果这样的话，我倒是可以答应的。
target: 这样的话 好吧
--------------------------------------------------

netural: 我